##Importing all the libraries required

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Dense,AveragePooling2D,Dropout,Flatten,Dense,Input,Concatenate,GlobalAveragePooling2D
from google.colab.patches import cv2_imshow,cv2
import os
import pandas as pd
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from sklearn.metrics import confusion_matrix,classification_report
from google.colab import files
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau,TensorBoard,LambdaCallback
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.utils import class_weight
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from glob import glob
import shutil
import json
from json import JSONEncoder
import seaborn as sn
from sklearn.utils.class_weight import compute_sample_weight
#import tensorflow_addons as tfa
#from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
%pip install tensorflow_addons

In [ ]:


class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


## Mounting onto google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

#New implementation for model


Full code below .. dont run this


In [ ]:

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = AveragePooling2D(pool_size=(7,7))(x)
x = Flatten(name ="flatten")(x)
x = Dense(128,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model
for layer in base_model.layers:
  layer.trainable = False

model = Model(inputs = base_model.input,outputs = preds)


for layer in base_model.layers:
    layer.trainable = False

y = model.output
k_1 = Dense(1,name = 'topic_1')(y)
k_2 = Dense(1,name = 'topic_2')(y)
k_3 = Dense(1,name = 'topic_3')(y)
k_4 = Dense(1,name = 'topic_4')(y)
conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
flat_2 = Flatten(name = 'flat_c')(conc2)
#Adding the Dense layers now
topic = Dense(256,activation='relu')(flat_2)
topic = Dropout(0.2)(topic)
topic = Dense(64,activation='relu')(topic)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)

cent_model_1 = Model(inputs = base_model.input,outputs = final)

In [ ]:
cent_model_1.summary()

In [ ]:
train_meta = pd.read_csv('/content/gdrive/MyDrive/train_names.csv')
val_meta = pd.read_csv('/content/gdrive/MyDrive/val_names.csv')
test_meta = pd.read_csv('/content/gdrive/MyDrive/test_names.csv')
train_meta.columns = ['ID','Path']
train_met = train_meta.Path
val_meta.columns = ['ID','Path']
val_met = val_meta.Path
train_met = pd.DataFrame(train_met)
val_met = pd.DataFrame(val_met)
tv = [train_met,val_met]
tnv = pd.concat(tv)
tnv = tnv.sample(frac=1,random_state = 42)

class_names = np.array(sorted([dir1 for dir1 in os.listdir('/content/gdrive/MyDrive/data/test')]))
len(class_names)

def kfold_label_maker(meta):

  meta['PicName'] = meta['Path'].apply(lambda x: x.split('/')[-1])
  meta['Label'] = meta['Path'].apply(lambda x: x.split('/')[-2])
  #meta['Image'] = meta['Image'].str.replace('/content/gdrive/MyDrive','/data/s4133366')
  kfold = meta[['Path','Label']]
  return kfold

train = kfold_label_maker(tnv)
train = kfold_label_maker(tnv)

def get_label(file_path):
  # convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label


Y = train['Label']
X = train['Path']
num_folds = 4
batch_size = 64
skfold = StratifiedKFold(n_splits=num_folds, shuffle=True,random_state=42)
fold_no = 1
history_path = '/content/gdrive/MyDrive/history'
print('LOADING THE DATA ON TO TENSORS......')
save_dir = '/content/gdrive/MyDrive/saved_models'

for train_index, val_index in skfold.split(X,Y):
  training_data = train.iloc[train_index]
  valid_data = train.iloc[val_index]
  print(len(training_data))
  print(len(valid_data))
  train_data = training_data['Path']
  train_labels = training_data['Label']
  val_labels = valid_data['Label']
  val_data = valid_data['Path']
  train_tensor = tf.data.Dataset.from_tensors(train_data)
  val_tensor = tf.data.Dataset.from_tensors(val_data)
  train_tensor.shuffle(len(train_data))
  val_tensor.shuffle(len(val_data))
  print(train_tensor)
  tra_tens = train_tensor.unbatch().map(process_TL)
  val_tens = val_tensor.unbatch().map(process_TL)
  print(tra_tens,' ',tf.data.experimental.cardinality(tra_tens))
  train_cache = tra_tens.cache()
  val_cache = val_tens.cache()
  #normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
  val_batch = val_tens.batch(batch_size)
  train_batch= tra_tens.batch(batch_size)
  train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  print(train_ds)
  

  # class_weights = class_weight.compute_class_weight(
  #          'balanced',
  #           np.unique(ktrainGen.classes), 
  #           ktrainGen.classes)

  # train_class_weights = dict(enumerate(class_weights))

  ##############                                    #################
  #####                   TOPIC MODEL WITH 1 UNIT     #################
  ###################################################################
  ##############                                    #################
  ##############                                    ################# 

  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  x = base_model.output
  x = AveragePooling2D(pool_size=(7,7))(x)
  x = Flatten(name ="flatten")(x)
  x = Dense(128,activation="relu")(x)
  x = Dropout(0.5)(x)
  preds = Dense(1000,activation="softmax")(x)

  #Activating the model
  for layer in base_model.layers:
    layer.trainable = False

  model = Model(inputs = base_model.input,outputs = preds)


  for layer in base_model.layers:
      layer.trainable = False

  y = model.output
  k_1 = Dense(1,name = 'topic_1')(y)
  k_2 = Dense(1,name = 'topic_2')(y)
  k_3 = Dense(1,name = 'topic_3')(y)
  k_4 = Dense(1,name = 'topic_4')(y)
  conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
  flat_2 = Flatten(name = 'flat_c')(conc2)
  #Adding the Dense layers now
  topic = Dense(256,activation='relu')(flat_2)
  topic = Dropout(0.2)(topic)
  topic = Dense(64,activation='relu')(topic)
  topic = Dropout(0.2)(topic)
  final = Dense(24,activation='softmax')(topic)

  cent_model_1 = Model(inputs = base_model.input,outputs = final)
  lr = 1e-1
  num_epochs = 1
  opti = tf.keras.optimizers.Adadelta(learning_rate=lr, rho=0.95, epsilon=1e-07, name="Adadelta")
  cent_model_1.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

  callbacks = [
    ModelCheckpoint(
        
        filepath=os.path.join(save_dir,'top_m_n1_'+str(fold_no)),
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="accuracy",
        verbose=1,
        save_freq = 442,mode = 'max'
    ),
    EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    ),
    ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=3, min_lr=1e-5,verbose=1)
    ]

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = cent_model_1.fit(train_ds,epochs=num_epochs,callbacks=callbacks,verbose=1,workers=8,use_multiprocessing = False,shuffle=True)
  cent_model_1.save('/content/gdrive/MyDrive/saved_models/topic_1_neuro_kfold_'+str(fold_no))
  report = history.history
  np.save(os.path.join(history_path,'history'+str(fold_no)+'.npy'), history)
  print('Results saved to disk ...')
  print('Evaluating the model ...') 
  cent_model_1 = keras.models.load_model("/content/gdrive/MyDrive/saved_models/topic_1_neuro_kfold_"+str(fold_no))
  results = cent_model_1.evaluate(val_ds,workers = 4)
  results = dict(zip(cent_model_1.metrics_names,results))
  np.save(os.path.join(history_path,'results_'+str(fold_no)+'.npy'), results)
  



  # tf.keras.backend.clear_session()

  #fold_no += 1

  print('Going to next fold................')



In [ ]:
 def return_dataframes(file): 
  test_labels = pd.read_csv(file)
  df_list = []
  df_list_names = []
  class_names_meta = test_labels['Label'].unique()
  class_names_meta
  len(class_names_meta)
  sorted(class_names_meta)
  for class_name in class_names_meta:
    class_name_rep = class_name.replace(" ","_")
    class_name_rep = class_name_rep.replace("/","_")
    class_name_rep = class_name_rep.replace(" ","")
    class_name_rep = class_name_rep.replace(",_","_")
    class_name_rep = class_name_rep.replace(",","_")
    #print(class_name_rep)
    h = "df_" + str(class_name_rep)
    df_name = h
    
    vars()[h] = test_labels[test_labels['Label'] == class_name]
    print("Created: ",h,' ',len(vars()[h]))
    print()
    df_list.append(vars()[h])
    df_list_names.append(df_name)
  return df_list,df_list_names

  

stuff for small data set

In [ ]:
 def return_dataframes(file): 
  test_labels = pd.read_csv(file)
  df_list = []
  df_list_names = []
  class_names_meta = test_labels['Label'].unique()
  class_names_meta
  len(class_names_meta)
  sorted(class_names_meta)
  for class_name in class_names_meta:
    class_name_rep = class_name.replace(" ","_")
    class_name_rep = class_name_rep.replace("/","_")
    class_name_rep = class_name_rep.replace(" ","")
    class_name_rep = class_name_rep.replace(",_","_")
    class_name_rep = class_name_rep.replace(",","_")
    #print(class_name_rep)
    h = "df_" + str(class_name_rep)
    df_name = h
    
    vars()[h] = test_labels[test_labels['Label'] == class_name]
    print("Created: ",h,' ',len(vars()[h]))
    print()
    df_list.append(vars()[h])
    df_list_names.append(df_name)
  return df_list,df_list_names

In [ ]:
def get_small_ds(path):
  df,name = return_dataframes(path)
  
  df_Office_updated = df_Office.sample(3000)
  df_Transportation_updated = df_Transportation.sample(3000)
  df_Education_science_updated = df_Education_science.sample(3000)
  df_Pathways_updated = df_Pathways.sample(3000)
  df_Restaurant_Bar_updated = df_Restaurant_Bar.sample(3000)

  df_list = [df_Bedroom,df_Office_updated,df_Other_rooms,df_Living_room,df_Noise,df_Bathroom,df_Pathways_updated,df_Shop,df_Buildings,df_Others,df_Sport_fields,df_Kitchen,df_Garden,df_Balcony,df_Beach,df_Recreation,df_Museum,df_Hospital,df_Water,df_Forest_field_jungle,df_Mountains_hills_desert_sky,df_Transportation_updated,df_Education_science_updated,df_Restaurant_Bar_updated]
  train_df = pd.concat(df_list)
  len(train_df)

  tr_df = train_df 
  num_epochs = 10
  tr_df['Image'] = tr_df['Image'].str.replace('/data/s4133366/data','/content/gdrive/MyDrive/data')
  tr_df = shuffle(tr_df)
  return tr_df


In [ ]:
from sklearn.utils import shuffle
test_labels = pd.read_csv('/content/val.csv')
class_names_meta = test_labels['Label'].unique()
class_names_meta
len(class_names_meta)
sorted(class_names_meta)
for class_name in class_names_meta:
  class_name_rep = class_name.replace(" ","_")
  class_name_rep = class_name_rep.replace("/","_")
  class_name_rep = class_name_rep.replace(" ","")
  class_name_rep = class_name_rep.replace(",_","_")
  class_name_rep = class_name_rep.replace(",","_")
  #print(class_name_rep)
  h = "df_" + str(class_name_rep)
  vars()[h] = test_labels[test_labels['Label'] == class_name]
  print("Created: ",h,' ',len(vars()[h]))
df_Office_updated = df_Office.sample(1500)
df_Transportation_updated = df_Transportation.sample(1500)
df_Education_science_updated = df_Education_science.sample(1500)
df_Pathways_updated = df_Pathways.sample(1500)
df_Restaurant_Bar_updated = df_Restaurant_Bar.sample(1500)

df_list = [df_Bedroom,df_Office_updated,df_Other_rooms,df_Living_room,df_Noise,df_Bathroom,df_Pathways_updated,df_Shop,df_Buildings,df_Others,df_Sport_fields,df_Kitchen,df_Garden,df_Balcony,df_Beach,df_Recreation,df_Museum,df_Hospital,df_Water,df_Forest_field_jungle,df_Mountains_hills_desert_sky,df_Transportation_updated,df_Education_science_updated,df_Restaurant_Bar_updated]
train_df = pd.concat(df_list)
print(len(train_df))


#num_epochs = 10
tr_df['Image'] = tr_df['Image'].str.replace('/data/s4133366/data','/content/gdrive/MyDrive/data')
#tr_df = shuffle(tr_df)


In [ ]:
train_df = train_df.sample(frac=1)
len(train_df)
train_df.to_csv('val_short_collab.csv')

In [ ]:
train_df['Image'] = train_df['Image'].str.replace('/data/s4133366/data','/content/gdrive/MyDrive/data')
train_df.head()

##Topic model with small data_set

In [ ]:
###################
#Using small df####
###################
cache_file_t = '/content/gdrive/MyDrive/cache_train'
cache_file_v = '/content/gdrive/MyDrive/cache_val'
class_names = np.array(sorted([dir1 for dir1 in os.listdir('/content/gdrive/MyDrive/data/test')]))
num_folds = 4
batch_size = 64
skfold = StratifiedKFold(n_splits=num_folds, shuffle=True,random_state=42)
fold_no = 1

history_path = '/content/gdrive/MyDrive/history'
print('LOADING THE DATA ON TO TENSORS......')
save_dir = '/content/gdrive/MyDrive/saved_models'

def get_label(file_path):
  # convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label


Y = tr_df['Label']
X = tr_df['Image']

for train_index, val_index in skfold.split(X,Y):
  training_data = tr_df.iloc[train_index]
  valid_data = tr_df.iloc[val_index]
  print(len(training_data))
  print(len(valid_data))
  train_data = training_data['Image']
  train_labels = training_data['Label']
  val_labels = valid_data['Label']
  val_data = valid_data['Image']
  train_tensor = tf.data.Dataset.from_tensors(train_data)
  val_tensor = tf.data.Dataset.from_tensors(val_data)
  train_tensor.shuffle(len(train_data))
  val_tensor.shuffle(len(val_data))
  print(train_tensor)
  tra_tens = train_tensor.unbatch().map(process_TL)
  val_tens = val_tensor.unbatch().map(process_TL)
  print(tra_tens,' ',tf.data.experimental.cardinality(tra_tens))
  train_cache = tra_tens.cache(cache_file_t)
  val_cache = val_tens.cache(cache_file_v)
  #normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
  val_batch = val_tens.batch(batch_size)
  train_batch= tra_tens.batch(batch_size)
  train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  print(train_ds)
  

  # class_weights = class_weight.compute_class_weight(
  #          'balanced',
  #           np.unique(ktrainGen.classes), 
  #           ktrainGen.classes)

  # train_class_weights = dict(enumerate(class_weights))

  ##############
  ##############
  #####TOPIC MODEL
  ###########################################################
  ##############
  ##############

  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  x = base_model.output
  x = AveragePooling2D(pool_size=(7,7))(x)
  x = Flatten(name ="flatten")(x)
  x = Dense(128,activation="relu")(x)
  x = Dropout(0.5)(x)
  preds = Dense(1000,activation="softmax")(x)

  #Activating the model
  for layer in base_model.layers:
    layer.trainable = False

  model = Model(inputs = base_model.input,outputs = preds)


  for layer in base_model.layers:
      layer.trainable = False

  y = model.output
  k_1 = Dense(1,name = 'topic_1')(y)
  k_2 = Dense(1,name = 'topic_2')(y)
  k_3 = Dense(1,name = 'topic_3')(y)
  k_4 = Dense(1,name = 'topic_4')(y)
  conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
  flat_2 = Flatten(name = 'flat_c')(conc2)
  #Adding the Dense layers now
  topic = Dense(256,activation='relu')(flat_2)
  topic = Dropout(0.2)(topic)
  topic = Dense(64,activation='relu')(topic)
  topic = Dropout(0.2)(topic)
  final = Dense(24,activation='softmax')(topic)

  cent_model_1 = Model(inputs = base_model.input,outputs = final)
  lr = 1e-1
  num_epochs = 1
  opti = tf.keras.optimizers.Adadelta(learning_rate=lr, rho=0.95, epsilon=1e-07, name="Adadelta")
  cent_model_1.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

  callbacks = [
    ModelCheckpoint(
        
        filepath=os.path.join(save_dir,'top_m_n1_'+str(fold_no)),
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="categorical_accuracy",
        verbose=1,
        save_freq = 442,mode = 'max'
    ),
    EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    ),
    ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=3, min_lr=1e-5,verbose=1)
    ]

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = cent_model_1.fit(train_ds,epochs=num_epochs,callbacks=callbacks,verbose=1,workers=8,use_multiprocessing = False,shuffle=True)
  cent_model_1.save('/content/gdrive/MyDrive/saved_models/topic_1_neuro_kfold_'+str(fold_no))
  report = history.history
  np.save(os.path.join(history_path,'history'+str(fold_no)+'.npy'), report)
  print('Results saved to disk ...')
  print('Evaluating the model ...') 
  cent_model_1 = keras.models.load_model("/content/gdrive/MyDrive/saved_models/topic_1_neuro_kfold_"+str(fold_no))
  results = cent_model_1.evaluate(val_ds,workers = 4)
  results = dict(zip(cent_model_1.metrics_names,results))
  np.save(os.path.join(history_path,'results_'+str(fold_no)+'.npy'), results)
  



  tf.keras.backend.clear_session()

  fold_no += 1

  print('Going to next fold................')



In [ ]:
#data_set = get_small_ds('/content/train.csv')
for i, j in zip(name,df):
    i = pd.DataFrame(j)
    #print(len(i))
    print(vars()[j])


In [ ]:
/content/history1.npy

# Plotting


In [ ]:
train_list = ['history_3_1.npy','history_3_2.npy','history_3_3.npy','history_3_4.npy']

In [ ]:
read_train_1 = np.load('/content/history_3_1.npy',allow_pickle=True).item()
read_train_2 = np.load('/content/history_3_2.npy',allow_pickle=True).item()
read_train_3 = np.load('/content/history_3_3.npy',allow_pickle=True).item()
read_train_2 = np.load('/content/history_3_4.npy',allow_pickle=True).item()
read_test_1 = np.load('/content/results_3_1.npy',allow_pickle=True).item()

In [ ]:
read_train_1

In [ ]:
train_list = ['history_3_1.npy','history_3_2.npy','history_3_3.npy','history_3_4.npy']
for train in train_list:
  train = train.replace('.npy','')
  fold = 'read_train_fold_'+str(train)
  vars()[fold] = np.load('/content/history_3_1.npy',allow_pickle='TRUE').item()
  print(fold)


In [ ]:
read_train_fold_history_3_4

In [ ]:
a = unzip(read_test.keys())


## Use this declaration to save plots in file

In [ ]:
plot_dir = '/content/gdrive/MyDrive/plots'
#For peregrine
#plot_dir = '/data/s4133366/saved_models/history/plots'
def plot_metrics(history_path,results_path):
  read_train = np.load(history_path,allow_pickle=True).item()
  read_test = np.load(results_path,allow_pickle=True).item()
  history_path = os.path.split(history_path)[-1]
  get_fold = history_path[-5]
  history_path = history_path.replace('.npy','')
  if not os.path.isdir(plot_dir):
    print('Plot directory '+plot_dir+' does not exist at the moment. Creating ....')
    os.mkdir(plot_dir)
    print(plot_dir+' created ...')
  if not os.path.isdir(os.path.join(plot_dir,history_path)) :
    os.mkdir(os.path.join(plot_dir,history_path))
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path+' created...')
  else:
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path,' exists ...')  
  print('Adding the metric plots to ',folder_path)
  
  #summarize history for accuracy
  
  plt.plot(read_train['categorical_accuracy'])
  plt.axhline(read_test['categorical_accuracy'], color='g', linestyle='--')
  plt.plot(read_test['categorical_accuracy'])
  plt.title('model accuracy for fold_'+str(get_fold))
  plt.ylabel('Categorical Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'accuracy_'+str(history_path)+'.png'))
  plt.clf()
  
  # summarize history for loss
  
  plt.plot(read_train['loss'])
  plt.axhline(read_test['loss'], color='g', linestyle='--')
  plt.title('model loss for fold_'+str(get_fold))
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()
  plt.savefig(os.path.join(folder_path,'loss_'+str(history_path)+'.png'))
  plt.clf()
  
  #summarize history for Precision 
  plt.plot(read_train['precision'])
  plt.axhline(read_test['precision'], color='g', linestyle='--')
  plt.title('model precision for fold_'+str(get_fold))
  plt.ylabel('Precision')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'precision_'+str(history_path)+'.png'))
  plt.clf()
  
  #summarize history for Recall 
  
  plt.plot(read_train['recall'])
  plt.axhline(read_test['recall'], color='g', linestyle='--')
  plt.title('Model Recall for fold_'+str(get_fold))
  plt.ylabel('Recall')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'recall_'+str(history_path)+'.png'))
  plt.clf()

In [ ]:


shutil.rmtree('/content/gdrive/MyDrive/plots')

testing

In [ ]:
cache_file_t = '/content/gdrive/MyDrive/cache_train'
cache_file_v = '/content/gdrive/MyDrive/cache_val'
class_names = np.array(sorted([dir1 for dir1 in os.listdir('/content/gdrive/MyDrive/data/test')]))
num_folds = 4
batch_size = 64
skfold = StratifiedKFold(n_splits=num_folds, shuffle=True,random_state=42)
fold_no = 1

history_path = '/content/gdrive/MyDrive/history'
print('LOADING THE DATA ON TO TENSORS......')
save_dir = '/content/gdrive/MyDrive/saved_models'

def get_label(file_path):
  # convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label


Y = tr_df['Label']
X = tr_df['Image']
print(len(X),len(Y))

In [ ]:
for train_index, val_index in skfold.split(X,Y):
  training_data = tr_df.iloc[train_index]
  valid_data = tr_df.iloc[val_index]
  print('train:',len(training_data))
  print('val:',len(valid_data))
  train_data = training_data['Image']
  train_labels = training_data['Label']
  val_labels = valid_data['Label']
  val_data = valid_data['Image']
  print('train:',len(train_data))
  print('val:',len(valid_data))
  train_tensor = tf.data.Dataset.from_tensors(train_data)
  val_tensor = tf.data.Dataset.from_tensors(val_data)
  train_tensor.shuffle(len(train_data))
  val_tensor.shuffle(len(val_data))
  print('train_tensor',train_tensor)
  print('val_tensor',val_tensor)
  tra_tens = train_tensor.unbatch().map(process_TL)
  val_tens = val_tensor.unbatch().map(process_TL)
  print(tra_tens,'mapped train tensor ',tf.data.experimental.cardinality(tra_tens))
  print(val_tens,' mapped val tensor',tf.data.experimental.cardinality(val_tens))
  train_cache = tra_tens.cache(cache_file_t)
  val_cache = val_tens.cache(cache_file_v)
  #normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
  val_batch = val_tens.batch(batch_size)
  train_batch= tra_tens.batch(batch_size)
  train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  print(train_ds)
  

In [ ]:
path = '/content/results_sds_ns_4.npy'
results = np.load(path,allow_pickle=True).item()
results

##Trying out weights

###testing

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/gdrive/MyDrive/data/train',
  seed=123,
  image_size=(224, 224),
  batch_size=64,color_mode = 'rgb',shuffle = True,label_mode = 'categorical')

In [ ]:
class_names = test_dataset.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in test_dataset.take(2):
  #print(image_batch)
  print(labels_batch)
  break

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))
#image_batch, labels_batch = next(iter(test_dataset))


In [ ]:
test_dataset

In [ ]:
iterator = test_dataset.make_one_shot_iterator()
next_ele = iterator.get_next()
with tf.Session() as sess:
  try:
    while True:
      val = sess.run(next_ele)
      print(val)
  except tf.errors.OutOfRangeError:
    pass

In [ ]:
train_ds = test_dataset.cache()

In [ ]:
train_ds

In [ ]:
history = cent_model_c.fit(train_ds,epochs=5,verbose=1,steps_per_epoch=100,shuffle=True)

In [ ]:
cent

In [ ]:
top_1 = cent_model_c.get_layer(name='topic_1', index=None)
top_2 = cent_model_c.get_layer(name='topic_2', index=None)
top_3 = cent_model_c.get_layer(name='topic_3', index=None)
top_4 = cent_model_c.get_layer(name='topic_4', index=None)

In [ ]:
#top_3 =  0.00572042,  0.07700813,  0.00037249, ...,  0.01569479,
#         -0.04370036,  0.00142679
#top_4 = 0.07224628,  0.04283633,  0.08743759, ...,  0.12188006,
#         -0.01654566,  0.0358057 
# top_1 -0.00504995,  0.05073379, -0.01360468, ..., -0.01115265,
#          -0.03401065, -0.00033989],
#         [-0.0618718 , -0.03967685,  0.03000757, ...,  0.08042301,
#          -0.04552665, -0.07524551],
#         [-0.02856736,  0.01151073,  0.0035671 , ...,  0.00878615,
#          -0.00205357,  0.01009549],
# top_2 = 0.16858643,  0.06966868, -0.02712329, ...,  0.07811368,
#           0.06262797,  0.00686309],
#         [ 0.15656647, -0.03285585,  0.05510008, ...,  0.03848132,
#           0.00037507, -0.00980724],
#         [ 0.13819823,  0.02557753,  0.0020411 , ..., -0.01188827,
#          -0.02261599, -0.0185649
top_2.weights

In [ ]:
weig

In [ ]:
train_num_files=len([file for file in glob(str(train_dir + '/*/*'))])
val_num_files=len([file for file in glob(str(val_dir + '/*/*'))])

In [ ]:
train_files=[file for file in glob(str(train_dir + '/*/*'))]
val_files=[file for file in glob(str(val_dir + '/*/*'))]

In [ ]:
len(train_files),train_num_files

In [ ]:
tra_ten = tf.data.Dataset.from_tensor_slices(train_files)
val_ten = tf.data.Dataset.from_tensor_slices(val_files)
tra_ten = tra_ten.shuffle(train_num_files)
val_ten = val_ten.shuffle(val_num_files)

In [ ]:
tr = tra_ten.take(1)
for el in tr:
  print(el)

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

In [ ]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

In [ ]:
def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label

In [ ]:
tra_ten_test = tra_ten.map(process_TL)
val_ten_test = val_ten.map(process_TL)

In [ ]:
for image,label in tra_ten_test.take(1):
  print(image)
  print(label)

In [ ]:
tf.data.experimental.cardinality(tra_ten_test)

In [ ]:
train_fi
for item in train_fi:
  print(item)

In [ ]:
train_fi = tra_ten_test.cache()
val_fi = val_ten_test.cache()
#test_ds = train_fi.map(process_TL).shuffle(buffer_size = train_num_files,seed = 42,reshuffle_each_iteration=True).batch(batch_size)

In [ ]:
train_fi

In [ ]:
print(tf.data.experimental.cardinality(test_ds))
for it in test_ds:
  print(it)


In [ ]:
for images, labels in tra_ten.take(1):
  print('images.shape: ', images)
  print('labels.shape: ', labels)

In [ ]:
train_files = tf.data.Dataset.list_files([file for file in glob(str(train_dir + '/*/*'))], shuffle=False)
val_files = tf.data.Dataset.list_files([file for file in glob(str(val_dir + '/*/*'))], shuffle=False)

In [ ]:
len(train_files)

In [ ]:
train_ds = train_files.interleave(lambda x: tf.data.Dataset.list_files([file for file in glob(str(train_dir + '/*/*'))], shuffle=True),cycle_length=4 ).map(process_TL,num_parallel_calls = tf.data.experimental.AUTOTUNE)
val_ds = val_files.interleave(lambda x: tf.data.Dataset.list_files([file for file in glob(str(val_dir + '/*/*'))], shuffle=True),cycle_length=4 ).map(process_TL,num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [ ]:
train_ds

In [ ]:
train_ds = tra_ten.cache()
val_ds = val_ten.cache()


In [ ]:
train_ds

In [ ]:
def plot_batch_sizes(ds):
  batch_sizes = [batch.shape[0] for batch in ds]
  plt.bar(range(len(batch_sizes)), batch_sizes)
  plt.xlabel('Batch number')
  plt.ylabel('Batch size')

In [ ]:
test = train_ds.take(1)
test
for image, label in test:
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
batch_size = 128
num_epochs = 1
steps = int(train_num_files/batch_size)
steps_val = int(val_num_files/batch_size)

In [ ]:
tf.data.experimental.cardinality(train_fi)

In [ ]:
t_ds = train_fi.repeat(num_epochs).batch(batch_size)
#v_ds = val_ten_test.batch(batch_size)

In [ ]:
tf.data.experimental.cardinality(t_ds)

In [ ]:
test = t_ds.take(1)
tf.data.experimental.cardinality(test)

In [ ]:
train_prefetch_ds = t_ds.prefetch(tf.data.experimental.AUTOTUNE)
#val_prefetch_ds = v_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
tf.data.experimental.cardinality(train_prefetch_ds)

In [ ]:
t_ds

In [ ]:
import datetime
save_dir = '/content/gdrive/MyDrive/topic_m'
log_dir = "/content/gdrive/MyDrive/topic_m/logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
%reload_ext tensorboard

In [ ]:
callbacks = [
    ModelCheckpoint(
        filepath=os.path.join(save_dir,'topic'),
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="categorical_accuracy",
        verbose=1,
        save_freq='epoch',mode = 'max'
    ),
    EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=2,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    ),
    ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=1, min_lr=1e-6),
    TensorBoard(log_dir=log_dir, histogram_freq=1)         
    ]

In [ ]:
TensorBoard

In [ ]:
cent_model = tf.keras.models.load_model('/content/gdrive/MyDrive/topic_m/top')

In [ ]:
%tensorboard --logdir /content/gdrive/MyDrive/topic_m/logs

In [ ]:
history = cent_model.fit(t_ds,epochs=num_epochs,steps_per_epoch=steps,callbacks=callbacks,verbose=1)

In [ ]:
cent_model_c.save(os.path.join(save_dir,'top_1'))

In [ ]:
###TESTING MULTI GPU WORKFLOW

!pip install d2l==0.16.6
!pip install -U mxnet-cu101==1.7.0


In [ ]:
test = tf.data.Dataset.from_tensor_slices(train_files)
test = test.repeat().shuffle(buffer_size = 500,seed = 42).batch(batch_size, drop_remainder=True)
t = test.take(1)
for t1 in t:
  print(t1)
#20161202_151053_000

#Parameterized function for reading and training data

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Dense,AveragePooling2D,Dropout,Flatten,Dense,Input,Concatenate
import os
import pandas as pd
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from sklearn.metrics import multilabel_confusion_matrix,classification_report
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.utils import shuffle
import json
import os, re


#HYPER PARAMS #########
num_folds = 4
t_batch_size = 128
v_batch_size = 128
num_epochs =1
model_name = 'train_tf_kfold_sds_noshuff_2_'
learning_rate = 0.4
#Data directory stuff ###############################################
cache_dir = '/data/s4133366/data_cache'
train_csv = ''
train_dir = #'/data/s4133366/data/train'
save_dir = '/data/s4133366/saved_models'
data_dir = '/data/s4133366/data'
####################################################################
train_csv_file = 'short_df_collab.csv'
val_csv_file = 'val_short_collab.csv'
####################################################################
cache_t = os.path.join(cache_dir,model_name+'_t')
cache_v = os.path.join(cache_dir,model_name+'_v')

regu_t = os.path.split(cache_t)[-1]+'*'
regu_v = os.path.split(cache_v)[-1]+'*'
print(regu_v,regu_t)

#Change paths 
model_dir = os.path.join(save_dir,model_name)


plot_dir = os.path.join(model_dir,'plots')
history_dir = os.path.join(model_dir,'history')


train_metrics_path = os.path.join(history_dir,model_name+'.npy')
res_path = os.path.join(history_path,'results_'+model_name+'.npy')
chk_path = os.path.join(model_dir,model_name+'_chk')
#Saving weights for retraining ....
weights_path = os.path.join(model_dir,model_name+'_weights')
#######################

#Function for clearing cache
def purge(dir, pattern):
  for f in os.listdir(dir):
    if re.search(pattern, f):
      os.remove(os.path.join(dir, f))
#Function for making and storing the plots

def plot_metrics(history_path,results_path):
  read_train = np.load(history_path,allow_pickle=True).item()
  read_test = np.load(results_path,allow_pickle=True).item()
  history_path = os.path.split(history_path)[-1]
  get_fold = history_path[-5]
  history_path = history_path.replace('.npy','')
  if not os.path.isdir(plot_dir):
    print('Plot directory '+plot_dir+' does not exist at the moment. Creating ....')
    os.mkdir(plot_dir)
  print(plot_dir+' created ...')
  if not os.path.isdir(os.path.join(plot_dir,history_path)) :
    os.mkdir(os.path.join(plot_dir,history_path))
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path+' created...')
  else:
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path,' exists ...')  
    print('Adding the metric plots to ',folder_path)

  #summarize history for accuracy

  plt.plot(read_train['categorical_accuracy'])
  plt.axhline(read_test['categorical_accuracy'], color='g', linestyle='--')
  plt.title('model accuracy for fold_'+str(get_fold))
  plt.ylabel('Categorical Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'accuracy_'+str(history_path)+'.png'))
  plt.clf()

  # summarize history for loss

  plt.plot(read_train['loss'])
  plt.axhline(read_test['loss'], color='g', linestyle='--')
  plt.title('model loss for fold_'+str(get_fold))
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()
  plt.savefig(os.path.join(folder_path,'loss_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Precision 
  plt.plot(read_train['precision'])
  plt.axhline(read_test['precision'], color='g', linestyle='--')
  plt.title('model precision for fold_'+str(get_fold))
  plt.ylabel('Precision')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'precision_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Recall 

  plt.plot(read_train['recall'])
  plt.axhline(read_test['recall'], color='g', linestyle='--')
  plt.title('Model Recall for fold_'+str(get_fold))
  plt.ylabel('Recall')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'recall_'+str(history_path)+'.png'))
  plt.clf()



print('########## BASELINE MODEL TRAINING FOR SHORT DS USING STRATIFIED K FOLD ... ######################')
print('Loading CSVs')

train_csv = pd.read_csv('/content/short_df_collab.csv')
#train_csv = pd.read_csv(os.path.join(data_dir,train_csv_file))
train_csv = train_csv.sample(frac=1,random_state = 42)

val_csv = pd.read_csv('/content/val.csv')
#val_csv = pd.read_csv(os.path.join(data_dir,val_csv_file))
val_csv = val_csv.sample(frac=1,random_state = 42)

print('Data read and loading onto tensors.....')

def get_label(file_path):
# convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
# convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label

class_names = np.array(sorted([dir1 for dir1 in os.listdir(train_dir)]))


print('LOADING THE DATA ON TO TENSORS......')



print('Cleaning prior cache ...')
#purge(cache_dir,regu_v)   
print('value cache cleared ....')
#purge(cache_dir,regu_t)
print('train cache cleared ....')
training_data = train_csv
valid_data = val_csv
print(len(training_data))
print(len(valid_data))
train_data = training_data['Image']
train_labels = training_data['Label']
print(len(train_data))
val_labels = valid_data['Label']
val_data = valid_data['Image']
print(len(val_data))
steps = 1 #int(len(train_data)/t_batch_size)
steps_val = 1 #int(len(val_data)/v_batch_size)

train_tensor = tf.data.Dataset.from_tensors(train_data)
val_tensor = tf.data.Dataset.from_tensors(val_data)
train_tensor = train_tensor.unbatch()
val_tensor = val_tensor.unbatch()
print('un-shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('un-shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

train_tensor = train_tensor.shuffle(len(train_data))
val_tensor = val_tensor.shuffle(len(val_data))

print('shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

print('Shuffling the dataframes internally for more randomness...')

tra_tens = train_tensor.map(process_TL)
val_tens = val_tensor.map(process_TL)
print('tra_tens:',tf.data.experimental.cardinality(tra_tens))
print('tra_tens:',tf.data.experimental.cardinality(val_tens))
train_cache = tra_tens.cache()
val_cache = val_tens.cache()
print(tf.data.experimental.cardinality(train_cache))
print(tf.data.experimental.cardinality(val_cache))
print('Created cache ...')
#normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
val_batch = val_cache.batch(v_batch_size)
train_batch= train_cache.repeat(num_epochs).batch(t_batch_size)
print(tf.data.experimental.cardinality(train_batch))
print(tf.data.experimental.cardinality(val_batch))
print(train_batch)
print(train_batch)
train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(tf.data.experimental.cardinality(train_ds))
print(tf.data.experimental.cardinality(val_ds))
print(train_ds)


#Create model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = AveragePooling2D(pool_size=(7,7))(x)
x = Flatten(name ="flatten")(x)
x = Dense(128,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(24,activation="softmax")(x)

# #Activating the model

model = Model(inputs = base_model.input,outputs = preds)


for layer in base_model.layers:
  layer.trainable = False


opti = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, rho=0.95, epsilon=1e-07, name="Adadelta")
model.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

callbacks = [
  ModelCheckpoint(
    
    filepath=os.path.join(model_dir,model_name),
    save_best_only=True,  # Only save a model if `val_loss` has improved.
    monitor="categorical_accuracy",
    verbose=1,
    save_freq = 'epoch',mode = 'max'
  ),
  EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=7,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
  ),
  ReduceLROnPlateau(monitor='loss', factor=0.1,
                          patience=3, min_lr=1e-5,verbose=1)
]


#print('Loading model ...')
#model = keras.models.load_model(os.path.join(save_dir,'train_tf_kfold_sds_noshuff_'+str(fold_no)))    
print('Model loaded starting training...')
print('Accessing the model directory. creating if needed....')
if not os.path.isdir(model_dir):
  print(os.path.join(save_dir,model_dir)+' not found. Creating ....')
  os.mkdir(model_dir)
  if not os.path.isdir(history_dir):
    print('Creating '+history_dir +'...')
    os.mkdir(history_dir)
  if not os.path.isdir(plots_dir):
    os.mkdir(plots_dir)
    print('Creating '+plots_dir+'...')

print('Folders checked. Training model ...')
history = model.fit(train_ds,epochs=num_epochs,callbacks=callbacks,verbose=1,workers=8,use_multiprocessing = False,shuffle=True,steps_per_epoch = steps)
model.save(os.path.join(model_dir,model_name))
report = history.history
his_path = train_metrics_path
np.save(os.path.join(history_path,train_metrics_path), report)
history_filename = os.path.join(history_dir,train_metrics_path)
print('Results saved to disk ...')
print('Evaluating the model ...') 
model = keras.models.load_model(os.path.join(model_dir,model_name))

results = model.evaluate(val_ds,workers = 4,steps = steps_val)
results = dict(zip(model.metrics_names,results))

np.save(os.path.join(history_path,res_path), results)
results_filename = os.path.join(history_dir,res_path)
model.save(os.path.join(model_dir,model_name))
model.save_weights(weights_path)
print('Storing the plots in '+plot_dir+'....')

print('Trying to plot ...')
try:
  plot_metrics(history_filename,results_filename)
except : 
  print('Something went wrong while plotting skipping ...')

print('Metrics have been plotted ...')
tf.keras.backend.clear_session()
print('Clearning cache ...')
#purge(cache_dir,regu_v)   
print('value cache cleared ....')
#purge(cache_dir,regu_t)
print('train cache cleared ....')

print('Cache cleared')




print('Job ended ...')

In [ ]:
train_dir = '/data/s4133366/data/train'
save_dir = '/data/s4133366/saved_models'
model_name = 'train_tf_kfold_sds_noshuff_2_'
model_dir = os.path.join(save_dir,model_name)


plot_dir = os.path.join(model_dir,'plots')
history_dir = os.path.join(model_dir,'history')


train_metrics_path = os.path.join(history_dir,model_name+'.npy')
res_path = os.path.join(history_path,'results_'+model_name+'.npy')
chk_path = os.path.join(model_dir,model_name+'_chk')
#Saving weights for retraining ....
weights_path = os.path.join(model_dir,model_name+'_weights')

train_dir,plot_dir,history_path,res_path,train_metrics_path,chk_path,model_dir,weights_path


In [ ]:
import pandas as pd
pd.read_csv('/content/')


#Making predictions on the topic layer

###Using image data generators

In [ ]:
model = keras.models.load_model("/content/gdrive/MyDrive/saved_models/topic_full_ds_eval")


In [ ]:
lr = 0.1
num_epochs = 20
opti = Adam(learning_rate= lr,decay = 0)
model.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
testGen = datagen.flow_from_directory('/content/gdrive/MyDrive/data/test',target_size=(224,224),
                     color_mode='rgb',batch_size=116,class_mode=None,shuffle=False,seed=42)
num_test_samples = len(testGen.filenames)
num_test_classes = len(testGen.class_indices)
STEP_SIZE_TEST = testGen.n//testGen.batch_size

In [ ]:
testGen.shape


In [ ]:
print("Predicting the model")
testGen.reset()
predictions = model.predict(testGen,steps=STEP_SIZE_TEST,verbose=1,workers = 4,use_multiprocessing = False)


In [ ]:
pd.DataFrame(predictions).to_csv("topic_full_ds_predictions.csv")

In [ ]:
predictions.shape

In [ ]:
predictions_test = np.argmax(predictions,axis=1)
#print(testGen[0].shape)
len(testGen.labels)
#First 128 images
#testing = testGen[0]


In [ ]:
print("Predicting the model")
testGen.reset()
predictions = model.predict(testGen,steps=STEP_SIZE_TEST,verbose=1,workers = 4,use_multiprocessing = False)


In [ ]:
pd.DataFrame(predictions).to_csv("predictions.csv")

In [ ]:
predictions.shape

In [ ]:
predictions_test = np.argmax(predictions,axis=1)
#print(testGen[0].shape)
len(testGen.labels)
#First 128 images
#testing = testGen[0]


In [ ]:
test_labels = testGen.labels
test_labels = test_labels[:128]
test_labels

###Making the confusion matrix and deriving the classification report with data generators

In [ ]:
#preds = np.argmax(predictions,axis=1)
#len(predictions)
#preds
print(classification_report(testGen.classes, predictions_test,target_names=testGen.class_indices.keys()))
#ecode_predictions(predictions, top=3)[0]
#print("[INFO] saving model...")




In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

y_true = testGen.classes
y_pred = predictions_test

#print(list(labels))
cm = confusion_matrix(y_true,y_pred,labels =  list(range(0,24)) )
df_cm = pd.DataFrame(cm,index = ['Balcony','Bathroom','Beach','Bedroom','Buildings','Education,science','Forest, field, jungle', 'Garden', 'Hospital','Kitchen','Living room','Mountains, hills, desert, sky','Museum','Noise','Office','Other rooms','Others','Pathways','Recreation','Restaurant,Bar','Shop','Sport fields','Transportation','Water'] ,columns =  ['Balcony','Bathroom','Beach','Bedroom','Buildings','Education,science','Forest, field, jungle', 'Garden', 'Hospital','Kitchen','Living room','Mountains, hills, desert, sky','Museum','Noise','Office','Other rooms','Others','Pathways','Recreation','Restaurant,Bar','Shop','Sport fields','Transportation','Water'] )
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=False, annot_kws={"size": 20},cmap="BuPu") # font size
plt.title('Confusion matrix for Topic Full DS')
plt.show()

In [ ]:
cm.shape

In [ ]:
labels = ['Balcony','Bathroom','Beach','Bedroom','Buildings','Education,science','Forest, field, jungle', 'Garden', 'Hospital','Kitchen','Living room','Mountains, hills, desert, sky','Museum','Noise','Office','Other rooms','Others','Pathways','Recreation','Restaurant,Bar','Shop','Sport fields','Transportation','Water']

# New Topic Layer Implementation

##Centralized approach (c) {Looks closest to the schematic on the slides}

* Taking the output of the resnet soft max into k_1,k_2,k_3 and k_4 
* Concatenating the layers in order to get 4000 neurons with the 1000 objects for each topic(k=4)
* Flattening the different topics into Dense layer of 4000 neurons(4 topics*1000 objects)
* Adding a dense layer of 4000 neurons with tanh activation
* Adding 50% Dropout rate
* Using softmax in order to get the places(p = 24 i.e num_classes)

**Issue : Does not give places probabilities for each topic individually as it is being flattened and fed to softmax with only 24 units for each class**

In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = AveragePooling2D(pool_size=(7,7))(x)
x = Flatten(name ="flatten")(x)
x = Dense(1024,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model

model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    layer.trainable = False
# y = Dense(1)(model.output)
k_1 = Dense(20,name = 'topic_1')(model.output)
k_2 = Dense(20,name = 'topic_2')(model.output)
k_3 = Dense(20,name = 'topic_3')(model.output)
k_4 = Dense(20,name = 'topic_4')(model.output)
wt_add = Wt_Topics(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4)
flat_2 = Flatten(name = 'flat_c')(sum_layer)
# w_1 = tf.Variable(1.0,name='weight_topic_1',trainable=True )
# w_2 = tf.Variable(1.0 ,name='weight_topic_2',trainable=True)
# w_3 = tf.Variable(1.0 ,name='weight_topic_3',trainable=True )
# w_4 = tf.Variable(1.0 ,name='weight_topic_4',trainable=True)
# conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
# flat_2 = Flatten(name = 'flat_c')(conc2)
# #Adding the Dense layers now
# topic = Dense(4000,activation='tanh')(flat_2)
# topic = Dropout(0.5)(topic)
topic = Dense(128,activation='tanh')(flat_2)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)


In [ ]:
# conc2 = Concatenate(name = 'conc_c')([k_1*w_1,k_2*w_2,k_3*w_3,k_4*w_4])

In [ ]:
cent_model_c = Model(inputs = base_model.input,outputs = final)

In [ ]:
cent_model_c.summary()

In [ ]:
opti = tf.keras.optimizers.Adadelta(learning_rate=1e-1, rho=0.95, epsilon=1e-07, name="Adadelta")
cent_model_c.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
    w_init = tf.random_normal_initializer()
    self.w1 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_1'
    )
    self.w2 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_2'
    )  
    self.w3 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_3'
    )       
    self.w4 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_4'
    )
  def call(self, input1, input2, input3,input4):
    #name = 'tops'
    return tf.multiply(input1,self.w1) + tf.multiply(input2, self.w2) + tf.multiply(input3, self.w3)+tf.multiply(input4, self.w4)

In [ ]:
top_1 = cent_model_c.get_layer( index=-5)
topic_layer_weights = top_1.get_weights()
topic_1 = cent_model_c.get_layer(name = 'topic_1')
topic_1_weights = topic_1.get_weights()
topic_1_weights[0].shape
topic_layer_weights
cent_model_c.layers[-5].get_weights()

### 2x2 Topic Testing Here

In [ ]:


#HYPER PARAMS #########
num_folds = 4
t_batch_size = 128
v_batch_size = 128
num_epochs =10
model_name = 'test_2x2'
learning_rate = 0.4
#Data directory stuff ###############################################
cache_dir = ''
train_csv = '/content/gdrive/MyDrive/data/train'
train_dir = '/content/gdrive/MyDrive/data/train'#/data/s4133366/data/train'
save_dir = '/content/gdrive/MyDrive/saved_models'#'/data/s4133366/saved_models'
data_dir = '/content/gdrive/MyDrive/data'
####################################################################
train_csv_file = '/content/gdrive/MyDrive/data/short_df_collab.csv'
val_csv_file = '/content/gdrive/MyDrive/data/val_short_collab.csv'
####################################################################
#cache_t = os.path.join(cache_dir,model_name+'_t')
#cache_v = os.path.join(cache_dir,model_name+'_v')

#regu_t = os.path.split(cache_t)[-1]+'*'
#regu_v = os.path.split(cache_v)[-1]+'*'
#print(regu_v,regu_t)

#Change paths 
model_dir = os.path.join(save_dir,model_name)


plot_dir = os.path.join(model_dir,'plots')
history_dir = os.path.join(model_dir,'history')


train_metrics_path = os.path.join(history_dir,model_name+'.npy')
res_path = os.path.join(history_dir,'results_'+model_name+'.npy')
chk_path = os.path.join(model_dir,model_name+'_chk')
#Saving weights for retraining ....
weights_path = os.path.join(model_dir,model_name+'_weights')
#######################

#Function for clearing cache
# def purge(dir, pattern):
#   for f in os.listdir(dir):
#     if re.search(pattern, f):
#       os.remove(os.path.join(dir, f))
# #Function for making and storing the plots

def plot_metrics(history_path,results_path):
  read_train = np.load(history_path,allow_pickle=True).item()
  read_test = np.load(results_path,allow_pickle=True).item()
  history_path = os.path.split(history_path)[-1]
  get_fold = history_path[-5]
  history_path = history_path.replace('.npy','')
  if not os.path.isdir(plot_dir):
    print('Plot directory '+plot_dir+' does not exist at the moment. Creating ....')
    os.mkdir(plot_dir)
  print(plot_dir+' created ...')
  if not os.path.isdir(os.path.join(plot_dir,history_path)) :
    os.mkdir(os.path.join(plot_dir,history_path))
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path+' created...')
  else:
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path,' exists ...')  
    print('Adding the metric plots to ',folder_path)

  #summarize history for accuracy

  plt.plot(read_train['categorical_accuracy'])
  plt.axhline(read_test['categorical_accuracy'], color='g', linestyle='--')
  plt.title('model accuracy for fold_'+str(get_fold))
  plt.ylabel('Categorical Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'accuracy_'+str(history_path)+'.png'))
  plt.clf()

  # summarize history for loss

  plt.plot(read_train['loss'])
  plt.axhline(read_test['loss'], color='g', linestyle='--')
  plt.title('model loss for fold_'+str(get_fold))
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()
  plt.savefig(os.path.join(folder_path,'loss_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Precision 
  plt.plot(read_train['precision'])
  plt.axhline(read_test['precision'], color='g', linestyle='--')
  plt.title('model precision for fold_'+str(get_fold))
  plt.ylabel('Precision')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'precision_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Recall 

  plt.plot(read_train['recall'])
  plt.axhline(read_test['recall'], color='g', linestyle='--')
  plt.title('Model Recall for fold_'+str(get_fold))
  plt.ylabel('Recall')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'recall_'+str(history_path)+'.png'))
  plt.clf()






In [ ]:
print('########## BASELINE MODEL TRAINING FOR SHORT DS USING STRATIFIED K FOLD ... ######################')
print('Loading CSVs')

train_csv = pd.read_csv('/content/gdrive/MyDrive/data/short_df_collab.csv')
#train_csv = pd.read_csv(os.path.join(data_dir,train_csv_file))
train_csv = train_csv.sample(frac=1,random_state = 42)

val_csv = pd.read_csv('/content/gdrive/MyDrive/data/val_short_collab.csv')
#val_csv = pd.read_csv(os.path.join(data_dir,val_csv_file))
val_csv = val_csv.sample(frac=1,random_state = 42)

print('Data read and loading onto tensors.....')

def get_label(file_path):
# convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
# convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  #img = tf.keras.preprocessing.image.img_to_array(img)
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.image.per_image_standardization(img)
  #img = tf.cast(img/255. ,tf.float32)
  return img, label
def process_Test(file_path):
  #img = Image.open(file_path)
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.image.per_image_standardization(img)
  #img = tf.cast(img/255. ,tf.float32)
  return img
class_names = np.array(sorted([dir1 for dir1 in os.listdir(train_dir)]))


print('LOADING THE DATA ON TO TENSORS......')



print('Cleaning prior cache ...')
#purge(cache_dir,regu_v)   
print('value cache cleared ....')
#purge(cache_dir,regu_t)
print('train cache cleared ....')
training_data = train_csv
valid_data = val_csv
print(len(training_data))
print(len(valid_data))
########## USING CLASS WEIGHTS ########
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(training_data['Label'])
                                               ,training_data['Label'])


###############################


train_data = training_data['Image']
train_labels = training_data['Label']
print(len(train_data))
val_labels = valid_data['Label']
val_data = valid_data['Image']
print(len(val_data))
steps = int(len(train_data)/t_batch_size)
steps_val = int(len(val_data)/v_batch_size)

train_tensor = tf.data.Dataset.from_tensors(train_data)
val_tensor = tf.data.Dataset.from_tensors(val_data)
train_tensor = train_tensor.unbatch()
val_tensor = val_tensor.unbatch()
print('un-shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('un-shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

train_tensor = train_tensor.shuffle(len(train_data))
val_tensor = val_tensor.shuffle(len(val_data))

print('shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

print('Shuffling the dataframes internally for more randomness...')

tra_tens = train_tensor.map(process_TL)
val_tens = val_tensor.map(process_TL)
print('tra_tens:',tf.data.experimental.cardinality(tra_tens))
print('tra_tens:',tf.data.experimental.cardinality(val_tens))
train_cache = tra_tens.cache()
val_cache = val_tens.cache()
print(tf.data.experimental.cardinality(train_cache))
print(tf.data.experimental.cardinality(val_cache))
print('Created cache ...')
#normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
val_batch = val_cache.batch(v_batch_size)
train_batch= train_cache.repeat(num_epochs).batch(t_batch_size)
print(tf.data.experimental.cardinality(train_batch))
print(tf.data.experimental.cardinality(val_batch))
print(train_batch)
print(train_batch)
train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(tf.data.experimental.cardinality(train_ds))
print(tf.data.experimental.cardinality(val_ds))
print(train_ds)


####Testing

In [ ]:
tra_test = train_tensor.map(process_Test)
#val_test = val_tensor.map(process_TL)

In [ ]:
for el in tra_test.take(1):
  print(el)

In [ ]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(training_data['Label'])
                                               ,training_data['Label'])



In [ ]:

weights = {i:el for el,i in zip(class_weight,range(0,24))}

In [ ]:
weights

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
      
    self.w1 = self.add_weight(name = 'weight_topic_1',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
    self.w2 = self.add_weight(name = 'weight_topic_2',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())  
    
    self.w3 = self.add_weight(name = 'weight_topic_3',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())       
    
    self.w4 = self.add_weight(name = 'weight_topic_4',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
  def call(self, input1, input2, input3,input4):
    #Sum of the softmax output*topic weights
    return tf.multiply(input1,self.w1) , tf.multiply(input2, self.w2) , tf.multiply(input3, self.w3),tf.multiply(input4, self.w4)


In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
    w_init = tf.random_normal_initializer()
    self.w1 = self.add_weight(
        shape=(self.units), dtype="float32"),trainable=True,name = 'weight_topic_1',initializer = 'random_normal'
    
   
    self.w2 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_2',initializer = 'random_normal'
    )  
    self.w3 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_3',initializer = 'random_normal'
    )       
    self.w4 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_4',initializer = 'random_normal'
    )
  def call(self, input1, input2, input3,input4):
    #name = 'tops'
    return tf.multiply(input2,self.w1) + tf.multiply(input1, self.w2) + tf.multiply(input3, self.w3)+tf.multiply(input4, self.w4)

In [ ]:
k_1 = Dense(1,name = 'topic_1')(model.output)
k_2 = Dense(1,name = 'topic_2')(model.output)
k_3 = Dense(1,name = 'topic_3')(model.output)
k_4 = Dense(1,name = 'topic_4')(model.output)
wt_add = Wt_Topics(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4)
topic = Dense(256,activation='relu')(sum_layer)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)


cent_model = Model(inputs = base_model.input,outputs = final)


In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name ="flatten")(x)
x = Dense(1024,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model

model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    layer.trainable = False
# y = Dense(1)(model.output)
k_1 = Dense(1,name = 'topic_1')(model.output)
k_2 = Dense(1,name = 'topic_2')(model.output)
k_3 = Dense(1,name = 'topic_3')(model.output)
k_4 = Dense(1,name = 'topic_4')(model.output)
wt_add = Wt_Topics(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4)
topic = Dense(256,activation='relu')(sum_layer)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)


cent_model = Model(inputs = base_model.input,outputs = final)


In [ ]:
cent_model_c = Model(inputs = base_model.input,outputs = final)
cent_model_c.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_2x2_1.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

#plot_model(cent_model_c, to_file='/content/plot/plot.png')

In [ ]:
opti = tf.keras.optimizers.Adadelta(learning_rate=1e-1, rho=0.95, epsilon=1e-07, name="Adadelta")
cent_model_c.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

Defining the callbacks

In [ ]:
cent_model_c.layers[-5].get_weights()

In [ ]:
#if not isfile(json_log):

json_log = open('loss_log.json', mode='wt', buffering=1)
callbacks = [
  ModelCheckpoint(
    
    filepath=os.path.join(model_dir,model_name),
    save_best_only=True,  # Only save a model if `val_loss` has improved.
    monitor="categorical_accuracy",
    verbose=1,
    save_freq = 'epoch',mode = 'max'
  ),
  EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=7,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
  ),
  ReduceLROnPlateau(monitor='loss', factor=0.1,
                          patience=3, min_lr=1e-5,verbose=1),
  
  LambdaCallback(
            on_epoch_end=lambda epoch, logs: json_log.write(
                json.dumps({'epoch': epoch, 
                            'categorical_accuracy': logs['categorical_accuracy'],
                            'loss': logs['loss'],
                            'topic_layer_weights': list(cent_model_c.layers[-5].get_weights())},cls = NumpyArrayEncoder) + '\n'),
            on_train_end=lambda logs: json_log.close()
  )

         
]


Fitting the new model

In [ ]:
history = cent_model_c.fit(train_ds,epochs=num_epochs,callbacks=callbacks,verbose=1,workers=8,use_multiprocessing = False,shuffle=True,steps_per_epoch = 100,)
cent_model_c.save(os.path.join(model_dir,model_name))


In [ ]:
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


In [ ]:
a =cent_model_c.layers[-5].get_weights()

# [array([[0.4940912]], dtype=float32),
#  array([[-0.6106522]], dtype=float32),
#  array([[0.5864157]], dtype=float32),
#  array([[-0.41294459]], dtype=float32)]
type(a[0])
# b

## 3x3 Topic Testing Here

###Parameter Definition and Functions

In [ ]:


#HYPER PARAMS #########
num_folds = 4
t_batch_size = 128
v_batch_size = 128
num_epochs =10
model_name = 'test_topic'
learning_rate = 0.4
#Data directory stuff ###############################################
cache_dir = ''
train_csv = '/content/gdrive/MyDrive/data/train'
train_dir = '/content/gdrive/MyDrive/data/train'#/data/s4133366/data/train'
save_dir = '/content/gdrive/MyDrive/saved_models'#'/data/s4133366/saved_models'
data_dir = '/content/gdrive/MyDrive/data'
####################################################################
train_csv_file = '/content/gdrive/MyDrive/data/short_df_collab.csv'
val_csv_file = '/content/gdrive/MyDrive/data/val_short_collab.csv'
####################################################################
#cache_t = os.path.join(cache_dir,model_name+'_t')
#cache_v = os.path.join(cache_dir,model_name+'_v')

#regu_t = os.path.split(cache_t)[-1]+'*'
#regu_v = os.path.split(cache_v)[-1]+'*'
#print(regu_v,regu_t)

#Change paths 
model_dir = os.path.join(save_dir,model_name)


plot_dir = os.path.join(model_dir,'plots')
history_dir = os.path.join(model_dir,'history')


train_metrics_path = os.path.join(history_dir,model_name+'.npy')
res_path = os.path.join(history_dir,'results_'+model_name+'.npy')
chk_path = os.path.join(model_dir,model_name+'_chk')
#Saving weights for retraining ....
weights_path = os.path.join(model_dir,model_name+'_weights')
#######################

#Function for clearing cache
# def purge(dir, pattern):
#   for f in os.listdir(dir):
#     if re.search(pattern, f):
#       os.remove(os.path.join(dir, f))
# #Function for making and storing the plots

def plot_metrics(history_path,results_path):
  read_train = np.load(history_path,allow_pickle=True).item()
  read_test = np.load(results_path,allow_pickle=True).item()
  history_path = os.path.split(history_path)[-1]
  get_fold = history_path[-5]
  history_path = history_path.replace('.npy','')
  if not os.path.isdir(plot_dir):
    print('Plot directory '+plot_dir+' does not exist at the moment. Creating ....')
    os.mkdir(plot_dir)
  print(plot_dir+' created ...')
  if not os.path.isdir(os.path.join(plot_dir,history_path)) :
    os.mkdir(os.path.join(plot_dir,history_path))
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path+' created...')
  else:
    folder_path = os.path.join(plot_dir,history_path)
    print(folder_path,' exists ...')  
    print('Adding the metric plots to ',folder_path)

  #summarize history for accuracy

  plt.plot(read_train['categorical_accuracy'])
  plt.axhline(read_test['categorical_accuracy'], color='g', linestyle='--')
  plt.title('model accuracy for fold_'+str(get_fold))
  plt.ylabel('Categorical Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'accuracy_'+str(history_path)+'.png'))
  plt.clf()

  # summarize history for loss

  plt.plot(read_train['loss'])
  plt.axhline(read_test['loss'], color='g', linestyle='--')
  plt.title('model loss for fold_'+str(get_fold))
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()
  plt.savefig(os.path.join(folder_path,'loss_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Precision 
  plt.plot(read_train['precision'])
  plt.axhline(read_test['precision'], color='g', linestyle='--')
  plt.title('model precision for fold_'+str(get_fold))
  plt.ylabel('Precision')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'precision_'+str(history_path)+'.png'))
  plt.clf()

  #summarize history for Recall 

  plt.plot(read_train['recall'])
  plt.axhline(read_test['recall'], color='g', linestyle='--')
  plt.title('Model Recall for fold_'+str(get_fold))
  plt.ylabel('Recall')
  plt.xlabel('Epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.savefig(os.path.join(folder_path,'recall_'+str(history_path)+'.png'))
  plt.clf()






###Data Wrangling and model tuning

In [ ]:
print('########## BASELINE MODEL TRAINING FOR SHORT DS USING STRATIFIED K FOLD ... ######################')
print('Loading CSVs')

train_csv = pd.read_csv(train_csv_file)
#train_csv = pd.read_csv(os.path.join(data_dir,train_csv_file))
train_csv = train_csv.sample(frac=1,random_state = 42)

val_csv = pd.read_csv(val_csv_file)
#val_csv = pd.read_csv(os.path.join(data_dir,val_csv_file))
val_csv = val_csv.sample(frac=1,random_state = 42)

print('Data read and loading onto tensors.....')

def get_label(file_path):
# convert the path to a list of path components separated by sep
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.cast(one_hot, tf.int32)

def decode_img(img):
# convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)  
  # resize the image to the desired size
  return tf.image.resize(img, [224, 224])

def process_TL(file_path):
  label = get_label(file_path) 
  img = tf.io.read_file(file_path) 
  img = decode_img(img)
  img = preprocess_input(img)
  img = tf.cast(img/255. ,tf.float32)
  return img, label

class_names = np.array(sorted([dir1 for dir1 in os.listdir(train_dir)]))


print('LOADING THE DATA ON TO TENSORS......')



print('Cleaning prior cache ...')
#purge(cache_dir,regu_v)   
print('value cache cleared ....')
#purge(cache_dir,regu_t)
print('train cache cleared ....')
training_data = train_csv
valid_data = val_csv
print(len(training_data))
print(len(valid_data))
train_data = training_data['Image']
train_labels = training_data['Label']
print(len(train_data))
val_labels = valid_data['Label']
val_data = valid_data['Image']
print(len(val_data))
steps = int(len(train_data)/t_batch_size)
steps_val = int(len(val_data)/v_batch_size)

train_tensor_1 = tf.data.Dataset.from_tensors(train_data)
val_tensor_1 = tf.data.Dataset.from_tensors(val_data)
train_tensor = train_tensor_1.unbatch()
val_tensor = val_tensor_1.unbatch()
print('un-shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('un-shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

train_tensor = train_tensor.shuffle(len(train_data))
val_tensor = val_tensor.shuffle(len(val_data))

print('shuffled_tensor:',tf.data.experimental.cardinality(train_tensor))
print('shuffled_tensor:',tf.data.experimental.cardinality(val_tensor))

print('Shuffling the dataframes internally for more randomness...')

tra_tens = train_tensor.map(process_TL)
val_tens = val_tensor.map(process_TL)
print('tra_tens:',tf.data.experimental.cardinality(tra_tens))
print('tra_tens:',tf.data.experimental.cardinality(val_tens))
train_cache = tra_tens.cache()
val_cache = val_tens.cache()
print(tf.data.experimental.cardinality(train_cache))
print(tf.data.experimental.cardinality(val_cache))
print('Created cache ...')
#normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
val_batch = val_cache.batch(v_batch_size)
train_batch= train_cache.repeat(num_epochs).batch(t_batch_size)
print(tf.data.experimental.cardinality(train_batch))
print(tf.data.experimental.cardinality(val_batch))
print(train_batch)
print(train_batch)
train_ds = train_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_batch.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(tf.data.experimental.cardinality(train_ds))
print(tf.data.experimental.cardinality(val_ds))
print(train_ds)


In [ ]:
for element in train_cache.take(1):
  print(element)
train_cache

###Defining the Topic Class

In [ ]:
#3x3
class Wt_Topics3x3(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics3x3, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
    w_init = tf.random_normal_initializer()
    #1
    self.w1 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_1'
    )
    self.w2 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_2'
    )  
    self.w3 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_3'
    ) 
    #2      
    self.w4 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_4'
    )
    self.w5 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_5'
    )
    self.w6 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_6'
    )
    #3
    self.w7 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_7'
    )
    self.w8 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_8'
    )
    self.w9 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_9'
    )
    #self.bias = tf.Variable(tf.zeros([self.units]), name="weight_biases",dtype = 'float32',trainable=True)

  

  def call(self, input1, input2, input3,input4,input5,input6,input7,input8,input9):
    #name = 'tops'
    return tf.multiply(input1,self.w1) + tf.multiply(input2, self.w2) + tf.multiply(input3, self.w3)+tf.multiply(input4, self.w4)+tf.multiply(input5,self.w5)+tf.multiply(input6,self.w6)+tf.multiply(input7,self.w7)+tf.multiply(input8,self.w8)+tf.multiply(input9,self.w9)#+self.bias

###Defining the base model here

In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D( )(x)
x = Flatten(name ="flatten")(x)
x = Dense(1024,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model

model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

###Declaring the rest of the model

In [ ]:

k_1 = Dense(20,name = 'topic_1')(model.output)
k_2 = Dense(20,name = 'topic_2')(model.output)
k_3 = Dense(20,name = 'topic_3')(model.output)
k_4 = Dense(20,name = 'topic_4')(model.output)
k_5 = Dense(20,name = 'topic_5')(model.output)
k_6 = Dense(20,name = 'topic_6')(model.output)
k_7 = Dense(20,name = 'topic_7')(model.output)
k_8 = Dense(20,name = 'topic_8')(model.output)
k_9 = Dense(20,name = 'topic_9')(model.output)
wt_add = Wt_Topics3x3(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4,k_5,k_6,k_7,k_8,k_9)
flat_2 = Flatten(name = 'flat_c')(sum_layer)
# w_1 = tf.Variable(1.0,name='weight_topic_1',trainable=True )
# w_2 = tf.Variable(1.0 ,name='weight_topic_2',trainable=True)
# w_3 = tf.Variable(1.0 ,name='weight_topic_3',trainable=True )
# w_4 = tf.Variable(1.0 ,name='weight_topic_4',trainable=True)
# conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
# flat_2 = Flatten(name = 'flat_c')(conc2)
# #Adding the Dense layers now
# topic = Dense(4000,activation='tanh')(flat_2)
# topic = Dropout(0.5)(topic)
topic = Dense(256,activation='relu')(flat_2)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)



In [ ]:
cent_model_c = Model(inputs = base_model.input,outputs = final)
cent_model_c.summary()

In [ ]:
cent_model_c.layers[-5].get_weights()

###Callbacks and compiling the model

In [ ]:
opti = tf.keras.optimizers.Adadelta(learning_rate=1e-1, rho=0.95, epsilon=1e-07, name="Adadelta")
cent_model_c.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

Defining the callbacks

In [ ]:
#if not isfile(json_log):

json_log = open('loss_log.json', mode='wt', buffering=1)
callbacks = [
  ModelCheckpoint(
    
    filepath=os.path.join(model_dir,model_name),
    save_best_only=True,  # Only save a model if `val_loss` has improved.
    monitor="categorical_accuracy",
    verbose=1,
    save_freq = 'epoch',mode = 'max'
  ),
  EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=7,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
  ),
  ReduceLROnPlateau(monitor='loss', factor=0.1,
                          patience=3, min_lr=1e-5,verbose=1),
  
  LambdaCallback(
            on_epoch_end=lambda epoch, logs: json_log.write(
                json.dumps({'epoch': epoch, 
                            'categorical_accuracy': logs['categorical_accuracy'],
                            'loss': logs['loss'],
                            'topic_layer_weights': list(cent_model_c.layers[-5].get_weights())},cls = NumpyArrayEncoder) + '\n'),
            on_train_end=lambda logs: json_log.close()
  )

         
]


Fitting the new model

In [ ]:
history = cent_model_c.fit(train_ds,epochs=num_epochs,callbacks=callbacks,verbose=1,workers=8,use_multiprocessing = False,shuffle=True,steps_per_epoch = 100)
cent_model_c.save(os.path.join(model_dir,model_name))


In [ ]:
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)


In [ ]:
a =cent_model_c.layers[-5].get_weights()

# [array([[0.4940912]], dtype=float32),
#  array([[-0.6106522]], dtype=float32),
#  array([[0.5864157]], dtype=float32),
#  array([[-0.41294459]], dtype=float32)]
type(a[0])
# b

## Transferring weights of the model

###This is performed in for Places365 dataset in order to load the trained baseline model for faster training of the Places365 Topic Models

In [ ]:
model1 = keras.models.load_model('/content/gdrive/MyDrive/saved_models/baseline_p365_without_top')
for layer in model1.layers:
  layer.trainable = False
  x = model1.output
preds = Dense(1000,activation="softmax")(x)
model = Model(model1.input,preds)
for layer in model.layers:
  layer.trainable = False
k_1 = Dense(1,name = 'topic_1')(model.output)
k_2 = Dense(1,name = 'topic_2')(model.output)
k_3 = Dense(1,name = 'topic_3')(model.output)
k_4 = Dense(1,name = 'topic_4')(model.output)
wt_add = Wt_Topics(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4)
topic = Dense(1024,activation='relu')(sum_layer)
topic = Dropout(0.2)(topic)
final = Dense(365,activation='softmax')(topic)


cent_model = Model(inputs = model.input,outputs = final)

In [ ]:
model1 = keras.models.load_model('/content/gdrive/MyDrive/saved_models/baseline_p365_without_top')

In [ ]:
for layer in model1.layers:
  layer.trainable = False

In [ ]:
x = model1.output
preds = Dense(1000,activation="softmax")(x)
model = Model(model1.input,preds)
for layer in model.layers:
  layer.trainable = False

In [ ]:
k_1 = Dense(1,name = 'topic_1')(model.output)
k_2 = Dense(1,name = 'topic_2')(model.output)
k_3 = Dense(1,name = 'topic_3')(model.output)
k_4 = Dense(1,name = 'topic_4')(model.output)
wt_add = Wt_Topics(1)
sum_layer = wt_add(k_1,k_2,k_3,k_4)
topic = Dense(1024,activation='relu')(sum_layer)
topic = Dropout(0.2)(topic)
final = Dense(365,activation='softmax')(topic)


cent_model = Model(inputs = model.input,outputs = final)

In [ ]:
cent_model.summary()

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
base_model.summary()

In [ ]:
train = pd.read_csv('train_mod.csv')
train = train[['Image','Label']]

In [ ]:
train.to_csv('train_new.csv')
files.download('train_new.csv')

#Topic Model Evaluation

In [ ]:
topic_model = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models/topic_weight_k4_places365_1_eval')

In [ ]:
topic_model_2 = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models/topic_weight_k4_places365_2_eval')

In [ ]:
topic_model_2.summary()

In [ ]:
topic_model.get_layer('resnet_out').get_weights()

In [ ]:
a = topic_model.get_layer('resnet_out').get_weights()
weights = a[0]
biases = a[1]

In [ ]:
weights.shape

In [ ]:
b= topic_model.get_layer('wt__topics').
#weights_out = b[0]
#biases_out = b[1]

In [ ]:
topic_model.get_layer('wt__topics').output

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
c = base_model.get_layer('predictions').get_weights()

In [ ]:
c[0].shape

In [ ]:
img = keras.preprocessing.image.load_img(
    '/content/gdrive/MyDrive/places365_sampled/val/airfield/Places365_val_00000435.jpg', target_size=(224,224,3)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
x= img_array
 

In [ ]:
x = x/255.

In [ ]:
x

In [ ]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([topic_model_2.input],
                                  [topic_model_2.get_layer('predictions').output])
layer_output = get_3rd_layer_output([x])[0]

In [ ]:
layer_output
# array([[9.28297084e-08, 4.01620838e-07, 9.37058394e-06, 8.11473183e-06,
#         8.00038033e-05, 4.70403074e-06, 3.78478546e-07, 3.41940085e-06,
#         1.02050910e-06, 3.02595481e-07, 4.38917569e-09, 3.00811962e-08,
#         1.02478033e-07, 2.52272923e-08, 2.13475346e-06, 5.36712790e-08,
#         3.58117632e-07, 4.86063186e-07, 1.58908813e-06, 6.16022930e-08,
#         1.55278556e-07, 7.01976148e-08, 3.24510040e-07, 1.70968656e-06,
#         1.58559942e-07, 7.26027665e-08, 1.70016961e-08, 5.52901334e-08,
#         1.41197640e-08, 1.67404338e-07, 3.79966174e-08, 1.40522147e-07,
#         5.02371122e-08, 2.05255446e-07, 7.79303662e-07, 1.30195019e-08,
#         3.17171782e-07, 9.17453690e-07, 9.98555993e-09, 2.40200990e-07,
#         1.12698416e-07, 8.08853216e-08, 5.35295790e-08, 5.57538620e-08,
#         6.81153125e-08, 8.33016571e-08, 1.08284425e-07, 5.90509011e-08,
#         1.09753626e-06, 4.82614375e-08, 1.20412790e-06, 1.01259111e-05,
#         2.72258660e-07, 2.23813696e-08, 2.93538953e-08, 3.92554938e-07,
#         1.06888436e-07, 2.76315024e-08, 9.75771144e-08, 4.03090070e-07,
#         1.00771410e-06, 1.73865510e-06, 1.05170876e-07, 8.13673893e-08,
#         8.72278108e-08, 2.65132314e-07, 3.06861416e-08, 3.52348195e

In [ ]:
layer_output
topic_lab = np.argmax(layer_output) 
topic_list = np.argsort(layer_output[0])[::-1][:10]
#place_lab = np.argmax(layer_output) 
#place_lab
topic_lab

In [ ]:
topic_list

In [ ]:
#3.66683019e-15 for topic 1 for first label
#9.282971e-08 from resnet 50 first element

# layer_output
checking = np.array(layer_output)
# np.argmax(layer_output)

In [ ]:
topic_model_2.get_layer('wt__topics').get_weights() 

In [ ]:
-6.754724e-08*9.282971e-08

In [ ]:
first_top_el1 = checking[0][0][0][0]
second_top_el1 = checking[0][1][0][0]

In [ ]:
second_top_el1

In [ ]:
first_out = layer_output[0][0]

In [ ]:
top_1 = checking[0]
top_1[0][0]
# top_2 = checking[1][0][0]

###Testing class weights and sample weights

In [ ]:
class_weight = class_weight.compute_class_weight('balanced',np.unique(training_data['Label']),training_data['Label'])

In [ ]:
sample_weights = class_weight/np.max(class_weight)

In [ ]:
weights = compute_sample_weight(class_weight='balanced', y=training_data['Label'])

In [ ]:
weights

In [ ]:
weights_test = np.ones(shape=(len(training_data['Label']),))

In [ ]:
len(weights_test)

In [ ]:
# class_weight = array([  9.42866162,   3.24956484,   2.17965558,   1.02266502,
#          8.70337995,   0.41486111,   2.13845934,   3.19944302,
#         30.35569106,   0.92191358,   0.45142667,   2.12386234,
#         17.04908676,   0.37691803,   0.41486111,   1.2725801 ,
#          5.76195988,   0.41486111,   2.05376788,   0.41486111,
#          1.09462035,   3.08829611,   0.41486111, 311.14583333])


##Defining callback to return the model intermediate outputs to JSON

###Class to obtain the output from ResNet-50

In [ ]:
from keras import backend as K
# def model_output(model,layer):
# # with a Sequential model
#   get_3rd_layer_output = K.function([model.input],
#                                     [model.get_layer(layer).output])
#   layer_output = get_3rd_layer_output([x])[0]

In [ ]:
img = keras.preprocessing.image.load_img(
    '/content/gdrive/MyDrive/places365_sampled/val/airfield/Places365_val_00000435.jpg', target_size=(224,224,3)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
x= img_array
 

####For implementations without custom layer

Writing the JSON file to test

In [ ]:
json_out = open('test_out.json', mode='wt', buffering=1)

In [ ]:
get_t1_output = K.function([cent_model.input],[cent_model.get_layer('topic_1').output])
test = get_t1_output(x)[0]
test.shape
get_t2_output = K.function([cent_model.input],[cent_model.get_layer('topic_2').output])
get_t2_output(x)[0]

###Class Definition

In [ ]:
class OutputCallback(keras.callbacks.Callback):

    def on_train_begin(self,logs):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))
        
    
    def on_train_batch_end(self,batch,logs = {}):
        
        keys = list(logs.keys())
        get_pred_layer_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('predictions').output])
        #Getting Resnet-50 softmax output
        print('check_1')
        print('Input:',self.model.input)
      
        resnet50_output = get_pred_layer_output([self.model.input])
        print('Resnet:',resnet50_output)
        #Getting the Intermediate topic outputs
        get_t1_output = K.function(inputs=[self.model.input],outputs=[self.model.get_layer('topic_1').output])
        get_t2_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('topic_2').output])
        get_t3_output = K.function(inputs = [self.model.input],outputs=[self.model.get_layer('topic_3').output])
        get_t4_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('topic_4').output])
        print('check_2')
        print('Calling the values before writing to file')
        #Calling the values before writing to file
        t1_out = list(get_t1_output([self.model.input]))#get_t1_output.outputs
        t2_out = list(get_t2_output([self.model.input]))
        t3_out = list(get_t3_output([self.model.input]))#get_t3_output.outputs
        t4_out = list(get_t4_output([self.model.input]))#get_t4_output.outputs#
        print('Topics: ',t1_out,t2_out,t3_out,t4_out)
        print('check')
        # json_out.write(
        #         json.dumps({'batch': batch, 
        #                     'ResNet': list(resnet50_output),
        #                     'Topic 1': t1_out,
        #                     'Topic 2': t2_out,
        #                     'Topic 3': t3_out,
        #                     'Topic 4': t4_out,

        #                     #,'model_output' : list(self.model.layers[-1].output)
        #         },cls = NumpyArrayEncoder) + '\n')


In [ ]:
class OutputCallback(keras.callbacks.Callback):

    def on_train_begin(self,logs):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))
        
    
    def on_train_batch_end(self,batch,logs = {}):
        
        keys = list(logs.keys())
        get_pred_layer_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('predictions').output])
        #Getting Resnet-50 softmax output
        
        #Getting the Intermediate topic outputs
        get_t1_output = K.function(inputs=[self.model.input],outputs=[self.model.get_layer('topic_1').output])
        get_t2_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('topic_2').output])
        get_t3_output = K.function(inputs = [self.model.input],outputs=[self.model.get_layer('topic_3').output])
        get_t4_output = K.function(inputs = [self.model.input],outputs = [self.model.get_layer('topic_4').output])
        print('check_1')
        print('Input:',self.model.input)
        images = self.model.input
        images = images.unbatch()
        print('Images',images)
        i=0
        for image in self.model.input:
          
          print('image',i)
          resnet50_output = get_pred_layer_output([image])
          print('Resnet:',resnet50_output)
          print('check_2')
          print('Calling the values before writing to file')
          #Calling the values before writing to file
          t1_out = list(get_t1_output([image]))#get_t1_output.outputs
          t2_out = list(get_t2_output([image]))
          t3_out = list(get_t3_output([image]))#get_t3_output.outputs
          t4_out = list(get_t4_output([image]))#get_t4_output.outputs#
          print('Topics: ',t1_out,t2_out,t3_out,t4_out)
          i = i+1
        print('check done ...')
        # json_out.write(
        #         json.dumps({'batch': batch, 
        #                     'ResNet': list(resnet50_output),
        #                     'Topic 1': t1_out,
        #                     'Topic 2': t2_out,
        #                     'Topic 3': t3_out,
        #                     'Topic 4': t4_out,

        #                     #,'model_output' : list(self.model.layers[-1].output)
        #         },cls = NumpyArrayEncoder) + '\n')


#Trying out variations of model implementations

##Implementation 3

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
      
    self.w1 = self.add_weight(name = 'weight_topic_1',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
    self.w2 = self.add_weight(name = 'weight_topic_2',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())  
    
    self.w3 = self.add_weight(name = 'weight_topic_3',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())       
    
    self.w4 = self.add_weight(name = 'weight_topic_4',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
  def call(self, input1, input2, input3,input4):
    #Sum of the softmax output*topic weights
    return tf.multiply(input1,self.w1), tf.multiply(input2, self.w2), tf.multiply(input3, self.w3),tf.multiply(input4, self.w4)


In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

# Create a model that includes the augmentation stage
input_shape = (224, 224, 3)
classes = 24
inputs = keras.Input(shape=input_shape)
# Augment images
x = data_augmentation(inputs)
# Rescale image values to [0, 1]
x = preprocessing.Rescaling(1.0 / 255)(x)


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
wt_add = Wt_Topics(1000) #Calling the Topic Layer
sum_layer = wt_add(base_model.output,base_model.output,
                    base_model.output,base_model.output)
conc = keras.layers.Concatenate()(sum_layer)
flat_2 = keras.layers.Flatten(name = 'flat_2')(conc)
topic_flat = Dense(2000,name = 'first_dense')(flat_2)
topic = Dropout(0.45,name = 'drop_1')(topic_flat)
topic = Dense(256,activation='relu')(topic)
topic = Dropout(0.2,name = 'drop_2')(topic)
final = Dense(24,activation='softmax')(topic)

cent_model = Model(inputs = base_model.input,outputs = final)


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



#for layer in base_model.layers:
#  layer.trainable = False
#for layer in model.layers:
#  layer.trainable = True

wt_add = Wt_Topics(1)
#new_tensor = tf.reshape(wt_add, shape=[tf.shape(wt_add)[0]*tf.shape(wt_add)[1],4])
#sum_layer = wt_add(k_1,k_2,k_3,k_4)
sum_layer = wt_add(base_model.output,base_model.output,base_model.output,base_model.output)

conc = keras.layers.Concatenate()(sum_layer)
#flat = keras.layers.GlobalAveragePooling1D()(conc)
#test = keras.layers.GlobalAveragePooling1D()(flat)
flat_2 = keras.layers.Flatten(name = 'flat_2')(conc)
topic_flat = Dense(2000,name = 'first_dense')(flat_2)
topic = Dropout(0.45,name = 'drop_1')(topic_flat)
topic = Dense(256,activation='relu')(topic)
topic = Dropout(0.2,name = 'drop_2')(topic)
final = Dense(24,activation='softmax')(topic)


cent_model = Model(inputs = base_model.input,outputs = final)


In [ ]:
opti = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, name="Adam")
cent_model.compile(optimizer=opti,loss='categorical_crossentropy',metrics=[keras.metrics.Precision(),keras.metrics.CategoricalAccuracy()])

In [ ]:
cent_model.summary()

In [ ]:
cent_model.fit(train_ds,verbose = 1,use_multiprocessing=False,workers=4,epochs = 20)

In [ ]:
cent_model.save('content/drive/myDrive/saved_models/test_model')

##Implementation 4

With Max Pooling 

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
      
    self.w1 = self.add_weight(name = 'weight_topic_1',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
    self.w2 = self.add_weight(name = 'weight_topic_2',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())  
    
    self.w3 = self.add_weight(name = 'weight_topic_3',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())       
    
    self.w4 = self.add_weight(name = 'weight_topic_4',shape = (1, self.units),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
  def call(self, input1, input2, input3,input4):
    #Sum of the softmax output*topic weights
    topics = tf.multiply(input1,self.w1), tf.multiply(input2, self.w2), tf.multiply(input3, self.w3),tf.multiply(input4, self.w4)
    
    return topics


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
#for layer in model.layers:
#  layer.trainable = True

wt_add = Wt_Topics(1000)
sum_layer = tf.stack(wt_add(base_model.output,base_model.output,base_model.output,base_model.output),axis = -1)


max_pool = tf.keras.layers.MaxPool1D(pool_size=4)(sum_layer)
flat_2 = tf.keras.layers.Flatten(name = 'flat_2')(max_pool)
topic_flat = tf.keras.layers.Dense(1000,name = 'first_dense')(flat_2)
topic = tf.keras.layers.Dropout(0.45,name = 'drop_1')(topic_flat)
topic = tf.keras.layers.Dense(256,activation='relu')(topic)
topic = tf.keras.layers.Dropout(0.2,name = 'drop_2')(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)

###Implementation without custom layer

In [ ]:
#y = base_model.output
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
k_1 = tf.keras.layers.Dense(1000,name = 'topic_1',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_2 = tf.keras.layers.Dense(1000,name = 'topic_2',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_3 = tf.keras.layers.Dense(1000,name = 'topic_3',use_bias=False,kernel_initializer='random_normal',activation = 'relu')(base_model.output)
k_4 = tf.keras.layers.Dense(1000,name = 'topic_4',use_bias=False,kernel_initializer='random_normal',activation = 'relu')(base_model.output)
stack = tf.stack([k_1,k_2,k_3,k_4],axis=-1)
max_pool = tf.keras.layers.MaxPool1D(pool_size=4)(stack)
#conc = tf.keras.layers.Concatenate(name = 'conc')([k_1,k_2,k_3,k_4])
flat = tf.keras.layers.Flatten(name = 'flat')(max_pool)
# #add = tf.keras.layers.Add(name = 'add_layer')([k_1,k_2,k_3,k_4])
topic = tf.keras.layers.Dense(1000,activation='relu',name = 'first_dense')(flat)
topic = tf.keras.layers.Dropout(0.2)(topic)
topic = tf.keras.layers.Dense(256,activation='softmax',name = 'second_dense')(topic)
topic = tf.keras.layers.Dropout(0.2)(topic)
final = tf.keras.layers.Dense(24,activation='softmax',name = 'places')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
cent_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model,to_file = 'impl_5.png',  show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [ ]:
tf.rank(cent_model.get_layer('wt__topics_15').output)

##Implementation 5

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
#for layer in model.layers:
#  layer.trainable = True

wt_add = Wt_Topics(1000)
sum_layer = tf.stack(wt_add(base_model.output,base_model.output,base_model.output,base_model.output),axis = -1)
#Entire sum_layer
full = tf.keras.layers.Concatenate(axis = -1)(tf.unstack(sum_layer,axis = -1))

dense_full = tf.keras.layers.Dense(1000,name = 'dense_full',activation='relu')(full)
#Pool 1
max_pool = tf.keras.layers.MaxPool1D(pool_size=4,name = 'pool_1')(sum_layer)
flat_2 = tf.keras.layers.Flatten(name = 'flat_2')(max_pool)
#Pool 2
max_pool_2 = tf.keras.layers.MaxPool1D(pool_size=2,name = 'pool_2')(sum_layer)
uns = tf.unstack(max_pool_2,axis = -1)
test_layer = tf.keras.layers.Concatenate()(uns)
flat_3 = tf.keras.layers.Flatten(name = 'flat_3')(test_layer)
dense_pool = tf.keras.layers.Dense(1000,name = 'dense_pool',activation = 'relu')(flat_3)

comb_layer = tf.keras.layers.Add(name = 'comb')([dense_full,flat_2,dense_pool])

topic_flat = tf.keras.layers.Dense(1000,name = 'first_dense',activation='relu')(comb_layer)
topic = tf.keras.layers.Dropout(0.25,name = 'drop_1')(topic_flat)
topic = tf.keras.layers.Dense(256,activation='relu')(topic)
topic = tf.keras.layers.Dropout(0.2,name = 'drop_2')(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)

###Implementation without custom layer

In [ ]:
tf.keras.utils.plot_model(
    cent_model,to_file = 'impl_5_wo_custom.png',  show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [ ]:
#y = base_model.output
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
k_1 = tf.keras.layers.Dense(1000,name = 'topic_1',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_2 = tf.keras.layers.Dense(1000,name = 'topic_2',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_3 = tf.keras.layers.Dense(1000,name = 'topic_3',use_bias=False,kernel_initializer='random_normal',activation = 'relu')(base_model.output)
k_4 = tf.keras.layers.Dense(1000,name = 'topic_4',use_bias=False,kernel_initializer='random_normal',activation = 'relu')(base_model.output)

conc_1 = tf.keras.layers.Concatenate(name = 'full_tops')([k_1,k_2,k_3,k_4])
dense_full = tf.keras.layers.Dense(1000,name = 'dense_full')(conc_1)
#Creating first pooling layer
stack = tf.stack([k_1,k_2,k_3,k_4],axis=-1)
max_pool = tf.keras.layers.MaxPool1D(pool_size=4,name = 'pool_1')(stack)
flat = tf.keras.layers.Flatten(name = 'flat')(max_pool)

#Creating second pooling layer
max_pool_2 = tf.keras.layers.MaxPool1D(pool_size=2,name = 'pool_2')(stack)
flat_2 = tf.keras.layers.Flatten(name = 'flat_2')(max_pool_2)
dense_pool = tf.keras.layers.Dense(1000,name = 'dense_pool',activation = 'relu')(flat_2)

#Combining the layers
comb_layer = tf.keras.layers.Add(name = 'comb')([dense_full,flat,dense_pool])

topic = tf.keras.layers.Dense(1000,activation='relu',name = 'first_dense')(comb_layer)
topic = tf.keras.layers.Dropout(0.2)(topic)
topic = tf.keras.layers.Dense(256,activation='softmax',name = 'second_dense')(topic)
topic = tf.keras.layers.Dropout(0.2)(topic)
final = tf.keras.layers.Dense(24,activation='softmax',name = 'places')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
cent_model.summary()

##Implementation 6

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

In [ ]:
img = keras.preprocessing.image.load_img(
    '/content/gdrive/MyDrive/data/train/Museum/20160609_074119_000.jpg', target_size=(224,224,3)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
x= img_array
x = preprocess_input(x)

In [ ]:
prediction_test = base_model.predict(x)
pred = np.argmax(prediction_test_2)

In [ ]:
topic_df_2 = pd.DataFrame(create_topics(prediction_test))
topic_df_2.columns = ['Topic']

In [ ]:
topic_df.iloc[pred]

In [ ]:
decode_predictions(prediction_test)

In [ ]:
topic_df_3.equals(topic_df_2)

In [ ]:
topic_df_2

###Making declaration for storing the topics values for use in making TF-IDF per class

In [ ]:
def create_topics(predictions):
  topics = decode_predictions(predictions,top = 1000)
  t = pd.DataFrame(topics)
  lis = []
  for top in t:
    a,b,c = t[top].explode()
    lis.append(b)
  return lis

def get_topic_from_image(image_name):
  #topic_list = []
  img = keras.preprocessing.image.load_img(image_name, target_size=(224,224,3))
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  x= img_array
  predict = base_model.predict(x)
  pred = np.argmax(predict)
  img_topic = topic_df['Topic'].iloc[pred]
  return img_topic

def place_topics(place):
  topics_per_class = []
  class_name = place
  df_class_images = csv_data[csv_data['Label']==class_name]
  image_list = list(df_class_images['Image'])
  for image in image_list:
    print(image)
    img_topic = get_topic_from_image(image)
    topics_per_class.append(img_topic)
  return topics_per_class

def topic_place_dict(filename,dir):
  t_p_dict = {}
  csv_data = pd.read_csv(filename)
  places = os.listdir(dir)
  for place in places:
    topics = place_topics(place)
    t_p_dict.update({place:topics})
  return t_p_dict


In [ ]:
def create_topics(predictions):
  topics = decode_predictions(predictions,top = 1000)
  t = pd.DataFrame(topics)
  lis = []
  for top in t:
    a,b,c = t[top].explode()
    lis.append(b)
  return lis

In [ ]:
def get_topic_from_image(image_name):
  #topic_list = []
  img = keras.preprocessing.image.load_img(image_name, target_size=(224,224,3))
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  x= img_array
  predict = base_model.predict(x)
  pred = np.argmax(predict)
  img_topic = topic_df['Topic'].iloc[pred]
  return img_topic

In [ ]:
def place_topics(place):
  topics_per_class = []
   
  class_name = place
  df_class_images = csv_data[csv_data['Label']==class_name]
  image_list = list(df_class_images['Image'])
  for image in image_list:
    print('Loaded:'+image)
    img_topic = get_topic_from_image(image)
    topics_per_class.append(img_topic)
  return topics_per_class

In [ ]:
def topic_place_dict(filename,dir):
  t_p_dict = {}
  csv_data = pd.read_csv(filename)
  places = os.listdir(dir)
  for place in places:
    topics = place_topics(place)
    t_p_dict.update({place:topics})
  return t_p_dict

In [ ]:
topic_place_dict('/content/gdrive/MyDrive/data/short_df_collab.csv','/content/gdrive/MyDrive/data/train')

In [ ]:
office_list = place_topics('Water')

In [ ]:
test = {'Office':office_list,'Label_2':['abc','cde']}

In [ ]:
test['Label_2']

In [ ]:
office_list

## Impelmentation 7


In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Flatten(name ="flatten")(x)
#x = Dense(1024,activation="relu")(x)
#x = Dropout(0.5)(x)
#preds = Dense(1000,activation="softmax")(x)

#Activating the model

#model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
#for layer in model.layers:
#    layer.trainable = False


In [ ]:
#y = base_model.output
k_1 = tf.keras.layers.Dense(1,name = 'topic_1',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_2 = tf.keras.layers.Dense(1,name = 'topic_2',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_3 = tf.keras.layers.Dense(1,name = 'topic_3',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
k_4 = tf.keras.layers.Dense(1,name = 'topic_4',use_bias=False,kernel_initializer='random_normal',activation='relu')(base_model.output)
conc = tf.keras.layers.Concatenate(name = 'conc')([k_1,k_2,k_3,k_4])
flat = tf.keras.layers.Flatten(name = 'flat')(conc)
topic = tf.keras.layers.Dense(256,activation='relu')(flat)
topic = tf.keras.layers.Dropout(0.2)(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
%pip install numpy==1.19.5

In [ ]:
cent_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_imp7.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False,dpi=128
)

In [ ]:
cent_model.get_layer('topic_1').get_weights()

## Impelmentation 8

With skip connections



In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Flatten(name ="flatten")(x)
#x = Dense(1024,activation="relu")(x)
#x = Dropout(0.5)(x)
#preds = Dense(1000,activation="softmax")(x)

#Activating the model

#model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
#for layer in model.layers:
#    layer.trainable = False


In [ ]:
k_1

In [ ]:
#y = base_model.output
k_1 = tf.keras.layers.Dense(1000,name = 'topic_1',use_bias=False,kernel_initializer='random_normal')(base_model.output)
k_2 = tf.keras.layers.Dense(1000,name = 'topic_2',use_bias=False,kernel_initializer='random_normal')(base_model.output)
k_3 = tf.keras.layers.Dense(1000,name = 'topic_3',use_bias=False,kernel_initializer='random_normal')(base_model.output)
k_4 = tf.keras.layers.Dense(1000,name = 'topic_4',use_bias=False,kernel_initializer='random_normal')(base_model.output)
#conc = tf.keras.layers.Concatenate(name = 'conc')([k_1,k_2,k_3,k_4])
#flat = tf.keras.layers.Flatten(name = 'flat')(conc)
add = tf.keras.layers.Add(name = 'add_layer')([k_1,k_2,k_3,k_4])
topic = tf.keras.layers.Dense(256,activation='relu')(add)
topic = tf.keras.layers.Dropout(0.2)(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
k_1.output

In [ ]:
%pip install numpy==1.19.5

In [ ]:
cent_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_imp8.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False,dpi=128
)

In [ ]:
opti = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, name="Adam")
cent_model.compile(optimizer=opti,loss='categorical_crossentropy',metrics=[keras.metrics.Precision(),keras.metrics.CategoricalAccuracy()])


In [ ]:
mo = OutputCallback()
mo

In [ ]:
history = cent_model.fit(train_ds,epochs=10,shuffle=True,steps_per_epoch=10)
cent_model.save('modelll')

In [ ]:
cent_model.get_layer('predictions')

##Implementation 9

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
      
    self.w1 = self.add_weight(name = 'weight_topic_1',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
    self.w2 = self.add_weight(name = 'weight_topic_2',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())  
    
    self.w3 = self.add_weight(name = 'weight_topic_3',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())       
    
    self.w4 = self.add_weight(name = 'weight_topic_4',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
  def call(self, input1, input2, input3,input4):
    #Sum of the softmax output*topic weights
    
    return tf.multiply(input1,self.w1), tf.multiply(input2, self.w2),tf.multiply(input3, self.w3),tf.multiply(input4, self.w4)


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
#for layer in model.layers:
#  layer.trainable = True

wt_add = Wt_Topics(1000)
#new_tensor = tf.reshape(wt_add, shape=[tf.shape(wt_add)[0]*tf.shape(wt_add)[1],4])
#sum_layer = wt_add(k_1,k_2,k_3,k_4)
sum_layer = wt_add(base_model.output,base_model.output,base_model.output,base_model.output)

conc = keras.layers.Concatenate()(sum_layer)
#flat = keras.layers.GlobalAveragePooling1D()(conc)
#test = keras.layers.GlobalAveragePooling1D()(flat)
flat_2 = tf.keras.layers.Flatten(name = 'flat_2')(conc)
topic_flat = tf.keras.layers.Dense(1000,name = 'first_dense')(flat_2)
topic = tf.keras.layers.Dropout(0.45,name = 'drop_1')(topic_flat)
topic = tf.keras.layers.Dense(256,activation='relu')(topic)
topic = tf.keras.layers.Dropout(0.2,name = 'drop_2')(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
cent_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_imp9.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False,dpi=128
)

In [ ]:
abc = np.array(cent_model.get_layer('wt__topics_6').get_weights())
abc

##Implementation 10

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    self.sum1 = 0
    self.sum2 = 0
    self.sum3 = 0
    self.sum4 = 0
    # self.name = name      

  def build(self,input_shape):
    
      
    self.w1 = self.add_weight(name = 'weight_topic_1',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
    self.w2 = self.add_weight(name = 'weight_topic_2',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())  
    
    self.w3 = self.add_weight(name = 'weight_topic_3',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())       
    
    self.w4 = self.add_weight(name = 'weight_topic_4',shape = (self.units,),trainable = True,initializer = 'random_normal',regularizer = tf.keras.regularizers.l1_l2())
    
  def call(self, input1, input2, input3,input4):
    #Sum of the softmax output*topic weights
    self.sum1 += tf.multiply(input1,self.w1)
    self.sum2 +=  tf.multiply(input2, self.w2)
    self.sum3 +=  tf.multiply(input3, self.w3)
    self.sum4 += tf.multiply(input4, self.w4)
    return self.sum1,self.sum2,self.sum3,self.sum4


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))



for layer in base_model.layers:
  layer.trainable = False
#for layer in model.layers:
#  layer.trainable = True

wt_add = Wt_Topics(1000)
#new_tensor = tf.reshape(wt_add, shape=[tf.shape(wt_add)[0]*tf.shape(wt_add)[1],4])
#sum_layer = wt_add(k_1,k_2,k_3,k_4)
sum_layer = wt_add(base_model.output,base_model.output,base_model.output,base_model.output)

conc = keras.layers.Concatenate()(sum_layer)
#flat = keras.layers.GlobalAveragePooling1D()(conc)
#test = keras.layers.GlobalAveragePooling1D()(flat)
flat_2 = tf.keras.layers.Flatten(name = 'flat_2')(conc)
topic_flat = tf.keras.layers.Dense(1024,name = 'first_dense')(flat_2)
topic = tf.keras.layers.Dropout(0.45,name = 'drop_1')(topic_flat)
topic = tf.keras.layers.Dense(256,activation='relu')(topic)
topic = tf.keras.layers.Dropout(0.2,name = 'drop_2')(topic)
final = tf.keras.layers.Dense(24,activation='softmax')(topic)


cent_model = tf.keras.models.Model(inputs = base_model.input,outputs = final)


In [ ]:
cent_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_imp10.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False,dpi=128
)

In [ ]:
files.download('/content/topic_model_imp9.png')

# Evaluating the Topic Models

##Prediction for Implementation 4

###Testing

In [ ]:
test_data = pd.read_csv('/content/gdrive/MyDrive/data/test (1).csv')
test_data = test_data[test_data['Label'] == 'Office']
test_data = test_data['Image'].apply(lambda x:x.replace('/data/s4133366/data','/content/gdrive/MyDrive/data'))


In [ ]:
test = pd.read_csv('/content/place.csv')
test = test[['Image','Object']]
test

In [ ]:
test = test[['Object','Image']]

In [ ]:
total

In [ ]:
total

In [ ]:
test

In [ ]:
test.Object.value_counts().sort_values().plot(kind = 'barh')
plt.xlabel('Number of Images')
plt.ylabel('Objects')
plt.title('Top 10 Objects highlighted by Resnet-50 for Place: Water')
plt.xticks([])


In [ ]:
data

In [ ]:
total = test.groupby('Object').count()
bar1 = sn.barplot(x="Object", data=total, color='darkblue')


In [ ]:
data

In [ ]:
data = create_numpy_img('/content/gdrive/MyDrive/data/test/Office/20160621_102654_000.jpg')

In [ ]:
correct = []
incorrect = []

In [ ]:
for test in test_data:
  data = create_numpy_img(test)
# cent_model = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models/topic_weight_k4_imp_4_4_eval')
  prediction = cent_model.predict(data)
  preds = np.argmax(prediction)
  if preds == 14:
    correct.append(test)
  else:
    print('Predicted Value = ',preds)
    incorrect.append(test)  

In [ ]:
corr = pd.DataFrame(correct)
corr.to_csv('correct.csv')
files.download('correct.csv')

###Loading the saved model

In [ ]:
cent_model = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models/topic_weight_k4_places_20_imp1_1_eval')

###Output Functions

In [ ]:
from keras import backend as K
get_base_output = K.function([cent_model.input],[cent_model.get_layer('predictions').output])
get_t1_output = K.function([cent_model.input],[cent_model.get_layer('topic_1').output])
get_t2_output = K.function([cent_model.input],[cent_model.get_layer('topic_2').output])
get_t3_output = K.function([cent_model.input],[cent_model.get_layer('topic_3').output])
get_t4_output = K.function([cent_model.input],[cent_model.get_layer('topic_4').output])
# get_t5_output = K.function([cent_model.input],[cent_model.get_layer('topic_5').output])
# get_t6_output = K.function([cent_model.input],[cent_model.get_layer('topic_6').output])
# get_t7_output = K.function([cent_model.input],[cent_model.get_layer('topic_7').output])
# get_t8_output = K.function([cent_model.input],[cent_model.get_layer('topic_8').output])
# get_t9_output = K.function([cent_model.input],[cent_model.get_layer('topic_9').output])

In [ ]:
tf.keras.utils.plot_model(
    cent_model, to_file='topic_model_imp9_1.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)


In [ ]:
data = create_numpy_img('/content/gdrive/MyDrive/data/test/Office/20170608_115437_000.jpg')
# cent_model = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_models/topic_weight_k4_imp_4_4_eval')
prediction = cent_model.predict(data)
preds = np.argmax(prediction)
preds

###Resnet output

In [ ]:
x = data

In [ ]:
from keras import backend as K
get_base_output = K.function([cent_model.input],[cent_model.get_layer('predictions').output])
resnet = get_base_output(x)[0]
#print('Resnet output:',resnet)

In [ ]:
resnet_test = np.squeeze(resnet)
resnet_test.shape

In [ ]:
n=10
best_topics_resnet = np.argsort(resnet_test)[-n:]
best_topics_resnet

array([512, 418, 584, 813, 432, 677, 644, 683, 499, 111])

In [ ]:
resnet_topic_t10_values = []
for i in best_topics_resnet:
  resnet_topic_t10_values.append(resnet_test[i])

In [ ]:
resnet_topic_t10_values

In [ ]:
all_resnet_sum = np.sum(resnet_test)
top10_resnet_sum = np.sum(resnet_topic_t10_values)
print('Top 10 Sum: ',top10_resnet_sum)
print('Entire Resnet topics sum: ',all_resnet_sum)

array([[512, 418, 584, 813, 432, 677, 644, 683, 499, 111]])

###Topic 1

In [ ]:
#Topic 1
get_t1_output = K.function([cent_model.input],[cent_model.get_layer('topic_1').output])
topic_1 = get_t1_output(x)[0]
# print('Topic 1:',topic_1)
top_1 = np.divide(topic_1,resnet)
# print('Topic 1 Values',top_1)


In [ ]:
topic_1.shape

In [ ]:
n=10
topic_1_test = np.squeeze(topic_1)
best_topics_topic_1 = np.argsort(topic_1_test)[-n:]

In [ ]:
rank_topic_1 = np.argsort(topic_1_test)
best_topics_topic_1 = rank_topic_1[::-1][:n]

In [ ]:
best_topics_topic_1

array([163, 729, 585, 813,  98, 463, 268,  31, 916, 788])

In [ ]:
#Without squeeze,Dont run
#best_topics_topic_1 = np.argsort(topic_1, axis=1)[:,-n:]

####Top 10 Topic Indexes from Topic 1

In [ ]:
topic1_t10_values = []
for i in best_topics_topic_1:
  topic1_t10_values.append(topic_1_test[i])

In [ ]:
topic1_t10_values

In [ ]:
all_topic_1_sum = np.sum(topic_1_test)
top10_topic_1_sum = np.sum(topic1_t10_values)
print('Top 10 Sum: ',top10_topic_1_sum)
print('Entire Topic 1 topic sum: ',all_topic_1_sum)

####Top 10 Topics for Topic 1 taken from Resnet

In [ ]:
best_topics_resnet

In [ ]:
topic_1_t10_resnet = []
for i in best_topics_resnet:
  topic_1_t10_resnet.append(topic_1_test[i])

In [ ]:
topic_1_t10_resnet

In [ ]:
all_topic_1_sum = np.sum(topic_1_test)
top10_topic_1_sum_resnet = np.sum(topic_1_t10_resnet)
print('Top 10 Sum: ',top10_topic_1_sum_resnet)
print('Entire Topic 1 topic sum: ',all_topic_1_sum)

In [ ]:
top_1_hist = top_1.squeeze()
top_1_hist = pd.DataFrame(top_1_hist)
top_1_hist
plt.plot(top_1_hist)
plt.title('Topic 1')

###Topic 2

In [ ]:
#Topic 2
get_t2_output = K.function([cent_model.input],[cent_model.get_layer('topic_2').output])
topic_2 = get_t2_output(x)
# print('Topic 2:',topic_2)
top_2 = np.divide(topic_2,resnet)
# print('Topic 2 Values',top_2)



In [ ]:
n=10
topic_2_test = np.squeeze(topic_2)
best_topics_topic_2 = np.argsort(topic_2_test)[-n:]

In [ ]:
#Without squeeze .. Dont run
best_topics_topic_2 = np.argsort(topic_2, axis=1)[:,-n:]

array([596, 646,  74, 255, 805,  22, 405, 726, 942, 140])

In [ ]:
best_topics_topic_2

####Top 10 Topic Indexes from Topic 2

In [ ]:
topic2_t10_values = []
for i in best_topics_topic_2:
  topic2_t10_values.append(topic_2_test[i])

In [ ]:
topic2_t10_values

In [ ]:
all_topic_2_sum = np.sum(topic_2_test)
top10_topic_2_sum = np.sum(topic2_t10_values)
print('Top 10 Sum from Topic 2: ',top10_topic_2_sum)
print('Entire Topic 2 topic sum: ',all_topic_2_sum)

In [ ]:
top_2_hist = top_2.squeeze()
top_2_hist = pd.DataFrame(top_2_hist)
# top_2_hist
plt.plot(top_2_hist)
plt.title('Topic 2')

####Top 10 Topics for Topic 1 taken from Resnet

In [ ]:
best_topics_resnet

In [ ]:
topic_2_t10_resnet = []
for i in best_topics_resnet:
  topic_2_t10_resnet.append(topic_2_test[i])

In [ ]:
topic_2_t10_resnet

In [ ]:
all_topic_2_sum = np.sum(topic_2_test)
top10_topic_2_sum_resnet = np.sum(topic_2_t10_resnet)
print('Sum of Top 10 indexes from Resnet: ',top10_topic_2_sum_resnet)
print('Entire Topic 2 topic sum: ',all_topic_2_sum)

###Topic 3

In [ ]:
get_t3_output = K.function([cent_model.input],[cent_model.get_layer('topic_3').output])
topic_3 = get_t3_output(x)[0]
# print('Topic 3:',topic_3)
top_3 = np.divide(topic_3,resnet)
# print('Topic 3 Values',top_3)


In [ ]:
#Without squeeze .. Dont run
best_topics_topic_3 = np.argsort(topic_3, axis=1)[:,-n:]

In [ ]:
n =10
topic_3_test = np.squeeze(topic_3)
best_topics_topic_3 = np.argsort(topic_3_test)[-n:]

In [ ]:
best_topics_topic_3

In [ ]:
top_3_hist = top_3.squeeze()
top_3_hist = pd.DataFrame(top_3_hist)
top_3_hist
plt.plot(top_3_hist)
plt.title('Topic 3')

####Top 10 Topic Indexes from Topic 3

In [ ]:
topic3_t10_values = []
for i in best_topics_topic_3:
  topic3_t10_values.append(topic_3_test[i])

In [ ]:
topic3_t10_values

In [ ]:
all_topic_3_sum = np.sum(topic_3_test)
top10_topic_3_sum = np.sum(topic3_t10_values)
print('Top 10 Sum: ',top10_topic_3_sum)
print('Entire Topic 1 topic sum: ',all_topic_3_sum)

####Top 10 Topics for Topic 3 taken from Resnet

In [ ]:
best_topics_resnet

In [ ]:
topic_3_t10_resnet = []
for i in best_topics_resnet:
  topic_3_t10_resnet.append(topic_3_test[i])

In [ ]:
topic_3_t10_resnet

[0.0049747974,
 0.004279796,
 0.0,
 0.0034174433,
 0.0007338119,
 0.0049685836,
 0.004004097,
 0.0,
 0.0,
 0.001525932]

In [ ]:
all_topic_3_sum = np.sum(topic_3_test)
top10_topic_3_sum_resnet = np.sum(topic_3_t10_resnet)
print('Top 10 Sum: ',top10_topic_3_sum_resnet)
print('Entire Topic 1 topic sum: ',all_topic_3_sum)

###Topic 4

In [ ]:
#Topic 4
get_t4_output = K.function([cent_model.input],[cent_model.get_layer('topic_4').output])
topic_4 = get_t4_output(x)[0]
# print('Topic 4:',topic_4)
top_4 = np.divide(topic_4,resnet)
# print('Topic 4 Values',top_4)

In [ ]:
best_topics_topic_4 = np.argsort(topic_4, axis=1)[:,-n:]

In [ ]:
n=10
topic_4_test = np.squeeze(topic_4)
best_topics_topic_4 = np.argsort(topic_4_test)[-n:]
best_topics_topic_4

In [ ]:
top_4_hist = top_4.squeeze()
top_4_hist = pd.DataFrame(top_4_hist)
top_4_hist
plt.plot(top_4_hist)
plt.title('Topic 4')

####Top 10 Topic Indexes from Topic 4

In [ ]:
topic4_t10_values = []
for i in best_topics_topic_1:
  topic4_t10_values.append(topic_4_test[i])

In [ ]:
topic4_t10_values

[0.0072678076,
 0.0,
 0.015088149,
 0.006569425,
 0.0035689166,
 0.009058334,
 0.002467542,
 0.010582676,
 0.0030508689,
 0.0007867259]

In [ ]:
all_topic_4_sum = np.sum(topic_4_test)
top10_topic_4_sum = np.sum(topic4_t10_values)
print('Top 10 Sum for Topic 1: ',top10_topic_4_sum)
print('Entire Topic 1 topic sum: ',all_topic_4_sum)

####Top 10 Topics for Topic 4 taken from Resnet

In [ ]:
best_topics_resnet

In [ ]:
topic_4_t10_resnet = []
for i in best_topics_resnet:
  topic_4_t10_resnet.append(topic_4_test[i])

In [ ]:
topic_4_t10_resnet

[0.013382193,
 0.0045781448,
 0.0,
 0.006569425,
 0.0016026746,
 0.0022247692,
 0.008066697,
 0.0,
 0.006562537,
 0.0]

In [ ]:
all_topic_4_sum = np.sum(topic_4_test)
top10_topic_4_sum_resnet = np.sum(topic_4_t10_resnet)
print('Top 10 values from Resnet for Topic 4: ',top10_topic_4_sum_resnet)
print('Entire Topic 4 topic sum: ',all_topic_4_sum)

###Declaration for data processing

In [ ]:
def create_numpy_img(image):
  img = keras.preprocessing.image.load_img(
      image, target_size=(224,224,3)
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = img_array/255.
  img_array = tf.expand_dims(img_array, 0)
  x= img_array
  
  return x

###Declaration for creating image arrays with images from a class

In [ ]:
def create_numpy_img_mul(imageList):
  img_list = []
  for image in imageList:
    print('Image: ',image)
    img = keras.preprocessing.image.load_img(
        image, target_size=(224,224,3)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array/255.
    # img_array = tf.expand_dims(img_array, 0)
    img_list.append(img_array)
  x = tf.stack(img_list)

  return x

##Creating a Declaration

In [ ]:
test_data = pd.read_csv('/content/gdrive/MyDrive/data/train (4).csv')
np.unique(test_data['Label'])


In [ ]:
test_data = pd.read_csv('/content/train_p20_final.csv')
test_data['Image'] = test_data['Image'].apply(lambda x: x.replace('/data/s4133366/places365_20','/content/gdrive/MyDrive/places365_sampled')) 
test_data = test_data[test_data['Label']=='volcano']#'underwater-ocean_deep'
test_data = test_data.sample(5)
images = list(test_data['Image'])
images

In [ ]:
type(images)

In [ ]:
imgg = create_numpy_img_mul(images)
np.save('office.npy',imgg)
files.download('office.npy')

In [ ]:
np.save('office.npy',imgg)
files.download('office.npy')

In [ ]:
len(images)

In [ ]:
images = list(test_data['Image'])

In [ ]:
images

In [ ]:
def create_Topic_data(image_list,number):

  n = number
  sum_resnet_total = []
  top10_resnet_total = []
  
  #Collecting the best performing indexes
  best_topic_resnet_list = []
  best_topics_topic1_list = []
  best_topics_topic2_list = []
  best_topics_topic3_list = []
  best_topics_topic4_list = []
  all_topic_1_sum_list = []
  all_topic_2_sum_list = []
  all_topic_3_sum_list = []
  all_topic_4_sum_list = []

  top10_topic_1_total = []
  top10_topic_1_total_from_resnet = []

  top10_topic_2_total = []
  top10_topic_2_total_from_resnet = []

  top10_topic_3_total = []
  top10_topic_3_total_from_resnet = []

  top10_topic_4_total = []
  top10_topic_4_total_from_resnet = []
  class_name = image_list[0]
  class_name = class_name.split('/')[-2]
  print('Class: ',class_name)
  for image in image_list:
    print(image)
    data = create_numpy_img(image)
    x = data

    #Getting the Resnet output
    resnet = get_base_output(x)[0]
    resnet_test = np.squeeze(resnet)
    best_topics_resnet = np.argsort(resnet_test)[-n:]
    best_topic_resnet_list.append(best_topics_resnet)
    resnet_topic_t10_values = []
    for i in best_topics_resnet:
      resnet_topic_t10_values.append(resnet_test[i])
    all_resnet_sum = np.sum(resnet_test)
    top10_resnet_sum = np.sum(resnet_topic_t10_values)
    # print('Top 10 Sum: ',top10_resnet_sum)
    # print('Entire Resnet topics sum: ',all_resnet_sum)  
    
    #Adding to list
    top10_resnet_total.append(top10_resnet_sum)
    sum_resnet_total.append(all_resnet_sum)
    
    #Topic 1
    topic_1 = get_t1_output(x)[0]

    topic_1_test = np.squeeze(topic_1)
    best_topics_topic_1 = np.argsort(topic_1_test)[-n:]
    best_topics_topic1_list.append(best_topics_topic_1)

    topic1_t10_values = []
    for i in best_topics_topic_1:
      topic1_t10_values.append(topic_1_test[i])

    all_topic_1_sum = np.sum(topic_1_test)
    top10_topic_1_sum = np.sum(topic1_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_1_total.append(top10_topic_1_sum)

    topic_1_t10_resnet = []
    for i in best_topics_resnet:
      topic_1_t10_resnet.append(topic_1_test[i])

    all_topic_1_sum = np.sum(topic_1_test)
    top10_topic_1_sum_resnet = np.sum(topic_1_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_1_sum_list.append(all_topic_1_sum)

    top10_topic_1_total_from_resnet.append(top10_topic_1_sum_resnet)

    #Topic 2
    topic_2 = get_t2_output(x)
    topic_2_test = np.squeeze(topic_2)
    best_topics_topic_2 = np.argsort(topic_2_test)[-n:]
    best_topics_topic2_list.append(best_topics_topic_2)

    topic2_t10_values = []
    for i in best_topics_topic_2:
      topic2_t10_values.append(topic_2_test[i])

    all_topic_2_sum = np.sum(topic_2_test)
    top10_topic_2_sum = np.sum(topic2_t10_values)
    # print('Top 10 Sum from Topic 2: ',top10_topic_2_sum)
    # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

    top10_topic_2_total.append(top10_topic_2_sum)

    topic_2_t10_resnet = []
    for i in best_topics_resnet:
      topic_2_t10_resnet.append(topic_2_test[i])

    all_topic_2_sum = np.sum(topic_2_test)
    top10_topic_2_sum_resnet = np.sum(topic_2_t10_resnet)
    # print('Sum of Top 10 indexes from Resnet: ',top10_topic_2_sum_resnet)
    # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

    all_topic_2_sum_list.append(all_topic_2_sum)

    top10_topic_2_total_from_resnet.append(top10_topic_2_sum_resnet)

    #Topic 3
    topic_3 = get_t3_output(x)[0]
    topic_3_test = np.squeeze(topic_3)
    best_topics_topic_3 = np.argsort(topic_3_test)[-n:]
    best_topics_topic3_list.append(best_topics_topic_3)

    topic3_t10_values = []
    for i in best_topics_topic_3:
      topic3_t10_values.append(topic_3_test[i])

    all_topic_3_sum = np.sum(topic_3_test)
    top10_topic_3_sum = np.sum(topic3_t10_values)
    # print('Top 10 Sum: ',top10_topic_3_sum)
    # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

    top10_topic_3_total.append(top10_topic_3_sum)

    topic_3_t10_resnet = []
    for i in best_topics_resnet:
      topic_3_t10_resnet.append(topic_3_test[i])

    all_topic_3_sum = np.sum(topic_3_test)
    top10_topic_3_sum_resnet = np.sum(topic_3_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_3_sum_resnet)
    # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

    all_topic_3_sum_list.append(all_topic_3_sum)


    top10_topic_3_total_from_resnet.append(top10_topic_3_sum_resnet)

    #Topic 4
    topic_4 = get_t4_output(x)[0]
    topic_4_test = np.squeeze(topic_4)
    best_topics_topic_4 = np.argsort(topic_4_test)[-n:]
    best_topics_topic4_list.append(best_topics_topic_4)
    topic4_t10_values = []
    for i in best_topics_topic_1:
      topic4_t10_values.append(topic_4_test[i])

    all_topic_4_sum = np.sum(topic_4_test)
    top10_topic_4_sum = np.sum(topic4_t10_values)
    # print('Top 10 Sum for Topic 4: ',top10_topic_4_sum)
    # print('Entire Topic 4 topic sum: ',all_topic_4_sum)

    top10_topic_4_total.append(top10_topic_4_sum)

    topic_4_t10_resnet = []
    for i in best_topics_resnet:
      topic_4_t10_resnet.append(topic_4_test[i])

    all_topic_4_sum = np.sum(topic_4_test)
    top10_topic_4_sum_resnet = np.sum(topic_4_t10_resnet)
    # print('Top 10 values from Resnet for Topic 4: ',top10_topic_4_sum_resnet)
    # print('Entire Topic 4 topic sum: ',all_topic_4_sum)
    all_topic_4_sum_list.append(all_topic_4_sum)

    top10_topic_4_total_from_resnet.append(top10_topic_4_sum_resnet)
  df_sum_all_topics = pd.DataFrame(list(zip(sum_resnet_total, all_topic_1_sum_list, all_topic_2_sum_list,all_topic_3_sum_list,all_topic_4_sum_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4'])
  df_sum_all_topics['Class'] = class_name
  df_sum_all_topics.to_csv('sum_topics_'+class_name+'_imp1.csv')
  df_best_topics = pd.DataFrame(list(zip(best_topic_resnet_list, best_topics_topic1_list, best_topics_topic2_list,best_topics_topic3_list,best_topics_topic4_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4'])
  df_best_topics['Class'] = class_name
  df_best_topics.to_csv('top10_topics_'+class_name+'_imp1.csv')

  df_resnet_1 = pd.DataFrame(top10_topic_1_total_from_resnet)
  df_resnet_1['Topic'] = 'Topic 1'
  df_resnet_2 = pd.DataFrame(top10_topic_2_total_from_resnet)
  df_resnet_2['Topic'] = 'Topic 2'
  df_resnet_3 = pd.DataFrame(top10_topic_3_total_from_resnet)
  df_resnet_3['Topic'] = 'Topic 3'
  df_resnet_4 = pd.DataFrame(top10_topic_4_total_from_resnet)
  df_resnet_4['Topic'] = 'Topic 4'
  
  df_resnet = pd.concat([df_resnet_1,df_resnet_2,df_resnet_3,df_resnet_4])
  
  df_topic_1 = pd.DataFrame(top10_topic_1_total)
  df_topic_1['Topic'] = 'Topic 1'
  df_topic_2 = pd.DataFrame(top10_topic_2_total)
  df_topic_2['Topic'] = 'Topic 2'
  df_topic_3 = pd.DataFrame(top10_topic_3_total)
  df_topic_3['Topic'] = 'Topic 3'
  df_topic_4 = pd.DataFrame(top10_topic_4_total)
  df_topic_4['Topic'] = 'Topic 4'

  df_topics = pd.concat([df_topic_1,df_topic_2,df_topic_3,df_topic_4])
  df_topics.columns = ['Sum','Topic']
  df_resnet.columns = ['Sum','Topic']


  #Plotting Figures
  ax = sn.boxplot(y="Sum", x="Topic", data=df_resnet,orient='v')
  plt.title('Top 10 Objects from ResNet Softmax for Topics =4 for '+class_name)
  plt.savefig('imp4_top10_resnet_'+class_name+'_imp1.png')
  plt.clf()
  ax = sn.boxplot(y="Sum", x="Topic", data=df_topics,orient='v')
  plt.title('Top 10 Objects from Dense Topic Layers for Topics =4 for '+class_name)
  plt.savefig('imp4_top10_topiclayer_'+class_name+'_imp1.png')
  plt.clf()

  return df_resnet,df_topics

##Creating declaration for 9 topics

In [ ]:
def create_Topic_data_k9(image_list,number):

  n = number
  sum_resnet_total = []
  top10_resnet_total = []
  
  #Collecting the best performing indexes
  best_topic_resnet_list = []
  best_topics_topic1_list = []
  best_topics_topic2_list = []
  best_topics_topic3_list = []
  best_topics_topic4_list = []
  best_topics_topic5_list = []
  best_topics_topic6_list = []
  best_topics_topic7_list = []
  best_topics_topic8_list = []
  best_topics_topic9_list = []

  all_topic_1_sum_list = []
  all_topic_2_sum_list = []
  all_topic_3_sum_list = []
  all_topic_4_sum_list = []
  all_topic_5_sum_list = []
  all_topic_6_sum_list = []
  all_topic_7_sum_list = []
  all_topic_8_sum_list = []
  all_topic_9_sum_list = []

  top10_topic_1_total = []
  top10_topic_1_total_from_resnet = []

  top10_topic_2_total = []
  top10_topic_2_total_from_resnet = []

  top10_topic_3_total = []
  top10_topic_3_total_from_resnet = []

  top10_topic_4_total = []
  top10_topic_4_total_from_resnet = []

  top10_topic_5_total = []
  top10_topic_5_total_from_resnet = []

  top10_topic_6_total = []
  top10_topic_6_total_from_resnet = []

  top10_topic_7_total = []
  top10_topic_7_total_from_resnet = []

  top10_topic_8_total = []
  top10_topic_8_total_from_resnet = []

  top10_topic_9_total = []
  top10_topic_9_total_from_resnet = []

  class_name = image_list[0]
  class_name = class_name.split('/')[-2]
  print('Class: ',class_name)
  for image in image_list:
    print(image)
    data = create_numpy_img(image)
    x = data

    #Getting the Resnet output
    resnet = get_base_output(x)[0]
    resnet_test = np.squeeze(resnet)
    best_topics_resnet = np.argsort(resnet_test)[-n:]
    best_topic_resnet_list.append(best_topics_resnet)
    resnet_topic_t10_values = []
    for i in best_topics_resnet:
      resnet_topic_t10_values.append(resnet_test[i])
    all_resnet_sum = np.sum(resnet_test)
    top10_resnet_sum = np.sum(resnet_topic_t10_values)
    # print('Top 10 Sum: ',top10_resnet_sum)
    # print('Entire Resnet topics sum: ',all_resnet_sum)  
    
    #Adding to list
    top10_resnet_total.append(top10_resnet_sum)
    sum_resnet_total.append(all_resnet_sum)
    
    #Topic 1
    topic_1 = get_t1_output(x)[0]

    topic_1_test = np.squeeze(topic_1)
    best_topics_topic_1 = np.argsort(topic_1_test)[-n:]
    best_topics_topic1_list.append(best_topics_topic_1)

    topic1_t10_values = []
    for i in best_topics_topic_1:
      topic1_t10_values.append(topic_1_test[i])

    all_topic_1_sum = np.sum(topic_1_test)
    top10_topic_1_sum = np.sum(topic1_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_1_total.append(top10_topic_1_sum)

    topic_1_t10_resnet = []
    for i in best_topics_resnet:
      topic_1_t10_resnet.append(topic_1_test[i])

    all_topic_1_sum = np.sum(topic_1_test)
    top10_topic_1_sum_resnet = np.sum(topic_1_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_1_sum_list.append(all_topic_1_sum)

    top10_topic_1_total_from_resnet.append(top10_topic_1_sum_resnet)

    #Topic 2
    topic_2 = get_t2_output(x)
    topic_2_test = np.squeeze(topic_2)
    best_topics_topic_2 = np.argsort(topic_2_test)[-n:]
    best_topics_topic2_list.append(best_topics_topic_2)

    topic2_t10_values = []
    for i in best_topics_topic_2:
      topic2_t10_values.append(topic_2_test[i])

    all_topic_2_sum = np.sum(topic_2_test)
    top10_topic_2_sum = np.sum(topic2_t10_values)
    # print('Top 10 Sum from Topic 2: ',top10_topic_2_sum)
    # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

    top10_topic_2_total.append(top10_topic_2_sum)

    topic_2_t10_resnet = []
    for i in best_topics_resnet:
      topic_2_t10_resnet.append(topic_2_test[i])

    all_topic_2_sum = np.sum(topic_2_test)
    top10_topic_2_sum_resnet = np.sum(topic_2_t10_resnet)
    # print('Sum of Top 10 indexes from Resnet: ',top10_topic_2_sum_resnet)
    # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

    all_topic_2_sum_list.append(all_topic_2_sum)

    top10_topic_2_total_from_resnet.append(top10_topic_2_sum_resnet)

    #Topic 3
    topic_3 = get_t3_output(x)[0]
    topic_3_test = np.squeeze(topic_3)
    best_topics_topic_3 = np.argsort(topic_3_test)[-n:]
    best_topics_topic3_list.append(best_topics_topic_3)

    topic3_t10_values = []
    for i in best_topics_topic_3:
      topic3_t10_values.append(topic_3_test[i])

    all_topic_3_sum = np.sum(topic_3_test)
    top10_topic_3_sum = np.sum(topic3_t10_values)
    # print('Top 10 Sum: ',top10_topic_3_sum)
    # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

    top10_topic_3_total.append(top10_topic_3_sum)

    topic_3_t10_resnet = []
    for i in best_topics_resnet:
      topic_3_t10_resnet.append(topic_3_test[i])

    all_topic_3_sum = np.sum(topic_3_test)
    top10_topic_3_sum_resnet = np.sum(topic_3_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_3_sum_resnet)
    # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

    all_topic_3_sum_list.append(all_topic_3_sum)


    top10_topic_3_total_from_resnet.append(top10_topic_3_sum_resnet)

    #Topic 4
    topic_4 = get_t4_output(x)[0]
    topic_4_test = np.squeeze(topic_4)
    best_topics_topic_4 = np.argsort(topic_4_test)[-n:]
    best_topics_topic4_list.append(best_topics_topic_4)
    topic4_t10_values = []
    for i in best_topics_topic_1:
      topic4_t10_values.append(topic_4_test[i])

    all_topic_4_sum = np.sum(topic_4_test)
    top10_topic_4_sum = np.sum(topic4_t10_values)
    # print('Top 10 Sum for Topic 4: ',top10_topic_4_sum)
    # print('Entire Topic 4 topic sum: ',all_topic_4_sum)

    top10_topic_4_total.append(top10_topic_4_sum)

    topic_4_t10_resnet = []
    for i in best_topics_resnet:
      topic_4_t10_resnet.append(topic_4_test[i])

    all_topic_4_sum = np.sum(topic_4_test)
    top10_topic_4_sum_resnet = np.sum(topic_4_t10_resnet)
    # print('Top 10 values from Resnet for Topic 4: ',top10_topic_4_sum_resnet)
    # print('Entire Topic 4 topic sum: ',all_topic_4_sum)
    all_topic_4_sum_list.append(all_topic_4_sum)

    top10_topic_4_total_from_resnet.append(top10_topic_4_sum_resnet)

    #Topic 5
    topic_5 = get_t5_output(x)[0]

    topic_5_test = np.squeeze(topic_1)
    best_topics_topic_5 = np.argsort(topic_5_test)[-n:]
    best_topics_topic5_list.append(best_topics_topic_5)

    topic5_t10_values = []
    for i in best_topics_topic_5:
      topic5_t10_values.append(topic_5_test[i])

    all_topic_5_sum = np.sum(topic_5_test)
    top10_topic_5_sum = np.sum(topic5_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_5_total.append(top10_topic_5_sum)

    topic_5_t10_resnet = []
    for i in best_topics_resnet:
      topic_5_t10_resnet.append(topic_5_test[i])

    all_topic_5_sum = np.sum(topic_5_test)
    top10_topic_5_sum_resnet = np.sum(topic_5_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_5_sum_list.append(all_topic_5_sum)

    top10_topic_5_total_from_resnet.append(top10_topic_5_sum_resnet)

    #Topic 6
        
    topic_6 = get_t6_output(x)[0]

    topic_6_test = np.squeeze(topic_6)
    best_topics_topic_6 = np.argsort(topic_6_test)[-n:]
    best_topics_topic6_list.append(best_topics_topic_6)

    topic6_t10_values = []
    for i in best_topics_topic_6:
      topic6_t10_values.append(topic_6_test[i])

    all_topic_6_sum = np.sum(topic_6_test)
    top10_topic_6_sum = np.sum(topic6_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_6_total.append(top10_topic_6_sum)

    topic_6_t10_resnet = []
    for i in best_topics_resnet:
      topic_6_t10_resnet.append(topic_6_test[i])

    all_topic_6_sum = np.sum(topic_6_test)
    top10_topic_6_sum_resnet = np.sum(topic_6_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_6_sum_list.append(all_topic_6_sum)

    top10_topic_6_total_from_resnet.append(top10_topic_6_sum_resnet)

    #Topic 7

    
    topic_7 = get_t7_output(x)[0]

    topic_7_test = np.squeeze(topic_7)
    best_topics_topic_7 = np.argsort(topic_7_test)[-n:]
    best_topics_topic7_list.append(best_topics_topic_7)

    topic7_t10_values = []
    for i in best_topics_topic_7:
      topic7_t10_values.append(topic_7_test[i])

    all_topic_7_sum = np.sum(topic_7_test)
    top10_topic_7_sum = np.sum(topic7_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_7_total.append(top10_topic_7_sum)

    topic_7_t10_resnet = []
    for i in best_topics_resnet:
      topic_7_t10_resnet.append(topic_7_test[i])

    all_topic_7_sum = np.sum(topic_7_test)
    top10_topic_7_sum_resnet = np.sum(topic_7_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_7_sum_list.append(all_topic_7_sum)

    top10_topic_7_total_from_resnet.append(top10_topic_7_sum_resnet)
  
    #Topic 8

    topic_8 = get_t8_output(x)[0]

    topic_8_test = np.squeeze(topic_8)
    best_topics_topic_8 = np.argsort(topic_8_test)[-n:]
    best_topics_topic8_list.append(best_topics_topic_8)

    topic8_t10_values = []
    for i in best_topics_topic_8:
      topic8_t10_values.append(topic_8_test[i])

    all_topic_8_sum = np.sum(topic_8_test)
    top10_topic_8_sum = np.sum(topic8_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_8_total.append(top10_topic_8_sum)

    topic_8_t10_resnet = []
    for i in best_topics_resnet:
      topic_8_t10_resnet.append(topic_8_test[i])

    all_topic_8_sum = np.sum(topic_8_test)
    top10_topic_8_sum_resnet = np.sum(topic_8_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_8_sum_list.append(all_topic_8_sum)

    top10_topic_8_total_from_resnet.append(top10_topic_8_sum_resnet)

    #Topic 9

    topic_9 = get_t9_output(x)[0]

    topic_9_test = np.squeeze(topic_9)
    best_topics_topic_9 = np.argsort(topic_9_test)[-n:]
    best_topics_topic9_list.append(best_topics_topic_9)

    topic9_t10_values = []
    for i in best_topics_topic_9:
      topic9_t10_values.append(topic_9_test[i])

    all_topic_9_sum = np.sum(topic_9_test)
    top10_topic_9_sum = np.sum(topic9_t10_values)
    # print('Top 10 Sum: ',top10_topic_1_sum)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    #Adding to list
    top10_topic_9_total.append(top10_topic_9_sum)

    topic_9_t10_resnet = []
    for i in best_topics_resnet:
      topic_9_t10_resnet.append(topic_9_test[i])

    all_topic_9_sum = np.sum(topic_9_test)
    top10_topic_9_sum_resnet = np.sum(topic_9_t10_resnet)
    # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
    # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

    all_topic_9_sum_list.append(all_topic_9_sum)

    top10_topic_9_total_from_resnet.append(top10_topic_9_sum_resnet)

  df_sum_all_topics = pd.DataFrame(list(zip(sum_resnet_total, all_topic_1_sum_list, all_topic_2_sum_list,all_topic_3_sum_list,all_topic_4_sum_list,all_topic_5_sum_list,all_topic_6_sum_list,all_topic_7_sum_list,all_topic_8_sum_list,all_topic_9_sum_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4','Topic 5','Topic 6','Topic 7','Topic 8','Topic 9'])
  df_sum_all_topics['Class'] = class_name
  df_sum_all_topics.to_csv('sum_topics_k9_'+class_name+'_imp2.csv')
  df_best_topics = pd.DataFrame(list(zip(best_topic_resnet_list, best_topics_topic1_list, best_topics_topic2_list,best_topics_topic3_list,best_topics_topic4_list,best_topics_topic5_list,best_topics_topic6_list,best_topics_topic7_list,best_topics_topic8_list,best_topics_topic9_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4','Topic 5','Topic 6','Topic 7','Topic 8','Topic 9'])
  df_best_topics['Class'] = class_name
  df_best_topics.to_csv('top10_topics_k9_'+class_name+'_imp2.csv')

  df_resnet_1 = pd.DataFrame(top10_topic_1_total_from_resnet)
  df_resnet_1['Topic'] = 'Topic 1'
  df_resnet_2 = pd.DataFrame(top10_topic_2_total_from_resnet)
  df_resnet_2['Topic'] = 'Topic 2'
  df_resnet_3 = pd.DataFrame(top10_topic_3_total_from_resnet)
  df_resnet_3['Topic'] = 'Topic 3'
  df_resnet_4 = pd.DataFrame(top10_topic_4_total_from_resnet)
  df_resnet_4['Topic'] = 'Topic 4'
  df_resnet_5 = pd.DataFrame(top10_topic_5_total_from_resnet)
  df_resnet_5['Topic'] = 'Topic 5'
  df_resnet_6 = pd.DataFrame(top10_topic_6_total_from_resnet)
  df_resnet_6['Topic'] = 'Topic 6'
  df_resnet_7 = pd.DataFrame(top10_topic_7_total_from_resnet)
  df_resnet_7['Topic'] = 'Topic 7'
  df_resnet_8 = pd.DataFrame(top10_topic_8_total_from_resnet)
  df_resnet_8['Topic'] = 'Topic 8'
  df_resnet_9 = pd.DataFrame(top10_topic_9_total_from_resnet)
  df_resnet_9['Topic'] = 'Topic 9'
  
  
  
  

  df_resnet = pd.concat([df_resnet_1,df_resnet_2,df_resnet_3,df_resnet_4,df_resnet_5,df_resnet_6,df_resnet_7,df_resnet_8,df_resnet_9])
  
  df_topic_1 = pd.DataFrame(top10_topic_1_total)
  df_topic_1['Topic'] = 'Topic 1'
  df_topic_2 = pd.DataFrame(top10_topic_2_total)
  df_topic_2['Topic'] = 'Topic 2'
  df_topic_3 = pd.DataFrame(top10_topic_3_total)
  df_topic_3['Topic'] = 'Topic 3'
  df_topic_4 = pd.DataFrame(top10_topic_4_total)
  df_topic_4['Topic'] = 'Topic 4'
  df_topic_5 = pd.DataFrame(top10_topic_5_total)
  df_topic_5['Topic'] = 'Topic 5'
  df_topic_6 = pd.DataFrame(top10_topic_6_total)
  df_topic_6['Topic'] = 'Topic 6'
  df_topic_7 = pd.DataFrame(top10_topic_7_total)
  df_topic_7['Topic'] = 'Topic 7'
  df_topic_8 = pd.DataFrame(top10_topic_8_total)
  df_topic_8['Topic'] = 'Topic 8'
  df_topic_9 = pd.DataFrame(top10_topic_9_total)
  df_topic_9['Topic'] = 'Topic 9'

  df_topics = pd.concat([df_topic_1,df_topic_2,df_topic_3,df_topic_4,df_topic_5,df_topic_6,df_topic_7,df_topic_8,df_topic_9])
  df_topics.columns = ['Sum','Topic']
  df_resnet.columns = ['Sum','Topic']


  #Plotting Figures
  ax = sn.boxplot(y="Sum", x="Topic", data=df_resnet,orient='v')
  plt.title('Top 10 Objects from ResNet Softmax for Topics =9 for '+class_name)
  plt.savefig('k9_top10_resnet_'+class_name+'_imp2.png')
  plt.clf()
  ax = sn.boxplot(y="Sum", x="Topic", data=df_topics,orient='v')
  plt.title('Top 10 Objects from Dense Topic Layers for Topics =9 for '+class_name)
  plt.savefig('k9_top10_topiclayer_'+class_name+'_imp2.png')
  plt.clf()

  return df_resnet,df_topics

In [ ]:
def create_Topic_data_mul(image_list,number):

  n = number
  sum_resnet_total = []
  top10_resnet_total = []
  
  #Collecting the best performing indexes
  best_topic_resnet_list = []
  best_topics_topic1_list = []
  best_topics_topic2_list = []
  best_topics_topic3_list = []
  best_topics_topic4_list = []
  all_topic_1_sum_list = []
  all_topic_2_sum_list = []
  all_topic_3_sum_list = []
  all_topic_4_sum_list = []

  top10_topic_1_total = []
  top10_topic_1_total_from_resnet = []

  top10_topic_2_total = []
  top10_topic_2_total_from_resnet = []

  top10_topic_3_total = []
  top10_topic_3_total_from_resnet = []

  top10_topic_4_total = []
  top10_topic_4_total_from_resnet = []
  class_name = image_list[0]
  class_name = class_name.split('/')[-2]
  print('Class: ',class_name)
  # for image in image_list:
  data = create_numpy_img(image)
  x = data

  #Getting the Resnet output
  resnet = get_base_output(x)[0]
  resnet_test = np.squeeze(resnet)
  best_topics_resnet = np.argsort(resnet_test)[-n:]
  best_topic_resnet_list.append(best_topics_resnet)
  resnet_topic_t10_values = []
  for i in best_topics_resnet:
    resnet_topic_t10_values.append(resnet_test[i])
  all_resnet_sum = np.sum(resnet_test)
  top10_resnet_sum = np.sum(resnet_topic_t10_values)
  # print('Top 10 Sum: ',top10_resnet_sum)
  # print('Entire Resnet topics sum: ',all_resnet_sum)  
  
  #Adding to list
  top10_resnet_total.append(top10_resnet_sum)
  sum_resnet_total.append(all_resnet_sum)
  
  #Topic 1
  topic_1 = get_t1_output(x)[0]

  topic_1_test = np.squeeze(topic_1)
  best_topics_topic_1 = np.argsort(topic_1_test)[-n:]
  best_topics_topic1_list.append(best_topics_topic_1)

  topic1_t10_values = []
  for i in best_topics_topic_1:
    topic1_t10_values.append(topic_1_test[i])

  all_topic_1_sum = np.sum(topic_1_test)
  top10_topic_1_sum = np.sum(topic1_t10_values)
  # print('Top 10 Sum: ',top10_topic_1_sum)
  # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

  #Adding to list
  top10_topic_1_total.append(top10_topic_1_sum)

  topic_1_t10_resnet = []
  for i in best_topics_resnet:
    topic_1_t10_resnet.append(topic_1_test[i])

  all_topic_1_sum = np.sum(topic_1_test)
  top10_topic_1_sum_resnet = np.sum(topic_1_t10_resnet)
  # print('Top 10 Sum: ',top10_topic_1_sum_resnet)
  # print('Entire Topic 1 topic sum: ',all_topic_1_sum)

  all_topic_1_sum_list.append(all_topic_1_sum)

  top10_topic_1_total_from_resnet.append(top10_topic_1_sum_resnet)

  #Topic 2
  topic_2 = get_t2_output(x)
  topic_2_test = np.squeeze(topic_2)
  best_topics_topic_2 = np.argsort(topic_2_test)[-n:]
  best_topics_topic2_list.append(best_topics_topic_2)

  topic2_t10_values = []
  for i in best_topics_topic_2:
    topic2_t10_values.append(topic_2_test[i])

  all_topic_2_sum = np.sum(topic_2_test)
  top10_topic_2_sum = np.sum(topic2_t10_values)
  # print('Top 10 Sum from Topic 2: ',top10_topic_2_sum)
  # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

  top10_topic_2_total.append(top10_topic_2_sum)

  topic_2_t10_resnet = []
  for i in best_topics_resnet:
    topic_2_t10_resnet.append(topic_2_test[i])

  all_topic_2_sum = np.sum(topic_2_test)
  top10_topic_2_sum_resnet = np.sum(topic_2_t10_resnet)
  # print('Sum of Top 10 indexes from Resnet: ',top10_topic_2_sum_resnet)
  # print('Entire Topic 2 topic sum: ',all_topic_2_sum)

  all_topic_2_sum_list.append(all_topic_2_sum)

  top10_topic_2_total_from_resnet.append(top10_topic_2_sum_resnet)

  #Topic 3
  topic_3 = get_t3_output(x)[0]
  topic_3_test = np.squeeze(topic_3)
  best_topics_topic_3 = np.argsort(topic_3_test)[-n:]
  best_topics_topic3_list.append(best_topics_topic_3)

  topic3_t10_values = []
  for i in best_topics_topic_3:
    topic3_t10_values.append(topic_3_test[i])

  all_topic_3_sum = np.sum(topic_3_test)
  top10_topic_3_sum = np.sum(topic3_t10_values)
  # print('Top 10 Sum: ',top10_topic_3_sum)
  # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

  top10_topic_3_total.append(top10_topic_3_sum)

  topic_3_t10_resnet = []
  for i in best_topics_resnet:
    topic_3_t10_resnet.append(topic_3_test[i])

  all_topic_3_sum = np.sum(topic_3_test)
  top10_topic_3_sum_resnet = np.sum(topic_3_t10_resnet)
  # print('Top 10 Sum: ',top10_topic_3_sum_resnet)
  # print('Entire Topic 3 topic sum: ',all_topic_3_sum)

  all_topic_3_sum_list.append(all_topic_3_sum)


  top10_topic_3_total_from_resnet.append(top10_topic_3_sum_resnet)

  #Topic 4
  topic_4 = get_t4_output(x)[0]
  topic_4_test = np.squeeze(topic_4)
  best_topics_topic_4 = np.argsort(topic_4_test)[-n:]
  best_topics_topic4_list.append(best_topics_topic_4)
  topic4_t10_values = []
  for i in best_topics_topic_1:
    topic4_t10_values.append(topic_4_test[i])

  all_topic_4_sum = np.sum(topic_4_test)
  top10_topic_4_sum = np.sum(topic4_t10_values)
  # print('Top 10 Sum for Topic 4: ',top10_topic_4_sum)
  # print('Entire Topic 4 topic sum: ',all_topic_4_sum)

  top10_topic_4_total.append(top10_topic_4_sum)

  topic_4_t10_resnet = []
  for i in best_topics_resnet:
    topic_4_t10_resnet.append( [i])

  all_topic_4_sum = np.sum(topic_4_test)
  top10_topic_4_sum_resnet = np.sum(topic_4_t10_resnet)
  # print('Top 10 values from Resnet for Topic 4: ',top10_topic_4_sum_resnet)
  # print('Entire Topic 4 topic sum: ',all_topic_4_sum)
  all_topic_4_sum_list.append(all_topic_4_sum)

  top10_topic_4_total_from_resnet.append(top10_topic_4_sum_resnet)
  df_sum_all_topics = pd.DataFrame(list(zip(sum_resnet_total, all_topic_1_sum_list, all_topic_2_sum_list,all_topic_3_sum_list,all_topic_4_sum_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4'])
  df_sum_all_topics['Class'] = class_name
  df_sum_all_topics.to_csv('sum_topics_'+class_name+'_uni.csv')
  df_best_topics = pd.DataFrame(list(zip(best_topic_resnet_list, best_topics_topic1_list, best_topics_topic2_list,best_topics_topic3_list,best_topics_topic4_list)),columns=['Resnet','Topic 1','Topic 2','Topic 3', 'Topic 4'])
  df_best_topics['Class'] = class_name
  df_best_topics.to_csv('top10_topics_'+class_name+'_uni.csv')

  df_resnet_1 = pd.DataFrame(top10_topic_1_total_from_resnet)
  df_resnet_1['Topic'] = 'Topic 1'
  df_resnet_2 = pd.DataFrame(top10_topic_2_total_from_resnet)
  df_resnet_2['Topic'] = 'Topic 2'
  df_resnet_3 = pd.DataFrame(top10_topic_3_total_from_resnet)
  df_resnet_3['Topic'] = 'Topic 3'
  df_resnet_4 = pd.DataFrame(top10_topic_4_total_from_resnet)
  df_resnet_4['Topic'] = 'Topic 4'
  
  df_resnet = pd.concat([df_resnet_1,df_resnet_2,df_resnet_3,df_resnet_4])
  
  df_topic_1 = pd.DataFrame(top10_topic_1_total)
  df_topic_1['Topic'] = 'Topic 1'
  df_topic_2 = pd.DataFrame(top10_topic_2_total)
  df_topic_2['Topic'] = 'Topic 2'
  df_topic_3 = pd.DataFrame(top10_topic_3_total)
  df_topic_3['Topic'] = 'Topic 3'
  df_topic_4 = pd.DataFrame(top10_topic_4_total)
  df_topic_4['Topic'] = 'Topic 4'

  df_topics = pd.concat([df_topic_1,df_topic_2,df_topic_3,df_topic_4])
  df_topics.columns = ['Sum','Topic']
  df_resnet.columns = ['Sum','Topic']


  #Plotting Figures
  ax = sn.boxplot(y="Sum", x="Topic", data=df_resnet,orient='v')
  plt.title('Top 10 Objects from ResNet Softmax for Topics =4 for '+class_name)
  plt.savefig('imp4_top10_resnet_'+class_name+'_uni.png')
  plt.clf()
  ax = sn.boxplot(y="Sum", x="Topic", data=df_topics,orient='v')
  plt.title('Top 10 Objects from Dense Topic Layers for Topics =4 for '+class_name)
  plt.savefig('imp4_top10_topiclayer_'+class_name+'_uni.png')
  plt.clf()

  return df_resnet,df_topics

In [ ]:
top10_topic_1_total_from_resnet,top10_topic_2_total_from_resnet,top10_topic_3_total_from_resnet,top10_topic_4_total_from_resnet,top10_topic_4_total,top10_topic_2_total,top10_topic_3_total,top10_topic_4_total

In [ ]:
list_re = top10_topic_1_total_from_resnet+top10_topic_2_total_from_resnet+top10_topic_3_total_from_resnet+top10_topic_4_total_from_resnet

In [ ]:
im = Image.open(images[0])

In [ ]:
im

In [ ]:
test = '/content/gdrive/MyDrive/data/train/Office/20171202_163137_000.jpg'
class_test = test.split('/')[-2]
class_test 

##Running the function

In [ ]:
df_resnet,df_topics = create_Topic_data(images,10)

In [ ]:
df_best_topics

In [ ]:
df_test = pd.DataFrame(df_resnet.to_numpy().flatten())
df_test['Topic'] = 'Topic'

In [ ]:
df_test['Topic']

In [ ]:
df_resnet

Vertical Box Plots

In [ ]:
ax = sn.boxplot(y="Sum", x="Topic", data=df_resnet,orient='v')
plt.title('Top 10 Predictions from ResNet Softmax for Topics =4')
plt.show()

###Box plots for Top 10 Softmax per topic

In [ ]:

ax_1 = sn.boxplot(x=df_resnet["Topic 1"])
plt.title('Distribution of Weights for top 10 Predictions from Softmax for Topic 1')
plt.show()

In [ ]:

ax_2 = sn.boxplot(x=df_resnet["Topic 2"])
plt.title('Distribution of Weights for top 10 Predictions from Softmax for Topic 2')
plt.show()

In [ ]:

ax_3 = sn.boxplot(x=df_resnet["Topic 3"])
plt.title('Distribution of Weights for top 10 Predictions from Softmax for Topic 3')
plt.show()

In [ ]:

ax_4 = sn.boxplot(x=df_resnet["Topic 4"])
plt.title('Distribution of Weights for top 10 Predictions from Softmax for Topic 4')
plt.show()

###Box Plots for Top 10 values for each individual Topic 

In [ ]:
ax_1 = sn.boxplot(x=df_topics["Topic 1"])
plt.title('Distribution of Weights for top 10 Predictions from Topic 1 Dense Layer')
plt.show()


In [ ]:
ax_2 = sn.boxplot(x=df_topics["Topic 2"])
plt.title('Distribution of Weights for top 10 Predictions from Topic 2 Dense Layer')
plt.show()


In [ ]:
ax_3 = sn.boxplot(x=df_topics["Topic 3"])
plt.title('Distribution of Weights for top 10 Predictions from Topic 3 Dense Layer')
plt.show()


In [ ]:
ax_4 = sn.boxplot(x=df_topics["Topic 4"])
plt.title('Distribution of Weights for top 10 Predictions from Topic 4 Dense Layer')
plt.show()

In [ ]:

df_resnet = pd.DataFrame(list(zip(topic1_resnet_list, topic2_resnet_list,topic3_resnet_list,topic4_resnet_list)),
               columns =['Topic 1', 'Topic 2','Topic 3','Topic 4'])

In [ ]:
df_resnet

In [ ]:
topic_1_list = [0.06528306, 0.060476344, 0.061662868, 0.058149464, 0.06436589]
topic_2_list = [0.19646133, 0.20384228, 0.20229053, 0.19908997, 0.18312304]
topic_3_list =  [0.20839123, 0.21751055, 0.21491227, 0.21268351, 0.18942289]
topic_4_list = [0.06528306, 0.060476344, 0.061662868, 0.058149464, 0.06436589]

In [ ]:
df_topics = pd.DataFrame(list(zip(topic_1_list, topic_2_list,topic_3_list,topic_4_list)),
               columns =['Topic 1', 'Topic 2','Topic 3','Topic 4'])

In [ ]:
df_topics

###Testing

In [ ]:
data_segs

In [ ]:
data_segs = pd.read_csv('testing_new.csv')
#print(data_segs.head())
#for i in range(len(data_segs)):
elements = data_segs.iloc[0,1:11]
#print(type(elements))
#a = int(elements['0'])
#b = int(elements['1'])
#print(a,b)
#check = np.array([int(elements['0']),int(elements['1']),int(elements['2']),int(elements['3']),int(elements['4']),int(elements['5']),int(elements['6']),int(elements['7']),int(elements['8']),int(elements['9'])])
#print(check)
print(elements)


In [ ]:
water = np.load('/content/Water_train_arrays.npy',allow_pickle=True)

In [ ]:
water1 = water[0]
water2 = water[1]
water3 = water[2]

In [ ]:
image1 = tf.keras.preprocessing.image.array_to_img(water1)
image2 = tf.keras.preprocessing.image.array_to_img(water2)
image3 = tf.keras.preprocessing.image.array_to_img(water3)

In [ ]:
plt.imshow(image3)

In [ ]:
image.

In [ ]:
predictions = np.load('/content/topic_weight_k4_imp_4_4_predictions_tens.npy',allow_pickle=True)

In [ ]:
y_pred = np.argmax(predictions,axis =1)
len(y_pred)

In [ ]:
class_names = np.array(sorted([dir1 for dir1 in os.listdir('/content/gdrive/MyDrive/data/train')]))
label_dict = dict(list(enumerate(class_names)))
labels = label_dict.keys()

In [ ]:
label_dict

In [ ]:
label_dict = {'Balcony': 0,
 'Bathroom': 1,
 'Beach': 2,
 'Bedroom': 3,
 'Buildings': 4,
 'Education,science': 5,
 'Forest, field, jungle': 6,
 'Garden': 7,
 'Hospital': 8,
 'Kitchen': 9,
 'Living room': 10,
 'Mountains, hills, desert, sky': 11,
 'Museum': 12,
 'Noise': 13,
 'Office': 14,
 'Other rooms': 15,
 'Others': 16,
 'Pathways': 17,
 'Recreation': 18,
 'Restaurant,Bar': 19,
 'Shop': 20,
 'Sport fields': 21,
 'Transportation': 22,
 'Water': 23}
labels = label_dict.keys()

In [ ]:
labels

In [ ]:
test_labels = testing['Label']
test_labels = 

In [ ]:
test_labels = test_labels.map(label_dict)
test_labels = test_labels.to_list()
y_true = test_labels

In [ ]:
labels

In [ ]:
print(classification_report(y_true, y_pred,target_names=labels))

In [ ]:
df_resnet

In [ ]:
class_names = np.array(sorted([dir1 for dir1 in os.listdir('/content/gdrive/MyDrive/data/train')]))
cm = confusion_matrix(y_true,y_pred,labels =  list(range(0,24)) )

df_cm = pd.DataFrame(cm,index = class_names,columns = class_names )
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=False, annot_kws={"size": 20},cmap="BuPu") # font size
plt.title('Confusion matrix for Topic Model Implementation 4')
plt.savefig('imp4_cm.png',bbox_inches="tight")

In [ ]:
class_report = classification_report( y_true, y_pred,target_names=labels,output_dict=True)
df_cr = pd.DataFrame(class_report).transpose()
df_cr.to_csv('cr_imp4.csv')

In [ ]:
tips = sn.load_dataset("tips")

In [ ]:
tips

#Creating eval dataset

In [ ]:
val_data = pd.read_csv('/content/gdrive/MyDrive/data/val.csv')

In [ ]:
def df_sampler(df_file):
  df = pd.read_csv(df_file)
  class_names_meta = np.unique(df['Label'])
  sorted(class_names_meta)
  df_list_names = []
  df_list = []
  for class_name in class_names_meta:
    class_name_rep = class_name.replace(" ","_")
    class_name_rep = class_name_rep.replace("/","_")
    class_name_rep = class_name_rep.replace(" ","")
    class_name_rep = class_name_rep.replace(",_","_")
    class_name_rep = class_name_rep.replace(",","_")
    #print(class_name_rep)
    h = "df_" + str(class_name_rep)
    vars()[h] = df[df['Label'] == class_name]
    #vars()[h] = vars()[h].sample(amount)
    print("Created: ",h,' ',len(vars()[h]))
    df_list.append(vars()[h])
    df_list_names.append(h)
  #final = pd.concat(df_list)
  return df_list,df_list_names

Created:  df_Balcony   45
Created:  df_Bathroom   127
Created:  df_Beach   191
Created:  df_Bedroom   403
Created:  df_Buildings   48
Created:  df_Education_science   1839
Created:  df_Forest_field_jungle   194
Created:  df_Garden   130
Created:  df_Hospital   14
Created:  df_Kitchen   448
Created:  df_Living_room   939
Created:  df_Mountains_hills_desert_sky   195
Created:  df_Museum   25
Created:  df_Noise   1107
Created:  df_Office   9013
Created:  df_Other_rooms   326
Created:  df_Others   72
Created:  df_Pathways   2975
Created:  df_Recreation   202
Created:  df_Restaurant_Bar   2871
Created:  df_Shop   380
Created:  df_Sport_fields   135
Created:  df_Transportation   2823
Created:  df_Water   2

In [ ]:
Created:  df_Balcony   45
Created:  df_Bathroom   127
Created:  df_Beach   191
Created:  df_Bedroom   403
Created:  df_Buildings   48
Created:  df_Education_science   1839
Created:  df_Forest_field_jungle   194
Created:  df_Garden   130
Created:  df_Hospital   14
Created:  df_Kitchen   448
Created:  df_Living_room   939
Created:  df_Mountains_hills_desert_sky   195
Created:  df_Museum   25
Created:  df_Noise   1107
Created:  df_Office   9013
Created:  df_Other_rooms   326
Created:  df_Others   72
Created:  df_Pathways   2975
Created:  df_Recreation   202
Created:  df_Restaurant_Bar   2871
Created:  df_Shop   380
Created:  df_Sport_fields   135
Created:  df_Transportation   2823
Created:  df_Water   2

In [ ]:
df_eval_list = [df_Balcony_eval,df_Bathroom_eval,df_Beach_eval,df_Bedroom_eval,df_Buildings_eval,df_Education_science_eval,df_Forest_field_jungle_eval,df_Garden_eval,df_Hospital_eval,df_Kitchen_eval,df_Living_room_eval,df_Mountains_hills_desert_sky_eval,df_Museum_eval,df_Noise_eval,df_Office_eval,df_Other_rooms_eval,df_Others_eval,df_Pathways_eval,df_Recreation_eval,df_Restaurant_Bar_eval,df_Shop_eval,df_Sport_fields_eval,df_Transportation_eval,df_Water_eval]

In [ ]:
df_val_list = [df_Balcony_val,df_Bathroom_val,df_Beach_val,df_Bedroom_val,df_Buildings_val,df_Education_science_val,df_Forest_field_jungle_val,df_Garden_val,df_Hospital_val,df_Kitchen_val,df_Living_room_val,df_Mountains_hills_desert_sky_val,df_Museum_val,df_Noise_val,df_Office_val,df_Other_rooms_val,df_Others_val,df_Pathways_val,df_Recreation_val,df_Restaurant_Bar_val,df_Shop_val,df_Sport_fields_val,df_Transportation_val,df_Water_val]

In [ ]:
len(final_eval_list),len(final_val_list)

In [ ]:
final_val_list = pd.concat(df_val_list)
final_val_list.to_csv('val_updated.csv')
files.download('val_updated.csv')

In [ ]:
final_eval_list = pd.concat(df_eval_list)
final_eval_list.to_csv('eval.csv')
files.download('eval.csv')

In [ ]:

df = pd.read_csv('/content/gdrive/MyDrive/data/val.csv')
class_names_meta = np.unique(df['Label'])
sorted(class_names_meta)
df_list_names = []
df_list = []
for class_name in class_names_meta:
  class_name_rep = class_name.replace(" ","_")
  class_name_rep = class_name_rep.replace("/","_")
  class_name_rep = class_name_rep.replace(" ","")
  class_name_rep = class_name_rep.replace(",_","_")
  class_name_rep = class_name_rep.replace(",","_")
  #print(class_name_rep)
  h = "df_" + str(class_name_rep)
  vars()[h] = df[df['Label'] == class_name]
  #vars()[h] = vars()[h].sample(amount)
  print("Created: ",h,' ',len(vars()[h]))
  #df_list.append(vars()[h])
  #df_list_names.append(h)
#final = pd.concat(df_list)


In [ ]:
df_Transportation_val,df_Transportation_eval = train_test_split(df_Transportation,test_size = 0.3,shuffle='True',random_state=42)

In [ ]:
df_Water_val,df_Water_eval = train_test_split(df_Water,test_size = 0.3,shuffle='True',random_state=42)

In [ ]:
len(df_Kitchen_val),len(df_Kitchen_eval)

In [ ]:
df_sampler('/content/gdrive/MyDrive/data/val.csv')

In [ ]:
df_balcony

#Eliminating Dups in the dataset

##Inputing the data frames

In [ ]:
eval_test = pd.read_csv('/content/gdrive/MyDrive/Final DataFrames for testing/eval (1).csv')
test_test = pd.read_csv('/content/gdrive/MyDrive/Final DataFrames for testing/test (2).csv')
train_test = pd.read_csv('/content/gdrive/MyDrive/Final DataFrames for testing/train (6).csv')
val_test = pd.read_csv('/content/gdrive/MyDrive/Final DataFrames for testing/val_updated (1).csv')

In [ ]:
def create_photo_col(df):
  df['Photo'] = df['Image'].apply(lambda x:x.split('/')[-1])
  return df

In [ ]:
eval_new = create_photo_col(eval_test)
test_new = create_photo_col(test_test)
train_new = create_photo_col(train_test)
val_new = create_photo_col(val_test)

In [ ]:
train_new = train_new[['Image','Label','Photo']]
test_new = test_new[['Image','Label','Photo']]
val_new = val_new[['Image','Label','Photo']]
eval_new = eval_new[['Image','Label','Photo']]
len(train_new),len(test_new),len(val_new),len(eval_new)

In [ ]:
np.unique(train_new['Label'])

In [ ]:
train_new

In [ ]:
non_train = pd.concat([eval_new,test_new,val_new])

In [ ]:
non_test = pd.concat([eval_new,val_new])

In [ ]:
len(non_test),len(test_new)

In [ ]:
len(non_train)

In [ ]:
non_train.to_csv('nontrain.csv')
files.download('nontrain.csv')

###Common elements between eval and train

In [ ]:
mergedStuff = pd.merge(eval_new, train_updated, on=['Photo'], how='inner')
len(np.unique(mergedStuff['Label_x'])),len(eval_new),len(train_updated),len(mergedStuff)

In [ ]:
mergedStuff

###Common elements between val and train

In [ ]:
mergedStuff_2 = pd.merge(final_user6, train_updated, on=['Photo'], how='inner')
len(np.unique(mergedStuff_2['Label_x']))

len(mergedStuff_2),len(val_new),len(np.unique(mergedStuff_2['Label_x'])),len(train_updated)

In [ ]:
common_test_train_updated = mergedStuff_2['Photo']

In [ ]:
mergedStuff_2

###Common elements between test and train

In [ ]:
mergedStuff_3 = pd.merge(test_new, train_updated, on=['Photo'], how='inner')
#len(np.unique(mergedStuff_3['Label_x']))
len(mergedStuff_3),len(test_new),len(train_updated),len(np.unique(mergedStuff_3['Label_x']))

In [ ]:
mergedStuff_3

In [ ]:
train_updated[train_updated['Label'] == 'Water']

###Common elements between eval and val

In [ ]:
mergedStuff_4 = pd.merge(val_new, eval_new, on=['Photo'], how='inner')
# len(np.unique(mergedStuff_4['Label_x']))

len(mergedStuff_4),len(val_new),len(np.unique(mergedStuff_4['Label_x'])),len(eval_new)

In [ ]:
mergedStuff_4

###Common elements between eval and test

In [ ]:
mergedStuff_5 = pd.merge(test_new, eval_new, on=['Photo'], how='inner')
# len(np.unique(mergedStuff_4['Label_x']))

len(mergedStuff_5),len(test_new),len(np.unique(mergedStuff_5['Label_x'])),len(eval_new)

In [ ]:
mergedStuff_5

###Common elements between val and test

In [ ]:
mergedStuff_6 = pd.merge(test_new, val_new, on=['Photo'], how='inner')
# len(np.unique(mergedStuff_4['Label_x']))

len(mergedStuff_6),len(test_new),len(np.unique(mergedStuff_6['Label_x'])),len(val_new)

In [ ]:
mergedStuff_6

###Common elements between train and non-train

In [ ]:
mergedStuff_7 = pd.merge(final_user6, val_new, on=['Photo'], how='inner')
# len(np.unique(mergedStuff_4['Label_x']))

len(mergedStuff_7),len(train_new),len(np.unique(mergedStuff_7['Label_x'])),len(np.unique(mergedStuff_7['Label_y'])),len(non_train)

In [ ]:
mergedStuff_7

In [ ]:
common_train_nontrain = list(mergedStuff_7['Photo'])

In [ ]:
len(common_train_nontrain)

In [ ]:
train_updated_1 = train_updated[~train_updated['Photo'].isin(common_test_train_updated)]

In [ ]:
len(np.unique(train_updated_1['Image'])),len(np.unique(train_updated_1['Photo']))

In [ ]:
test_training = train_updated[train_updated_1.isin(c)]

In [ ]:
train_updated_1

In [ ]:
train_updated_1[train_updated_1['Photo']=='20160608_152708_000.jpg']

In [ ]:
train_updated_1[train_updated_1.duplicated(['Photo'])]

In [ ]:
t = train_updated_1['Photo']
c = t[t.duplicated()]

In [ ]:
tt = train_updated_1['Image'].apply(lambda x:x.split('/')[-1])
a = np.unique(tt)
b= np.unique(train_updated_1['Photo'])
diff = list(set(a)-set(b))

In [ ]:
len(a),len(b)

In [ ]:
len(np.unique(train_updated_1['Label']))

In [ ]:
train_updated_1.to_csv('train_updated.csv')
files.download('train_updated.csv')

###Common elements between test and non-test

In [ ]:
test_new[test_new['Label']=='Water']

In [ ]:
non_test[non_test['Label']=='Water']

In [ ]:
train_updated[train_updated['Label']=='Water']

In [ ]:
mergedStuff_8 = pd.merge(test_new, non_test, on=['Photo'], how='inner')
# len(np.unique(mergedStuff_4['Label_x']))

len(mergedStuff_8),len(test_new),len(np.unique(mergedStuff_8['Label_x'])),len(np.unique(mergedStuff_8['Label_y'])),len(non_test)

In [ ]:
mergedStuff_8

In [ ]:
common_train_nontrain = list(mergedStuff_7['Photo'])

In [ ]:
len(common_train_nontrain)

In [ ]:
train_updated = train_new[~train_new['Photo'].isin(common_train_nontrain)]

In [ ]:
len(np.unique(train_updated['Image'])),len(np.unique(train_updated['Photo']))

In [ ]:
train_updated

In [ ]:
train_updated.to_csv('train_updated.csv')
files.download('train_updated.csv')

###Creating Test set from User 06

In [ ]:
test_user_6 = pd.read_csv('test_test.csv')


In [ ]:
# test_user_6 = test_user_6.loc[:]
test_user_6

In [ ]:
test_user_6.columns = ['ID','Path']

In [ ]:
test_user_6['Photo'] = test_user_6['Path'].apply(lambda x: x.split('/')[-1])

In [ ]:
user6_labels = pd.concat(pd.read_excel('/content/user_06.xlsx',sheet_name=None))

In [ ]:
user6_lab = pd.DataFrame(user6_labels)

In [ ]:
user6_lab.to_csv('user6.csv')

In [ ]:
user6_lab = pd.DataFrame(user6_lab)

In [ ]:
user6_csv = pd.read_csv('/content/user66.csv')

In [ ]:
user6_csv = user6_csv[['Photo','Class']]

In [ ]:
user6_csv

In [ ]:
final_user6 = pd.merge(user6_csv,test_user_6,on='Photo',how='inner')

In [ ]:
a=np.unique(final_user6['Class'])
b = np.unique(train_updated['Label'])
list(set(b)-set(a))

In [ ]:
final_user6['Class'] = final_user6['Class'].apply(lambda x:x.replace('/',','))

In [ ]:
final_user6['Image'] = '/data/s4133366/data/test'+os.sep+final_user6['Class']+os.sep+final_user6['Photo']

In [ ]:
final_user6.columns = ['Photo','Label','ID','Path','Image']

In [ ]:
final_user6.to_csv('new_test.csv')

In [ ]:
final_user6.groupby('Label').count()

#Creating collated plots for metrics

In [ ]:
a = np.load('/content/history_topic_weight_k4_places_20_imp2.npy',allow_pickle=True).item()
b = np.load('/content/history_topic_weight_k4_places_20_imp2_1.npy',allow_pickle=True).item()
# c = np.load('/content/history_baseline_places20_latest_2.npy',allow_pickle=True).item()
# d = np.load('/content/history_topic_weight_k4_places_365_imp9_final_3.npy',allow_pickle=True).item()
# e = np.load('/content/history_topic_weight_k4_places_365_imp9_final_4.npy',allow_pickle=True).item()

In [ ]:
len(df_a),len(df_b)#,len(df_c)

In [ ]:
df_a = pd.DataFrame(a)
df_b = pd.DataFrame(b)
#df_c = pd.DataFrame(c)
# df_d = pd.DataFrame(d)
# df_e = pd.DataFrame(e)

In [ ]:
df_b.index = df_b.index + 100
# df_c.index = df_c.index + 100
# df_d.index = df_d.index + 130
# df_e.index = df_e.index+170

In [ ]:
df_c.head(5)

In [ ]:
test = np.load('/content/results_topic_weight_k4_places_20_imp2_1.npy',allow_pickle=True).item()
test

In [ ]:
len(df_a),len(df_b),len(df_c)#,len(df_d),len(df_e)#170

In [ ]:
df_d.index = df_d.index+120

In [ ]:
df_final = pd.concat([df_a,df_b])

In [ ]:
len(df_final)

In [ ]:
test = pd.DataFrame(test,index=df_a.index)
test = test[:1]
test.columns = ['loss','categorical_accuracy','precision','recall']

In [ ]:
test

In [ ]:
df_final = df_final.append(test,ignore_index=True)

In [ ]:
df_final =df_final.drop(200)

In [ ]:
df_final

In [ ]:
b = a.tail(1)
b

In [ ]:
read_train = df_final[:200]

In [ ]:
read_train

In [ ]:
read_test = df_final.iloc[200,:]
read_test

In [ ]:
df_final =pd.concat([df_a,df_b,df_c,df_d],)

In [ ]:
read_test

In [ ]:
read_test = b

In [ ]:
model_string = 'Topic Model Implementation 2 using Places20'
history_path = 'top_model_2_p20_'
num = 200
plt.plot(read_train['categorical_accuracy']*100)
#plt.plot(read_train['val_categorical_accuracy']*100)
text = str(round(read_test['categorical_accuracy']*100))+'%'
plt.scatter(num,read_test['categorical_accuracy']*100, color='g')
plt.text(s = text,x = len(read_train+1),y = read_test['categorical_accuracy']*100, fontsize=9) #

plt.title('Model accuracy for '+str(model_string))
plt.ylabel('Categorical Accuracy')
plt.xlabel('Epoch')
plt.legend(['train','test'], loc='upper left')
plt.savefig('accuracy_'+str(history_path)+'.png')
plt.clf()

# summarize history for loss

plt.plot(read_train['loss'])
#plt.plot(read_train['val_loss'])
text = str(round(read_test['loss']))
plt.scatter(num,read_test['loss'], color='g')
plt.text(s = text,x = 2+len(read_train+1),y = 2+read_test['loss'], fontsize=9) #

plt.title('Model loss for '+str(model_string))
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train','test'], loc='upper left')
#plt.show()
plt.savefig('loss_'+str(history_path)+'.png')
plt.clf()

#summarize history for Precision 
plt.plot(read_train['precision']*100)
#plt.plot(read_train['val_precision']*100)
text = str(round(read_test['precision']*100))+'%'
plt.scatter(num,read_test['precision']*100, color='g')
plt.text(s = text,x = 2+len(read_train+1),y = 2+read_test['precision']*100, fontsize=9) #
plt.title('Model precision for '+str(model_string))
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['train','test'], loc='upper left')
plt.savefig('precision_'+str(history_path)+'.png')
plt.clf()

#summarize history for Recall 



plt.plot(read_train['recall']*100)
#plt.plot(read_train['val_recall']*100)
text = str(round(read_test['recall']*100))+'%'
plt.scatter(num,read_test['recall']*100, color='g')
plt.text(s = text,x = 2+len(read_train+1),y = 2+read_test['recall']*100, fontsize=9) #
plt.title('Model recall for '+str(model_string))
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('recall_'+str(history_path)+'.png')
plt.clf()


plt.plot(read_train['lr'])
plt.title('Model Learning rate for '+str(model_string))
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')
#plt.legend(['train', 'val'], loc='upper left')
plt.savefig('lr_'+str(history_path)+'.png')
plt.clf()


In [ ]:
read_test['recall']

In [ ]:
plt.plot(read_train['recall']*100)
plt.plot(read_train['val_recall']*100)
text = str(round(read_test['recall']*100))+'%'
plt.scatter(200,read_test['recall']*100, color='g')
plt.text(s = text,x = 2+len(read_train+2),y = 2+read_test['recall']*100, fontsize=9) #
plt.title('Model recall for '+str('model_string'))
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['train', 'val','test'], loc='upper left')
# plt.savefig('recall_'+str(history_path)+'.png')
plt.show()


In [ ]:
read_train['lr'].tail(1)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = tf.keras.layers.AveragePooling2D(pool_size=(7,7))(x)
x = tf.keras.layers.Flatten(name ="flatten")(x)
x = tf.keras.layers.Dense(1024,activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
preds = tf.keras.layers.Dense(365,activation="softmax")(x)

# #Activating the model

model = tf.keras.Model(inputs = base_model.input,outputs = preds)




for layer in base_model.layers:
  layer.trainable = False


# opti = tf.keras.optimizers.Adadelta(learning_rate=learning_rate, rho=0.95, epsilon=1e-07, name="Adadelta")
# model.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],loss='categorical_crossentropy')

model.summary()


In [ ]:
test = pd.read_csv('/content/test_updated (1).csv')

In [ ]:
test

In [ ]:
test.columns = ['ID','Image']

In [ ]:
test['Label'] = test['Image'].apply(lambda x:x.split('/')[-2])

In [ ]:
test.to_csv('test_updated.csv')

In [ ]:
labels = pd.read_csv('/content/short_p365_test (1).csv')

In [ ]:
labels = labels['Label']

In [ ]:
labels_list = np.unique(labels)

In [ ]:
labels_dict = {}

In [ ]:
range(365)

In [ ]:
labels_dict = dict(list(enumerate(labels_list)))

In [ ]:
inv_map = {v: k for k, v in labels_dict.items()}

In [ ]:
inv_map

#Plotting legible Confusion Matrix

In [ ]:
df_cm = pd.read_csv('/content/imp2_k9 - Sheet1.csv')

In [ ]:
df_cm = df_cm.iloc[:,1:]

In [ ]:
labels = dict(enumerate(df_cm.columns))
labels

In [ ]:
df_cm

In [ ]:
model_string = 'Topic Model Implementation 1'
db_name = 'EgoPlaces with 9 topics'
plt.figure(figsize=(20,20))
ax= plt.subplot()
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 14},cmap="BuPu") # font size
plt.title('Confusion matrix for '+model_string+' using '+db_name)
plt.yticks(rotation = 0) 
ax.xaxis.set_ticklabels(df_cm.columns)
ax.yaxis.set_ticklabels(df_cm.columns)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#print('Saving plot in '+cm_path+' ...')
plt.savefig('/content/imp1_pred_k9_partial.png',bbox_inches="tight")

print('Model predicted ....')

#Creating big test data-set

In [ ]:
a = pd.read_csv('/content/eval (2).csv')
b = pd.read_csv('/content/test_updated (3).csv')
c = pd.read_csv('/content/val_updated (2).csv')
df_final = pd.concat([a,b,c])
df_final = df_final[['Image','Label']]


In [ ]:
df_final = df_final.drop_duplicates()

In [ ]:
len(np.unique(df_final['Label']))

In [ ]:
len(df_final)

In [ ]:
df_final.to_csv('test_final.csv')
files.download('test_final.csv')

#Mapping the objects to imagenet values

In [ ]:
imagenet = {0: 'tench, Tinca tinca',
 1: 'goldfish, Carassius auratus',
 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 3: 'tiger shark, Galeocerdo cuvieri',
 4: 'hammerhead, hammerhead shark',
 5: 'electric ray, crampfish, numbfish, torpedo',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich, Struthio camelus',
 10: 'brambling, Fringilla montifringilla',
 11: 'goldfinch, Carduelis carduelis',
 12: 'house finch, linnet, Carpodacus mexicanus',
 13: 'junco, snowbird',
 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',
 15: 'robin, American robin, Turdus migratorius',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel, dipper',
 21: 'kite',
 22: 'bald eagle, American eagle, Haliaeetus leucocephalus',
 23: 'vulture',
 24: 'great grey owl, great gray owl, Strix nebulosa',
 25: 'European fire salamander, Salamandra salamandra',
 26: 'common newt, Triturus vulgaris',
 27: 'eft',
 28: 'spotted salamander, Ambystoma maculatum',
 29: 'axolotl, mud puppy, Ambystoma mexicanum',
 30: 'bullfrog, Rana catesbeiana',
 31: 'tree frog, tree-frog',
 32: 'tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui',
 33: 'loggerhead, loggerhead turtle, Caretta caretta',
 34: 'leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea',
 35: 'mud turtle',
 36: 'terrapin',
 37: 'box turtle, box tortoise',
 38: 'banded gecko',
 39: 'common iguana, iguana, Iguana iguana',
 40: 'American chameleon, anole, Anolis carolinensis',
 41: 'whiptail, whiptail lizard',
 42: 'agama',
 43: 'frilled lizard, Chlamydosaurus kingi',
 44: 'alligator lizard',
 45: 'Gila monster, Heloderma suspectum',
 46: 'green lizard, Lacerta viridis',
 47: 'African chameleon, Chamaeleo chamaeleon',
 48: 'Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis',
 49: 'African crocodile, Nile crocodile, Crocodylus niloticus',
 50: 'American alligator, Alligator mississipiensis',
 51: 'triceratops',
 52: 'thunder snake, worm snake, Carphophis amoenus',
 53: 'ringneck snake, ring-necked snake, ring snake',
 54: 'hognose snake, puff adder, sand viper',
 55: 'green snake, grass snake',
 56: 'king snake, kingsnake',
 57: 'garter snake, grass snake',
 58: 'water snake',
 59: 'vine snake',
 60: 'night snake, Hypsiglena torquata',
 61: 'boa constrictor, Constrictor constrictor',
 62: 'rock python, rock snake, Python sebae',
 63: 'Indian cobra, Naja naja',
 64: 'green mamba',
 65: 'sea snake',
 66: 'horned viper, cerastes, sand viper, horned asp, Cerastes cornutus',
 67: 'diamondback, diamondback rattlesnake, Crotalus adamanteus',
 68: 'sidewinder, horned rattlesnake, Crotalus cerastes',
 69: 'trilobite',
 70: 'harvestman, daddy longlegs, Phalangium opilio',
 71: 'scorpion',
 72: 'black and gold garden spider, Argiope aurantia',
 73: 'barn spider, Araneus cavaticus',
 74: 'garden spider, Aranea diademata',
 75: 'black widow, Latrodectus mactans',
 76: 'tarantula',
 77: 'wolf spider, hunting spider',
 78: 'tick',
 79: 'centipede',
 80: 'black grouse',
 81: 'ptarmigan',
 82: 'ruffed grouse, partridge, Bonasa umbellus',
 83: 'prairie chicken, prairie grouse, prairie fowl',
 84: 'peacock',
 85: 'quail',
 86: 'partridge',
 87: 'African grey, African gray, Psittacus erithacus',
 88: 'macaw',
 89: 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita',
 90: 'lorikeet',
 91: 'coucal',
 92: 'bee eater',
 93: 'hornbill',
 94: 'hummingbird',
 95: 'jacamar',
 96: 'toucan',
 97: 'drake',
 98: 'red-breasted merganser, Mergus serrator',
 99: 'goose',
 100: 'black swan, Cygnus atratus',
 101: 'tusker',
 102: 'echidna, spiny anteater, anteater',
 103: 'platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus',
 104: 'wallaby, brush kangaroo',
 105: 'koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus',
 106: 'wombat',
 107: 'jellyfish',
 108: 'sea anemone, anemone',
 109: 'brain coral',
 110: 'flatworm, platyhelminth',
 111: 'nematode, nematode worm, roundworm',
 112: 'conch',
 113: 'snail',
 114: 'slug',
 115: 'sea slug, nudibranch',
 116: 'chiton, coat-of-mail shell, sea cradle, polyplacophore',
 117: 'chambered nautilus, pearly nautilus, nautilus',
 118: 'Dungeness crab, Cancer magister',
 119: 'rock crab, Cancer irroratus',
 120: 'fiddler crab',
 121: 'king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica',
 122: 'American lobster, Northern lobster, Maine lobster, Homarus americanus',
 123: 'spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish',
 124: 'crayfish, crawfish, crawdad, crawdaddy',
 125: 'hermit crab',
 126: 'isopod',
 127: 'white stork, Ciconia ciconia',
 128: 'black stork, Ciconia nigra',
 129: 'spoonbill',
 130: 'flamingo',
 131: 'little blue heron, Egretta caerulea',
 132: 'American egret, great white heron, Egretta albus',
 133: 'bittern',
 134: 'crane',
 135: 'limpkin, Aramus pictus',
 136: 'European gallinule, Porphyrio porphyrio',
 137: 'American coot, marsh hen, mud hen, water hen, Fulica americana',
 138: 'bustard',
 139: 'ruddy turnstone, Arenaria interpres',
 140: 'red-backed sandpiper, dunlin, Erolia alpina',
 141: 'redshank, Tringa totanus',
 142: 'dowitcher',
 143: 'oystercatcher, oyster catcher',
 144: 'pelican',
 145: 'king penguin, Aptenodytes patagonica',
 146: 'albatross, mollymawk',
 147: 'grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus',
 148: 'killer whale, killer, orca, grampus, sea wolf, Orcinus orca',
 149: 'dugong, Dugong dugon',
 150: 'sea lion',
 151: 'Chihuahua',
 152: 'Japanese spaniel',
 153: 'Maltese dog, Maltese terrier, Maltese',
 154: 'Pekinese, Pekingese, Peke',
 155: 'Shih-Tzu',
 156: 'Blenheim spaniel',
 157: 'papillon',
 158: 'toy terrier',
 159: 'Rhodesian ridgeback',
 160: 'Afghan hound, Afghan',
 161: 'basset, basset hound',
 162: 'beagle',
 163: 'bloodhound, sleuthhound',
 164: 'bluetick',
 165: 'black-and-tan coonhound',
 166: 'Walker hound, Walker foxhound',
 167: 'English foxhound',
 168: 'redbone',
 169: 'borzoi, Russian wolfhound',
 170: 'Irish wolfhound',
 171: 'Italian greyhound',
 172: 'whippet',
 173: 'Ibizan hound, Ibizan Podenco',
 174: 'Norwegian elkhound, elkhound',
 175: 'otterhound, otter hound',
 176: 'Saluki, gazelle hound',
 177: 'Scottish deerhound, deerhound',
 178: 'Weimaraner',
 179: 'Staffordshire bullterrier, Staffordshire bull terrier',
 180: 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier',
 181: 'Bedlington terrier',
 182: 'Border terrier',
 183: 'Kerry blue terrier',
 184: 'Irish terrier',
 185: 'Norfolk terrier',
 186: 'Norwich terrier',
 187: 'Yorkshire terrier',
 188: 'wire-haired fox terrier',
 189: 'Lakeland terrier',
 190: 'Sealyham terrier, Sealyham',
 191: 'Airedale, Airedale terrier',
 192: 'cairn, cairn terrier',
 193: 'Australian terrier',
 194: 'Dandie Dinmont, Dandie Dinmont terrier',
 195: 'Boston bull, Boston terrier',
 196: 'miniature schnauzer',
 197: 'giant schnauzer',
 198: 'standard schnauzer',
 199: 'Scotch terrier, Scottish terrier, Scottie',
 200: 'Tibetan terrier, chrysanthemum dog',
 201: 'silky terrier, Sydney silky',
 202: 'soft-coated wheaten terrier',
 203: 'West Highland white terrier',
 204: 'Lhasa, Lhasa apso',
 205: 'flat-coated retriever',
 206: 'curly-coated retriever',
 207: 'golden retriever',
 208: 'Labrador retriever',
 209: 'Chesapeake Bay retriever',
 210: 'German short-haired pointer',
 211: 'vizsla, Hungarian pointer',
 212: 'English setter',
 213: 'Irish setter, red setter',
 214: 'Gordon setter',
 215: 'Brittany spaniel',
 216: 'clumber, clumber spaniel',
 217: 'English springer, English springer spaniel',
 218: 'Welsh springer spaniel',
 219: 'cocker spaniel, English cocker spaniel, cocker',
 220: 'Sussex spaniel',
 221: 'Irish water spaniel',
 222: 'kuvasz',
 223: 'schipperke',
 224: 'groenendael',
 225: 'malinois',
 226: 'briard',
 227: 'kelpie',
 228: 'komondor',
 229: 'Old English sheepdog, bobtail',
 230: 'Shetland sheepdog, Shetland sheep dog, Shetland',
 231: 'collie',
 232: 'Border collie',
 233: 'Bouvier des Flandres, Bouviers des Flandres',
 234: 'Rottweiler',
 235: 'German shepherd, German shepherd dog, German police dog, alsatian',
 236: 'Doberman, Doberman pinscher',
 237: 'miniature pinscher',
 238: 'Greater Swiss Mountain dog',
 239: 'Bernese mountain dog',
 240: 'Appenzeller',
 241: 'EntleBucher',
 242: 'boxer',
 243: 'bull mastiff',
 244: 'Tibetan mastiff',
 245: 'French bulldog',
 246: 'Great Dane',
 247: 'Saint Bernard, St Bernard',
 248: 'Eskimo dog, husky',
 249: 'malamute, malemute, Alaskan malamute',
 250: 'Siberian husky',
 251: 'dalmatian, coach dog, carriage dog',
 252: 'affenpinscher, monkey pinscher, monkey dog',
 253: 'basenji',
 254: 'pug, pug-dog',
 255: 'Leonberg',
 256: 'Newfoundland, Newfoundland dog',
 257: 'Great Pyrenees',
 258: 'Samoyed, Samoyede',
 259: 'Pomeranian',
 260: 'chow, chow chow',
 261: 'keeshond',
 262: 'Brabancon griffon',
 263: 'Pembroke, Pembroke Welsh corgi',
 264: 'Cardigan, Cardigan Welsh corgi',
 265: 'toy poodle',
 266: 'miniature poodle',
 267: 'standard poodle',
 268: 'Mexican hairless',
 269: 'timber wolf, grey wolf, gray wolf, Canis lupus',
 270: 'white wolf, Arctic wolf, Canis lupus tundrarum',
 271: 'red wolf, maned wolf, Canis rufus, Canis niger',
 272: 'coyote, prairie wolf, brush wolf, Canis latrans',
 273: 'dingo, warrigal, warragal, Canis dingo',
 274: 'dhole, Cuon alpinus',
 275: 'African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus',
 276: 'hyena, hyaena',
 277: 'red fox, Vulpes vulpes',
 278: 'kit fox, Vulpes macrotis',
 279: 'Arctic fox, white fox, Alopex lagopus',
 280: 'grey fox, gray fox, Urocyon cinereoargenteus',
 281: 'tabby, tabby cat',
 282: 'tiger cat',
 283: 'Persian cat',
 284: 'Siamese cat, Siamese',
 285: 'Egyptian cat',
 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor',
 287: 'lynx, catamount',
 288: 'leopard, Panthera pardus',
 289: 'snow leopard, ounce, Panthera uncia',
 290: 'jaguar, panther, Panthera onca, Felis onca',
 291: 'lion, king of beasts, Panthera leo',
 292: 'tiger, Panthera tigris',
 293: 'cheetah, chetah, Acinonyx jubatus',
 294: 'brown bear, bruin, Ursus arctos',
 295: 'American black bear, black bear, Ursus americanus, Euarctos americanus',
 296: 'ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus',
 297: 'sloth bear, Melursus ursinus, Ursus ursinus',
 298: 'mongoose',
 299: 'meerkat, mierkat',
 300: 'tiger beetle',
 301: 'ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle',
 302: 'ground beetle, carabid beetle',
 303: 'long-horned beetle, longicorn, longicorn beetle',
 304: 'leaf beetle, chrysomelid',
 305: 'dung beetle',
 306: 'rhinoceros beetle',
 307: 'weevil',
 308: 'fly',
 309: 'bee',
 310: 'ant, emmet, pismire',
 311: 'grasshopper, hopper',
 312: 'cricket',
 313: 'walking stick, walkingstick, stick insect',
 314: 'cockroach, roach',
 315: 'mantis, mantid',
 316: 'cicada, cicala',
 317: 'leafhopper',
 318: 'lacewing, lacewing fly',
 319: "dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk",
 320: 'damselfly',
 321: 'admiral',
 322: 'ringlet, ringlet butterfly',
 323: 'monarch, monarch butterfly, milkweed butterfly, Danaus plexippus',
 324: 'cabbage butterfly',
 325: 'sulphur butterfly, sulfur butterfly',
 326: 'lycaenid, lycaenid butterfly',
 327: 'starfish, sea star',
 328: 'sea urchin',
 329: 'sea cucumber, holothurian',
 330: 'wood rabbit, cottontail, cottontail rabbit',
 331: 'hare',
 332: 'Angora, Angora rabbit',
 333: 'hamster',
 334: 'porcupine, hedgehog',
 335: 'fox squirrel, eastern fox squirrel, Sciurus niger',
 336: 'marmot',
 337: 'beaver',
 338: 'guinea pig, Cavia cobaya',
 339: 'sorrel',
 340: 'zebra',
 341: 'hog, pig, grunter, squealer, Sus scrofa',
 342: 'wild boar, boar, Sus scrofa',
 343: 'warthog',
 344: 'hippopotamus, hippo, river horse, Hippopotamus amphibius',
 345: 'ox',
 346: 'water buffalo, water ox, Asiatic buffalo, Bubalus bubalis',
 347: 'bison',
 348: 'ram, tup',
 349: 'bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis',
 350: 'ibex, Capra ibex',
 351: 'hartebeest',
 352: 'impala, Aepyceros melampus',
 353: 'gazelle',
 354: 'Arabian camel, dromedary, Camelus dromedarius',
 355: 'llama',
 356: 'weasel',
 357: 'mink',
 358: 'polecat, fitch, foulmart, foumart, Mustela putorius',
 359: 'black-footed ferret, ferret, Mustela nigripes',
 360: 'otter',
 361: 'skunk, polecat, wood pussy',
 362: 'badger',
 363: 'armadillo',
 364: 'three-toed sloth, ai, Bradypus tridactylus',
 365: 'orangutan, orang, orangutang, Pongo pygmaeus',
 366: 'gorilla, Gorilla gorilla',
 367: 'chimpanzee, chimp, Pan troglodytes',
 368: 'gibbon, Hylobates lar',
 369: 'siamang, Hylobates syndactylus, Symphalangus syndactylus',
 370: 'guenon, guenon monkey',
 371: 'patas, hussar monkey, Erythrocebus patas',
 372: 'baboon',
 373: 'macaque',
 374: 'langur',
 375: 'colobus, colobus monkey',
 376: 'proboscis monkey, Nasalis larvatus',
 377: 'marmoset',
 378: 'capuchin, ringtail, Cebus capucinus',
 379: 'howler monkey, howler',
 380: 'titi, titi monkey',
 381: 'spider monkey, Ateles geoffroyi',
 382: 'squirrel monkey, Saimiri sciureus',
 383: 'Madagascar cat, ring-tailed lemur, Lemur catta',
 384: 'indri, indris, Indri indri, Indri brevicaudatus',
 385: 'Indian elephant, Elephas maximus',
 386: 'African elephant, Loxodonta africana',
 387: 'lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens',
 388: 'giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca',
 389: 'barracouta, snoek',
 390: 'eel',
 391: 'coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch',
 392: 'rock beauty, Holocanthus tricolor',
 393: 'anemone fish',
 394: 'sturgeon',
 395: 'gar, garfish, garpike, billfish, Lepisosteus osseus',
 396: 'lionfish',
 397: 'puffer, pufferfish, blowfish, globefish',
 398: 'abacus',
 399: 'abaya',
 400: "academic gown, academic robe, judge's robe",
 401: 'accordion, piano accordion, squeeze box',
 402: 'acoustic guitar',
 403: 'aircraft carrier, carrier, flattop, attack aircraft carrier',
 404: 'airliner',
 405: 'airship, dirigible',
 406: 'altar',
 407: 'ambulance',
 408: 'amphibian, amphibious vehicle',
 409: 'analog clock',
 410: 'apiary, bee house',
 411: 'apron',
 412: 'ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin',
 413: 'assault rifle, assault gun',
 414: 'backpack, back pack, knapsack, packsack, rucksack, haversack',
 415: 'bakery, bakeshop, bakehouse',
 416: 'balance beam, beam',
 417: 'balloon',
 418: 'ballpoint, ballpoint pen, ballpen, Biro',
 419: 'Band Aid',
 420: 'banjo',
 421: 'bannister, banister, balustrade, balusters, handrail',
 422: 'barbell',
 423: 'barber chair',
 424: 'barbershop',
 425: 'barn',
 426: 'barometer',
 427: 'barrel, cask',
 428: 'barrow, garden cart, lawn cart, wheelbarrow',
 429: 'baseball',
 430: 'basketball',
 431: 'bassinet',
 432: 'bassoon',
 433: 'bathing cap, swimming cap',
 434: 'bath towel',
 435: 'bathtub, bathing tub, bath, tub',
 436: 'beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon',
 437: 'beacon, lighthouse, beacon light, pharos',
 438: 'beaker',
 439: 'bearskin, busby, shako',
 440: 'beer bottle',
 441: 'beer glass',
 442: 'bell cote, bell cot',
 443: 'bib',
 444: 'bicycle-built-for-two, tandem bicycle, tandem',
 445: 'bikini, two-piece',
 446: 'binder, ring-binder',
 447: 'binoculars, field glasses, opera glasses',
 448: 'birdhouse',
 449: 'boathouse',
 450: 'bobsled, bobsleigh, bob',
 451: 'bolo tie, bolo, bola tie, bola',
 452: 'bonnet, poke bonnet',
 453: 'bookcase',
 454: 'bookshop, bookstore, bookstall',
 455: 'bottlecap',
 456: 'bow',
 457: 'bow tie, bow-tie, bowtie',
 458: 'brass, memorial tablet, plaque',
 459: 'brassiere, bra, bandeau',
 460: 'breakwater, groin, groyne, mole, bulwark, seawall, jetty',
 461: 'breastplate, aegis, egis',
 462: 'broom',
 463: 'bucket, pail',
 464: 'buckle',
 465: 'bulletproof vest',
 466: 'bullet train, bullet',
 467: 'butcher shop, meat market',
 468: 'cab, hack, taxi, taxicab',
 469: 'caldron, cauldron',
 470: 'candle, taper, wax light',
 471: 'cannon',
 472: 'canoe',
 473: 'can opener, tin opener',
 474: 'cardigan',
 475: 'car mirror',
 476: 'carousel, carrousel, merry-go-round, roundabout, whirligig',
 477: "carpenter's kit, tool kit",
 478: 'carton',
 479: 'car wheel',
 480: 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM',
 481: 'cassette',
 482: 'cassette player',
 483: 'castle',
 484: 'catamaran',
 485: 'CD player',
 486: 'cello, violoncello',
 487: 'cellular telephone, cellular phone, cellphone, cell, mobile phone',
 488: 'chain',
 489: 'chainlink fence',
 490: 'chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour',
 491: 'chain saw, chainsaw',
 492: 'chest',
 493: 'chiffonier, commode',
 494: 'chime, bell, gong',
 495: 'china cabinet, china closet',
 496: 'Christmas stocking',
 497: 'church, church building',
 498: 'cinema, movie theater, movie theatre, movie house, picture palace',
 499: 'cleaver, meat cleaver, chopper',
 500: 'cliff dwelling',
 501: 'cloak',
 502: 'clog, geta, patten, sabot',
 503: 'cocktail shaker',
 504: 'coffee mug',
 505: 'coffeepot',
 506: 'coil, spiral, volute, whorl, helix',
 507: 'combination lock',
 508: 'computer keyboard, keypad',
 509: 'confectionery, confectionary, candy store',
 510: 'container ship, containership, container vessel',
 511: 'convertible',
 512: 'corkscrew, bottle screw',
 513: 'cornet, horn, trumpet, trump',
 514: 'cowboy boot',
 515: 'cowboy hat, ten-gallon hat',
 516: 'cradle',
 517: 'crane',
 518: 'crash helmet',
 519: 'crate',
 520: 'crib, cot',
 521: 'Crock Pot',
 522: 'croquet ball',
 523: 'crutch',
 524: 'cuirass',
 525: 'dam, dike, dyke',
 526: 'desk',
 527: 'desktop computer',
 528: 'dial telephone, dial phone',
 529: 'diaper, nappy, napkin',
 530: 'digital clock',
 531: 'digital watch',
 532: 'dining table, board',
 533: 'dishrag, dishcloth',
 534: 'dishwasher, dish washer, dishwashing machine',
 535: 'disk brake, disc brake',
 536: 'dock, dockage, docking facility',
 537: 'dogsled, dog sled, dog sleigh',
 538: 'dome',
 539: 'doormat, welcome mat',
 540: 'drilling platform, offshore rig',
 541: 'drum, membranophone, tympan',
 542: 'drumstick',
 543: 'dumbbell',
 544: 'Dutch oven',
 545: 'electric fan, blower',
 546: 'electric guitar',
 547: 'electric locomotive',
 548: 'entertainment center',
 549: 'envelope',
 550: 'espresso maker',
 551: 'face powder',
 552: 'feather boa, boa',
 553: 'file, file cabinet, filing cabinet',
 554: 'fireboat',
 555: 'fire engine, fire truck',
 556: 'fire screen, fireguard',
 557: 'flagpole, flagstaff',
 558: 'flute, transverse flute',
 559: 'folding chair',
 560: 'football helmet',
 561: 'forklift',
 562: 'fountain',
 563: 'fountain pen',
 564: 'four-poster',
 565: 'freight car',
 566: 'French horn, horn',
 567: 'frying pan, frypan, skillet',
 568: 'fur coat',
 569: 'garbage truck, dustcart',
 570: 'gasmask, respirator, gas helmet',
 571: 'gas pump, gasoline pump, petrol pump, island dispenser',
 572: 'goblet',
 573: 'go-kart',
 574: 'golf ball',
 575: 'golfcart, golf cart',
 576: 'gondola',
 577: 'gong, tam-tam',
 578: 'gown',
 579: 'grand piano, grand',
 580: 'greenhouse, nursery, glasshouse',
 581: 'grille, radiator grille',
 582: 'grocery store, grocery, food market, market',
 583: 'guillotine',
 584: 'hair slide',
 585: 'hair spray',
 586: 'half track',
 587: 'hammer',
 588: 'hamper',
 589: 'hand blower, blow dryer, blow drier, hair dryer, hair drier',
 590: 'hand-held computer, hand-held microcomputer',
 591: 'handkerchief, hankie, hanky, hankey',
 592: 'hard disc, hard disk, fixed disk',
 593: 'harmonica, mouth organ, harp, mouth harp',
 594: 'harp',
 595: 'harvester, reaper',
 596: 'hatchet',
 597: 'holster',
 598: 'home theater, home theatre',
 599: 'honeycomb',
 600: 'hook, claw',
 601: 'hoopskirt, crinoline',
 602: 'horizontal bar, high bar',
 603: 'horse cart, horse-cart',
 604: 'hourglass',
 605: 'iPod',
 606: 'iron, smoothing iron',
 607: "jack-o'-lantern",
 608: 'jean, blue jean, denim',
 609: 'jeep, landrover',
 610: 'jersey, T-shirt, tee shirt',
 611: 'jigsaw puzzle',
 612: 'jinrikisha, ricksha, rickshaw',
 613: 'joystick',
 614: 'kimono',
 615: 'knee pad',
 616: 'knot',
 617: 'lab coat, laboratory coat',
 618: 'ladle',
 619: 'lampshade, lamp shade',
 620: 'laptop, laptop computer',
 621: 'lawn mower, mower',
 622: 'lens cap, lens cover',
 623: 'letter opener, paper knife, paperknife',
 624: 'library',
 625: 'lifeboat',
 626: 'lighter, light, igniter, ignitor',
 627: 'limousine, limo',
 628: 'liner, ocean liner',
 629: 'lipstick, lip rouge',
 630: 'Loafer',
 631: 'lotion',
 632: 'loudspeaker, speaker, speaker unit, loudspeaker system, speaker system',
 633: "loupe, jeweler's loupe",
 634: 'lumbermill, sawmill',
 635: 'magnetic compass',
 636: 'mailbag, postbag',
 637: 'mailbox, letter box',
 638: 'maillot',
 639: 'maillot, tank suit',
 640: 'manhole cover',
 641: 'maraca',
 642: 'marimba, xylophone',
 643: 'mask',
 644: 'matchstick',
 645: 'maypole',
 646: 'maze, labyrinth',
 647: 'measuring cup',
 648: 'medicine chest, medicine cabinet',
 649: 'megalith, megalithic structure',
 650: 'microphone, mike',
 651: 'microwave, microwave oven',
 652: 'military uniform',
 653: 'milk can',
 654: 'minibus',
 655: 'miniskirt, mini',
 656: 'minivan',
 657: 'missile',
 658: 'mitten',
 659: 'mixing bowl',
 660: 'mobile home, manufactured home',
 661: 'Model T',
 662: 'modem',
 663: 'monastery',
 664: 'monitor',
 665: 'moped',
 666: 'mortar',
 667: 'mortarboard',
 668: 'mosque',
 669: 'mosquito net',
 670: 'motor scooter, scooter',
 671: 'mountain bike, all-terrain bike, off-roader',
 672: 'mountain tent',
 673: 'mouse, computer mouse',
 674: 'mousetrap',
 675: 'moving van',
 676: 'muzzle',
 677: 'nail',
 678: 'neck brace',
 679: 'necklace',
 680: 'nipple',
 681: 'notebook, notebook computer',
 682: 'obelisk',
 683: 'oboe, hautboy, hautbois',
 684: 'ocarina, sweet potato',
 685: 'odometer, hodometer, mileometer, milometer',
 686: 'oil filter',
 687: 'organ, pipe organ',
 688: 'oscilloscope, scope, cathode-ray oscilloscope, CRO',
 689: 'overskirt',
 690: 'oxcart',
 691: 'oxygen mask',
 692: 'packet',
 693: 'paddle, boat paddle',
 694: 'paddlewheel, paddle wheel',
 695: 'padlock',
 696: 'paintbrush',
 697: "pajama, pyjama, pj's, jammies",
 698: 'palace',
 699: 'panpipe, pandean pipe, syrinx',
 700: 'paper towel',
 701: 'parachute, chute',
 702: 'parallel bars, bars',
 703: 'park bench',
 704: 'parking meter',
 705: 'passenger car, coach, carriage',
 706: 'patio, terrace',
 707: 'pay-phone, pay-station',
 708: 'pedestal, plinth, footstall',
 709: 'pencil box, pencil case',
 710: 'pencil sharpener',
 711: 'perfume, essence',
 712: 'Petri dish',
 713: 'photocopier',
 714: 'pick, plectrum, plectron',
 715: 'pickelhaube',
 716: 'picket fence, paling',
 717: 'pickup, pickup truck',
 718: 'pier',
 719: 'piggy bank, penny bank',
 720: 'pill bottle',
 721: 'pillow',
 722: 'ping-pong ball',
 723: 'pinwheel',
 724: 'pirate, pirate ship',
 725: 'pitcher, ewer',
 726: "plane, carpenter's plane, woodworking plane",
 727: 'planetarium',
 728: 'plastic bag',
 729: 'plate rack',
 730: 'plow, plough',
 731: "plunger, plumber's helper",
 732: 'Polaroid camera, Polaroid Land camera',
 733: 'pole',
 734: 'police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria',
 735: 'poncho',
 736: 'pool table, billiard table, snooker table',
 737: 'pop bottle, soda bottle',
 738: 'pot, flowerpot',
 739: "potter's wheel",
 740: 'power drill',
 741: 'prayer rug, prayer mat',
 742: 'printer',
 743: 'prison, prison house',
 744: 'projectile, missile',
 745: 'projector',
 746: 'puck, hockey puck',
 747: 'punching bag, punch bag, punching ball, punchball',
 748: 'purse',
 749: 'quill, quill pen',
 750: 'quilt, comforter, comfort, puff',
 751: 'racer, race car, racing car',
 752: 'racket, racquet',
 753: 'radiator',
 754: 'radio, wireless',
 755: 'radio telescope, radio reflector',
 756: 'rain barrel',
 757: 'recreational vehicle, RV, R.V.',
 758: 'reel',
 759: 'reflex camera',
 760: 'refrigerator, icebox',
 761: 'remote control, remote',
 762: 'restaurant, eating house, eating place, eatery',
 763: 'revolver, six-gun, six-shooter',
 764: 'rifle',
 765: 'rocking chair, rocker',
 766: 'rotisserie',
 767: 'rubber eraser, rubber, pencil eraser',
 768: 'rugby ball',
 769: 'rule, ruler',
 770: 'running shoe',
 771: 'safe',
 772: 'safety pin',
 773: 'saltshaker, salt shaker',
 774: 'sandal',
 775: 'sarong',
 776: 'sax, saxophone',
 777: 'scabbard',
 778: 'scale, weighing machine',
 779: 'school bus',
 780: 'schooner',
 781: 'scoreboard',
 782: 'screen, CRT screen',
 783: 'screw',
 784: 'screwdriver',
 785: 'seat belt, seatbelt',
 786: 'sewing machine',
 787: 'shield, buckler',
 788: 'shoe shop, shoe-shop, shoe store',
 789: 'shoji',
 790: 'shopping basket',
 791: 'shopping cart',
 792: 'shovel',
 793: 'shower cap',
 794: 'shower curtain',
 795: 'ski',
 796: 'ski mask',
 797: 'sleeping bag',
 798: 'slide rule, slipstick',
 799: 'sliding door',
 800: 'slot, one-armed bandit',
 801: 'snorkel',
 802: 'snowmobile',
 803: 'snowplow, snowplough',
 804: 'soap dispenser',
 805: 'soccer ball',
 806: 'sock',
 807: 'solar dish, solar collector, solar furnace',
 808: 'sombrero',
 809: 'soup bowl',
 810: 'space bar',
 811: 'space heater',
 812: 'space shuttle',
 813: 'spatula',
 814: 'speedboat',
 815: "spider web, spider's web",
 816: 'spindle',
 817: 'sports car, sport car',
 818: 'spotlight, spot',
 819: 'stage',
 820: 'steam locomotive',
 821: 'steel arch bridge',
 822: 'steel drum',
 823: 'stethoscope',
 824: 'stole',
 825: 'stone wall',
 826: 'stopwatch, stop watch',
 827: 'stove',
 828: 'strainer',
 829: 'streetcar, tram, tramcar, trolley, trolley car',
 830: 'stretcher',
 831: 'studio couch, day bed',
 832: 'stupa, tope',
 833: 'submarine, pigboat, sub, U-boat',
 834: 'suit, suit of clothes',
 835: 'sundial',
 836: 'sunglass',
 837: 'sunglasses, dark glasses, shades',
 838: 'sunscreen, sunblock, sun blocker',
 839: 'suspension bridge',
 840: 'swab, swob, mop',
 841: 'sweatshirt',
 842: 'swimming trunks, bathing trunks',
 843: 'swing',
 844: 'switch, electric switch, electrical switch',
 845: 'syringe',
 846: 'table lamp',
 847: 'tank, army tank, armored combat vehicle, armoured combat vehicle',
 848: 'tape player',
 849: 'teapot',
 850: 'teddy, teddy bear',
 851: 'television, television system',
 852: 'tennis ball',
 853: 'thatch, thatched roof',
 854: 'theater curtain, theatre curtain',
 855: 'thimble',
 856: 'thresher, thrasher, threshing machine',
 857: 'throne',
 858: 'tile roof',
 859: 'toaster',
 860: 'tobacco shop, tobacconist shop, tobacconist',
 861: 'toilet seat',
 862: 'torch',
 863: 'totem pole',
 864: 'tow truck, tow car, wrecker',
 865: 'toyshop',
 866: 'tractor',
 867: 'trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi',
 868: 'tray',
 869: 'trench coat',
 870: 'tricycle, trike, velocipede',
 871: 'trimaran',
 872: 'tripod',
 873: 'triumphal arch',
 874: 'trolleybus, trolley coach, trackless trolley',
 875: 'trombone',
 876: 'tub, vat',
 877: 'turnstile',
 878: 'typewriter keyboard',
 879: 'umbrella',
 880: 'unicycle, monocycle',
 881: 'upright, upright piano',
 882: 'vacuum, vacuum cleaner',
 883: 'vase',
 884: 'vault',
 885: 'velvet',
 886: 'vending machine',
 887: 'vestment',
 888: 'viaduct',
 889: 'violin, fiddle',
 890: 'volleyball',
 891: 'waffle iron',
 892: 'wall clock',
 893: 'wallet, billfold, notecase, pocketbook',
 894: 'wardrobe, closet, press',
 895: 'warplane, military plane',
 896: 'washbasin, handbasin, washbowl, lavabo, wash-hand basin',
 897: 'washer, automatic washer, washing machine',
 898: 'water bottle',
 899: 'water jug',
 900: 'water tower',
 901: 'whiskey jug',
 902: 'whistle',
 903: 'wig',
 904: 'window screen',
 905: 'window shade',
 906: 'Windsor tie',
 907: 'wine bottle',
 908: 'wing',
 909: 'wok',
 910: 'wooden spoon',
 911: 'wool, woolen, woollen',
 912: 'worm fence, snake fence, snake-rail fence, Virginia fence',
 913: 'wreck',
 914: 'yawl',
 915: 'yurt',
 916: 'web site, website, internet site, site',
 917: 'comic book',
 918: 'crossword puzzle, crossword',
 919: 'street sign',
 920: 'traffic light, traffic signal, stoplight',
 921: 'book jacket, dust cover, dust jacket, dust wrapper',
 922: 'menu',
 923: 'plate',
 924: 'guacamole',
 925: 'consomme',
 926: 'hot pot, hotpot',
 927: 'trifle',
 928: 'ice cream, icecream',
 929: 'ice lolly, lolly, lollipop, popsicle',
 930: 'French loaf',
 931: 'bagel, beigel',
 932: 'pretzel',
 933: 'cheeseburger',
 934: 'hotdog, hot dog, red hot',
 935: 'mashed potato',
 936: 'head cabbage',
 937: 'broccoli',
 938: 'cauliflower',
 939: 'zucchini, courgette',
 940: 'spaghetti squash',
 941: 'acorn squash',
 942: 'butternut squash',
 943: 'cucumber, cuke',
 944: 'artichoke, globe artichoke',
 945: 'bell pepper',
 946: 'cardoon',
 947: 'mushroom',
 948: 'Granny Smith',
 949: 'strawberry',
 950: 'orange',
 951: 'lemon',
 952: 'fig',
 953: 'pineapple, ananas',
 954: 'banana',
 955: 'jackfruit, jak, jack',
 956: 'custard apple',
 957: 'pomegranate',
 958: 'hay',
 959: 'carbonara',
 960: 'chocolate sauce, chocolate syrup',
 961: 'dough',
 962: 'meat loaf, meatloaf',
 963: 'pizza, pizza pie',
 964: 'potpie',
 965: 'burrito',
 966: 'red wine',
 967: 'espresso',
 968: 'cup',
 969: 'eggnog',
 970: 'alp',
 971: 'bubble',
 972: 'cliff, drop, drop-off',
 973: 'coral reef',
 974: 'geyser',
 975: 'lakeside, lakeshore',
 976: 'promontory, headland, head, foreland',
 977: 'sandbar, sand bar',
 978: 'seashore, coast, seacoast, sea-coast',
 979: 'valley, vale',
 980: 'volcano',
 981: 'ballplayer, baseball player',
 982: 'groom, bridegroom',
 983: 'scuba diver',
 984: 'rapeseed',
 985: 'daisy',
 986: "yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum",
 987: 'corn',
 988: 'acorn',
 989: 'hip, rose hip, rosehip',
 990: 'buckeye, horse chestnut, conker',
 991: 'coral fungus',
 992: 'agaric',
 993: 'gyromitra',
 994: 'stinkhorn, carrion fungus',
 995: 'earthstar',
 996: 'hen-of-the-woods, hen of the woods, Polyporus frondosus, Grifola frondosa',
 997: 'bolete',
 998: 'ear, spike, capitulum',
 999: 'toilet tissue, toilet paper, bathroom tissue'}

In [ ]:
df_off_resnet = pd.read_csv('/content/Office_Topic 2.csv')

In [ ]:
df_off_resnet['Object'] = df_off_resnet['Object'].map(imagenet) 

In [ ]:
df_off_resnet

In [ ]:
def create_plot_topics(filename,place_name,topic_name):
  df_off_resnet = pd.read_csv(filename)
  df_off_resnet['Object'] = df_off_resnet['Object'].map(imagenet) 
  topic = topic_name
  place = place_name
  topic = topic.replace(' ','')
  image_name = place+'_'+topic+'.png'
  df_off_resnet.Object.value_counts()[:10].sort_values().plot(kind = 'barh')
  plt.xlabel('Number of Images')
  plt.ylabel('Objects')
  plt.title('Top 10 Objects highlighted by '+topic+' Layer for Place: '+place)
  #plt.xticks([])
  #plt.show()
  plt.savefig(image_name,bbox_inches="tight")
  files.download(image_name)

In [ ]:
create_plot_topics('/content/Transportation_Topic 1.csv','Transportation','Topic 1')

In [ ]:
files.download('/content/Office_Topic 3.png')

In [ ]:
dat = ['Noise']

In [ ]:
c = pd.read_csv('/content/eval (2).csv')

In [ ]:
b = pd.read_csv('/content/val_updated (2).csv')

In [ ]:
a = pd.read_csv('/content/test_final.csv')

In [ ]:
a = a[~a['Label'].isin(dat)]
a.to_csv('test_nn_final.csv')
files.download('test_nn_final.csv')

In [ ]:
a[a['Label']=='Noise']

In [ ]:
a = pd.read_csv('/content/test_nn_final.csv')
len(a)

#Creating a collage

In [ ]:
content_list = os.listdir('/content')
image_list = []
for content in content_list:
  if content.endswith('.jpg'):
    image_list.append(content)

In [ ]:
image_list = sorted(image_list,)

In [ ]:
imshow(image_list[0])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def display_multiple_img(images, rows = 1, cols=1):
    figure, ax = plt.subplots(nrows=rows,ncols=cols )
    for ind,title in enumerate(images):
        ax.ravel()[ind].imshow(images[title])
        ax.ravel()[ind].set_title(title)
        ax.ravel()[ind].set_axis_off()
    plt.tight_layout()
    plt.show()


In [ ]:
for im in image_list:
  image = Image.open(im)

  x,y = image.size
  new_dimensions = (224, 224) #dimension set here
  output = image.resize(new_dimensions, Image.ANTIALIAS)

  output_file_name = os.path.join('/content/small',"small_" + im)
  output.save(output_file_name, "JPEG", quality = 95)

In [ ]:
small  = os.listdir('/content/small')
small = '/content/small'+small
for sm in small:
  files.download(sm)

# Using TFDS

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
    w_init = tf.random_normal_initializer()
    self.w1 = self.add_weight(
        shape=(self.units), dtype="float32"),trainable=True,name = 'weight_topic_1',initializer = 'random_normal'
    
   
    self.w2 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_2',initializer = 'random_normal'
    )  
    self.w3 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_3',initializer = 'random_normal'
    )       
    self.w4 = self.add_weight(
        shape=(self.units), dtype="float32",trainable=True,name = 'weight_topic_4',initializer = 'random_normal'
    )
  def call(self, input1, input2, input3,input4):
    #name = 'tops'
    return tf.multiply(input2,self.w1) + tf.multiply(input1, self.w2) + tf.multiply(input3, self.w3)+tf.multiply(input4, self.w4)

In [ ]:
class Wt_Topics(keras.layers.Layer):
  def __init__(self, units):
    super(Wt_Topics, self).__init__()
    self.units = units 
    # self.name = name      

  def build(self,input_shape):
    
    w_init = tf.random_normal_initializer()
    self.w1 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_1'
    )
    self.w2 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_2'
    )  
    self.w3 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_3'
    )       
    self.w4 = tf.Variable(
        initial_value=w_init(shape=(1, self.units), dtype="float32"),
        trainable=True,name = 'weight_topic_4'
    )
  def call(self, input1, input2, input3,input4):
    #name = 'tops'
    return tf.multiply(input1,self.w1) + tf.multiply(input2, self.w2) + tf.multiply(input3, self.w3)+tf.multiply(input4, self.w4)

In [ ]:
class Test_Topics(keras.layers.Layer):
    def __init__(self, units):
        super(Test_Topics, self).__init__()
        self.units = units
    def build(self,input_shape):
        self.w1 = self.add_weight(name='multiply_weight_1', shape=(self.units,),
                                  trainable=True,initializer='random_normal')
        self.w2 = self.add_weight(name='multiply_weight_2', shape=(self.units,), 
                                  trainable=True,initializer='random_normal')
        self.w3 = self.add_weight(name='multiply_weight_3', shape=(self.units,), 
                                  trainable=True,initializer='random_normal')
        self.w4 = self.add_weight(name='multiply_weight_4', shape=(self.units,), 
                                  trainable=True,initializer='random_normal')
        
    def call(self, input1,input2,input3,input4):
        #Weights taken individually  
        return tf.multiply(input1, self.w1),tf.multiply(input2, self.w2),tf.multiply(input3, self.w3),tf.multiply(input4, self.w4)



In [ ]:
class Topics_New(keras.layers.Layer):
    def __init__(self, units):
        super(Topics_New, self).__init__()
        self.units = units
    def build(self,input_shape):
        self.w = self.add_weight(name='multiply_weight', shape=(1,self.units), trainable=True,initializer='random_normal')
        
        #b_init = tf.zeros_initializer()
        #self.b = tf.Variable(
        #    initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        #)
        #self.add = tf.keras.layers.Add()
    def call(self, inputs):
          #topic = []
          #for input in inputs:
          #  weights = tf.multiply(input, self.w)
          #  topic.append(weights) #+ self.b)
        
        return tf.keras.layers.Multiply()([inputs,self.w])

In [ ]:
#Using the output from the resnet model in order to create topics
#Defining the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name ="flatten")(x)
x = Dense(1024,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model

model = Model(inputs = base_model.input,outputs = preds)

#predictions = Dense(26,activation='softmax')(x)
#Using softmax for base classification
#model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    layer.trainable = False


wt_add = Test_Topics(1)
sum_layer = wt_add(model.output,model.output,model.output,model.output)

conc = keras.layers.Concatenate()(sum_layer)
#flat = keras.layers.GlobalAveragePooling1D()(conc)
#test = keras.layers.GlobalAveragePooling1D()(flat)
flat_2 = keras.layers.Flatten(name = 'flat_2')(conc)
topic = Dense(128,activation='relu')(flat_2)
topic = Dropout(0.2)(topic)
final = Dense(24,activation='softmax')(topic)


cent_model_c = Model(inputs = base_model.input,outputs = final)
opti = tf.keras.optimizers.Adadelta(learning_rate=0.01, rho=0.95, epsilon=1e-07, name="Adadelta")
cent_model_c.compile(optimizer=opti,metrics=[tf.keras.metrics.CategoricalAccuracy(),keras.metrics.Precision(),keras.metrics.Recall()],loss='categorical_crossentropy')

In [ ]:
wt_add = Test_Topics(4)([k_1,k_2,k_3,k_4])
wt_add.shape
tf.reshape(wt_add, shape=[tf.shape(wt_add)[0]*tf.shape(wt_add)[1],4])

In [ ]:

#  np.random.randn(cent_model_c.layers[-6].get_weights().shape())
# np.random.randn(*self.model.layers[layer_index].get_weights().shape
#weights_shape = cent_model_c.layers[-6].get_weights()
a = [np.random.rand(1),np.random.rand(1),np.random.rand(1),np.random.rand(1)]
#weights_shape
#a
#cent_model_c.layers[-6].set_weights(a)
a

In [ ]:
cent_model_c.layers[-6].get_weights()

In [ ]:
cent_model_c.summary()

In [ ]:
tf.keras.utils.plot_model(
    cent_model_c, to_file='/content/plot_model_2.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

#plot_model(cent_model_c, to_file='/content/plot/plot.png')

In [ ]:
cent_model_c.layers[-6].get_weights()

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
train_df,info = tfds.load('oxford_flowers102',split='train',shuffle_files=True,with_info=True,as_supervised=True)
val_df,info = tfds.load('oxford_flowers102',split='validation',shuffle_files=True,with_info=True,as_supervised=True)

In [ ]:
for image,label in train_df.take(1):
  print(image)
  print(label)

In [ ]:
size = (224,224)
train_df = train_df.map(lambda x,y:(tf.image.resize(x,size),tf.one_hot(y,102)))
val_df = val_df.map(lambda x, y: (tf.image.resize(x, size), tf.one_hot(y,102)))
# info

In [ ]:
train_df

In [ ]:
# a = tf.keras.utils.to_categorical(, num_classes=4)
train_df = train_df.map(lambda x,y: (tf.cast(x/255.,tf.float32),y))
val_df = val_df.map(lambda x,y: (tf.cast(x/255.,tf.float32),y))

In [ ]:
for image,labels in train_df.take(1):
  print(image)
  #labels = tf.one_hot(labels,depth=len(classes))
  #print(labels)

In [ ]:
batch_size = 16

train_ds = train_df.cache('cache_t').batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = val_df.cache('cache_v').batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
train_ds

In [ ]:
data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
#if not isfile(json_log):

json_log = open('topic_k9.json', mode='wt', buffering=1)
callbacks = [
  ModelCheckpoint(
    
    filepath='/content/Flowers102',
    save_best_only=True,  # Only save a model if `val_loss` has improved.
    monitor="categorical_accuracy",
    verbose=1,
    save_freq = 'epoch',mode = 'max'
  ),
  EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=7,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
  ),
  ReduceLROnPlateau(monitor='loss', factor=0.1,
                          patience=3, min_lr=1e-5,verbose=1),
  
  LambdaCallback(
            on_epoch_end=lambda epoch, logs: json_log.write(
                json.dumps({'epoch': epoch, 
                            'categorical_accuracy': logs['categorical_accuracy'],
                            'loss': logs['loss'],
                            'topic_layer_weights': list(cent_model_c.layers[-6].get_weights())},cls = NumpyArrayEncoder) + '\n'),
            on_train_end=lambda logs: json_log.close()
  )

         
]


In [ ]:
wt_add

In [ ]:
cent_model_c.layers[-8].get_weights()

In [ ]:
# Create new model on top
#cant_model_c = keras.models.load_model('/content/Flowers102')
history = cent_model_c.fit(train_ds,epochs=10,verbose = 1,validation_data=val_ds,use_multiprocessing=False,workers = 2,callbacks = callbacks)


In [ ]:
cent_model_c.save('/content/Flowers102')

In [ ]:
tfds.list_builders(
) 

In [ ]:
for l in cent_model_c.layers:
    print(l.name, l.trainable)

In [ ]:
cent_model_c.layers[-4].get_weights()

In [ ]:
import json
# file = 

# 
test = []
for line in open('/content/json_now_cw_tk4_fds.json','r'):
    test.append(json.loads(line))

In [ ]:
data = [json.loads(line) for line in open('/content/json_now_cw_tk4_fds.json', 'r')]

In [ ]:
data

In [ ]:
test_dict = {}
test_2 = []
for elements in data:
  test_dict.update(elements)

In [ ]:
from functools import reduce
abc = reduce(lambda a, b: dict(a, **b), data)

In [ ]:
d = {k:v for x in data for k,v in x.items()}

In [ ]:
import pandas as pd
df_data = pd.DataFrame(data)

In [ ]:
text = df_data.head(1)
# for el in text:
#   for els in el:
#     print(els)
text

In [ ]:
abc = df_data['topic_layer_weights']

In [ ]:
pd.DataFrame(abc)
abc = abc.map(lambda x:str(x).replace('[[',''))
abc = abc.map(lambda x:str(x).replace(']]',''))
abc = abc.map(lambda x:str(x).replace('[',''))
abc = abc.map(lambda x:str(x).replace(']',''))

abc = abc.str.split(',',expand=True)

abc.columns = ['Topic 1','Topic 2','Topic 3','Topic 4']


In [ ]:
abc.columns = ['Topic 1','Topic 2','Topic 3','Topic 4']
abc

In [ ]:
testing = pd.merge(df_data,abc,how='inner',on=df_data.index)

In [ ]:
final_data = testing[['batch','categorical_accuracy','loss','Topic 1','Topic 2','Topic 3','Topic 4']]

##Designing a function to convert JSON to a dataframe


In [ ]:
def topic_col_to_df(frame):
  pd.DataFrame(frame)
  frame = frame.map(lambda x:str(x).replace('[[',''))
  frame = frame.map(lambda x:str(x).replace(']]',''))
  frame = frame.map(lambda x:str(x).replace('[',''))
  frame = frame.map(lambda x:str(x).replace(']',''))
  frame = frame.map(lambda x:str(x).replace(']]]',''))
  frame = frame.map(lambda x:str(x).replace('[[[',''))

  frame = frame.str.split(',',expand=True)

  frame.columns = ['Topic 1','Topic 2','Topic 3','Topic 4']
  return frame


In [ ]:
def make_topic_df(filename):
  data = [json.loads(line) for line in open(filename, 'r')]
  df_data = pd.DataFrame(data)
  df_data['epoch'] = df_data['epoch']+1 
  topic_column = df_data['topic_layer_weights']
  topic_df = topic_col_to_df(topic_column)
  testing = pd.merge(df_data,topic_df,how='inner',on=df_data.index)
  final_data = testing[['epoch','categorical_accuracy','loss','Topic 1','Topic 2','Topic 3','Topic 4']]
  return final_data

In [ ]:
final_data = make_topic_df('/content/MODEL_1_INIT.json')

In [ ]:
def plot_topics(df):
  topic_list = ['Topic 1','Topic 2','Topic 3','Topic 4']
  for topic in topic_list:
    weight = []
    #Topic Weights
    y = df[topic].tolist()
    x = df['epoch'].tolist()
    for el in y:
      
      weight.append(float(el))
    ax = sn.barplot(x=x, y=weight, data=df)
    plt.xlabel("Epoch")
    plt.ylabel("Topic Weight Value")
    plt.title("Weight Distribution for "+topic)
    plt.savefig('weights_'+topic+'.png')#Place in plot dir plt.savefig(os.path.join(plot_dir,'weights_'+topic+'.png')) 
    plt.close()
    print(topic+' Weights plotted...')
    #Accuracy
    
    plt.plot(df['categorical_accuracy'])
    plt.ylabel('Categorical Accuracy')
    plt.xlabel('Epoch')
    plt.title('Accuracy with respect to '+topic)
    plt.savefig('acc_'+topic+'.png')#Place in plot dir plt.savefig(os.path.join(plot_dir,'acc_'+topic+'.png'))
    plt.close()
    print(topic+' Accuracy plotted...')
    #Loss
    plt.plot(df['loss'])
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.title('Loss with respect to '+topic)
    plt.savefig('loss_'+topic+'.png')#Place in plot dir  plt.savefig(os.path.join(plot_dir,'loss_'+topic+'.png'))
    plt.close()
    print(topic+' Loss plotted...')


In [ ]:
plot_topics(final_data)


In [ ]:
plt.plot(final_data['categorical_accuracy'])
plt.plot(final_data['loss'], color='g', linestyle='--')
plt.plot(read_test['categorical_accuracy'])
plt.title('model accuracy for fold_'+str(get_fold))
plt.ylabel('Categorical Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(os.path.join(folder_path,'accuracy_'+str(history_path)+'.png'))
plt.clf()

In [ ]:

import seaborn as sns
#sns.set_style('darkgrid')
topic_list = ['Topic 1','Topic 2','Topic 3','Topic 4']
for topic in topic_list:
  weight = []
  y = final_data[topic].tolist()
  x = final_data['epoch'].tolist()
  for el in y:
    
    weight.append(float(el))
  ax = sns.barplot(x=x, y=weight, data=final_data)
  plt.xlabel("Epoch")
  plt.ylabel("Topic Weight Value")
  plt.title("Weight Distribution for "+topic)
  plt.savefig('weights_'+topic)

In [ ]:
plt.plot(final_data['categorical_accuracy'])
#plt.plot(final_data['loss'])
plt.ylabel('Categorical Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy with respect to Topic 1')
plt.show()

In [ ]:
#test_df = final_data.iloc[:,:]
weight = []
y = final_data['Topic 1'].tolist()
x = final_data['epoch'].tolist()
for el in y:
  # el = "{:.7f}".format(el)
  weight.append(float(el))

#Testing stuff

##Scrap

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = AveragePooling2D(pool_size=(7,7))(x)
x = Flatten(name ="flatten")(x)
x = Dense(128,activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1000,activation="softmax")(x)

#Activating the model
for layer in base_model.layers:
  layer.trainable = False

model = Model(inputs = base_model.input,outputs = preds)
opti = tf.keras.optimizers.Adam(
    learning_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)
cce = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opti,loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),metrics=[keras.metrics.Precision(),keras.metrics.CategoricalAccuracy()])

In [ ]:
data = np.array(Image.open('/content/drive/MyDrive/images_routine_users/user_01/user1_01/20171201_115957_000.jpg'))
data.resize(224,224,3)


In [ ]:
img_pil = array_to_img(data)
img_arr = img_to_array(img_pil)
img_arr = img_arr.reshape((1,224,224,3))
predictions = model.predict(img_arr)


In [ ]:
def create_topics(predictions):
  topics = decode_predictions(predictions,top = 1000)
  t = pd.DataFrame(topics)
  lis = []
  for top in t:
    a,b,c = t[top].explode()
    lis.append(b)
  return lis

In [ ]:
tops = []
topss = []
for i in range(0,4):
  predictions = model.predict(img_arr)
  tops = create_topics(predictions)
  topss.append(tops)

In [ ]:
predictions

In [ ]:
y = model.output
k_1 = Dense(1000,name = 'topic_1')(y)
k_2 = Dense(1000,name = 'topic_2')(y)
k_3 = Dense(1000,name = 'topic_3')(y)
k_4 = Dense(1000,name = 'topic_4')(y)
conc2 = Concatenate(name = 'conc_c')([k_1,k_2,k_3,k_4])
flat_2 = Flatten(name = 'flat_c')(conc2)
#Adding the Dense layers now
topic = Dense(4000,activation='relu')(flat_2)
topic = Dropout(0.5)(topic)
topic = Dense(4000,activation='relu')(topic)
topic = Dropout(0.5)(topic)
final = Dense(24,activation='softmax')(topic)

In [ ]:
cent_model_c.summary()

In [ ]:
cent_model_c = Model(inputs = base_model.input,outputs = final)

In [ ]:
model.compile(optimizer=opti,loss='categorical_crossentropy',metrics=[keras.metrics.Precision(),keras.metrics.CategoricalAccuracy()])
cent_model_c.compile(optimizer=opti,loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.Precision()])

In [ ]:
layer = cent_model_c.get_layer('topic_1')
weights = layer.get_weights()
weights

In [ ]:
cent_model_c.summary()

In [ ]:
predi = cent_model_c.predict(img_arr)
pre = np.argmax(predi)
pre

In [ ]:
%pip install keras-metrics


In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [ ]:
train_dir = '/content/gdrive/MyDrive/data/train'
val_dir = '/content/gdrive/MyDrive/data/val'
test_dir = '/content/gdrive/MyDrive/data/test'


In [ ]:


#getting the number of files in train and val dataset
train_num_files=len([file for file in glob(str(train_dir + '/*/*'))])
val_num_files=len([file for file in glob(str(val_dir + '/*/*'))])
print("No. of files in Train folder: ",train_num_files)
print("No. of files in Val folder: ",val_num_files)

In [ ]:
#Try using image data generator
datagen = ImageDataGenerator(rescale=1./255)
train = datagen.flow_from_directory('/content/gdrive/MyDrive/data/train',target_size=(224,224),color_mode='rgb',batch_size=151,shuffle=True,class_mode='categorical',seed=42)
STEP_SIZE_TRAIN = train.n//train.batch_size

In [ ]:
images, labels = next(datagen.flow_from_directory('/content/gdrive/MyDrive/data/train',target_size=(224,224),color_mode='rgb',batch_size=151,shuffle=True,class_mode='categorical',seed=42))

#try_img = tf.data.Dataset.from_generator(datagen.flow_from_directory(lambda x:'/content/gdrive/MyDrive/data/train',target_size=(224,224),color_mode='rgb',batch_size=151,shuffle=True,class_mode='categorical',seed=42))

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((next(datagen.flow_from_directory('/content/gdrive/MyDrive/data/train',target_size=(224,224),color_mode='rgb',batch_size=64,shuffle=True,class_mode='categorical',seed=42))))